#Installation

In [ ]:
code_two = ['en','ar','he','ru', 'ko', 'it','ja', 'zh-tw', 'zh-cn', 'es', 'fr', 'pt-br', 'nl', 'tr', 'ro', 'pl', 'bg', 'vi', 'de', 'fa', 'hu', 'sr', 'el', 'hr', 'uk', 'cs', 'th', 'id', 'sk', 'sv', 'pt', 'da', 'sq', 'lt', 'mk', 'fi', 'my', 'hy', 'fr-ca', 'sl', 'hi', 'nb', 'et', 'ku', 'gl', 'mr', 'mn', 'eo', 'ta', 'ur', 'az', 'bs\n', 'zh', 'eu', 'bn', 'be', 'kk', 'ka', 'ms']
code_three = ['eng','arb','heb','rus', 'kor', 'ita','jpn', 'chinese/taiwan', 'chinese/china', 'spa', 'fra', 'portugese/brazil', 'nld', 'tur', 'ron', 'pol', 'bul', 'vie', 'deu', 'pes', 'hun', 'srp', 'ell', 'hrv', 'ukr', 'ces', 'tha', 'ind', 'slk', 'swe','por', 'dan', 'sq/aln', 'lit', 'mkd', 'fin', 'mya', 'hye', 'fr-ca', 'slv', 'hin', 'nob', 'est', 'kur', 'glg', 'mar', 'mon', 'epo', 'tam', 'urd', 'azb', 'bos', 'csl', 'eus', 'ben','bel', 'kaz', 'kat','zsm']

In [ ]:
import random
import numpy as np
import lightgbm as lgb
from sklearn import metrics

In [ ]:
from google.colab import drive
import operator
drive.mount('/content/drive/', force_remount=True)
%cd "/content/drive/My Drive/OpenNMT"

Mounted at /content/drive/
/content/drive/My Drive/OpenNMT


In [ ]:
%cd '/content/drive/My Drive/'
!git clone https://github.com/antonisa/lang2vec
%cd lang2vec
!wget http://www.cs.cmu.edu/~aanastas/files/distances.zip .
!mv distances.zip lang2vec/data/
!python3 setup.py install
import lang2vec.lang2vec as l2v

/content/drive/My Drive
fatal: destination path 'lang2vec' already exists and is not an empty directory.
/content/drive/My Drive/lang2vec
--2021-05-14 08:39:26--  http://www.cs.cmu.edu/~aanastas/files/distances.zip
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275609596 (263M) [application/zip]
Saving to: ‘distances.zip’

distances.zip       100%[===================>] 262.84M   984KB/s    in 5m 47s  

2021-05-14 08:45:14 (775 KB/s) - ‘distances.zip’ saved [275609596/275609596]

--2021-05-14 08:45:14--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2021-05-14 08:45:14--
Total wall clock time: 5m 47s
Downloaded: 1 files, 263M in 5m 47s (775 KB/s)
running install
running bdist_egg
running egg_info
writing lang2vec.egg-info/PKG-INFO
writing dependency_links to lang2vec.eg

In [ ]:
%cd '/content/drive/My Drive/OpenNMT'

/content/drive/My Drive/OpenNMT


#Load TED and URIEL language codes

In [ ]:
# load all lines
lines = []
for line in open('ted_talks/raw_data/all_talks_train.txt', 'r').readlines():
  lines.append(line.split('\t'))

# create list with langauge items
ted_lan = lines[0]

# create dict where keys are indeces, and values are (lan, eng) tuples
ted_dct = {}
for i, lan in enumerate(lines[0][1:]):
  ted_dct[i] = []

for line in lines[1:]:
   for i,sen in enumerate(line[1:]):
      if 'NULL' in sen:
        continue
      elif len(sen)==0:
        continue
      else:
        ted_dct[i].append(sen)

fin_dct = {}
langs = []
for item in lines[0][1:]:
  langs.append(item)

# result: fin_dct with key=lan en value=sentences
for key,value in ted_dct.items():
  fin_dct[langs[key]] = value

# remove 'calv' key as it is unclear which language that is, and valule is emtpy
del fin_dct['calv']

In [ ]:
uriel_lan = l2v.available_uriel_languages()

ted_in_uriel = []
ted_not_in_uriel = []
ted_in_uriel_dct = {}

for code in code_three:
  if code in uriel_lan:
    ted_in_uriel.append(code)
    ted_in_uriel_dct[code] = fin_dct[code_two[code_three.index(code)]]
  else:
    ted_not_in_uriel.append(code)

sorted_tuples = sorted(ted_in_uriel_dct.items(), key=operator.itemgetter(1))
ted_in_uriel_dct = {k: v for k, v in sorted_tuples}

#Obtain LRLs and their sizes

In [ ]:
lrl = {}
for src_lan, sents in ted_in_uriel_dct.items():
  if len(sents)<50000:
    lrl[src_lan]=len(sents)

lrl_in_uriel = {}
for lan in lrl.keys():
  if lan == 'azb':
    continue
  if lan == 'slk':
    continue
  elif lan == 'glg':
    continue
  elif lan == 'bel':
    continue
  elif lan in ted_in_uriel:
    lrl_in_uriel[(code_two[code_three.index(lan)], lan)] = lrl[lan]
sorted_lrl = sorted(lrl_in_uriel.items(), key=operator.itemgetter(1))

#Functions

In [ ]:
''' Return child-helper tuples '''
def obtain_helper_pairs(lan_2code, lan_3code, ted_in_uriel=ted_in_uriel): 
  helper_options = ted_in_uriel.copy()
  if lan_3code in helper_options:
    helper_options.remove(lan_3code)
  if "eng" in helper_options:
    helper_options.remove("eng")

  child_helper_pairs = []
  for helper in helper_options:
    child_helper_pairs.append((lan_3code, helper))
  return child_helper_pairs

''' Return dictionary with data-dependent distance features '''
def data_dep_distances(child_helper_pairs, ted_in_uriel_dct=ted_in_uriel_dct):
  print('  calculate dependent distances ...')
  distance_dct = {}
  child_2code = code_two[code_three.index(child_helper_pairs[0][0])]
  if child_2code == 'bs\n':
    child_2code = 'bs'
  child_tokens = open('uriel/'+child_2code+'_toks.txt').readlines()
  child_tokens = [line.strip('\n') for line in child_tokens]
  child_total_tokens = [w for s in ted_in_uriel_dct[child_helper_pairs[0][0]] for w in s.strip().split()]
  ttr_child = len(child_tokens)/len(child_total_tokens)
  
  for i,pair in enumerate(child_helper_pairs):
    #print("***",i+1, "/" , len(child_helper_pairs), " ", pair,"***")
    distance_dct[pair] = {}

    # data-size ratio
    size_ratio = len(ted_in_uriel_dct[pair[1]])/len(ted_in_uriel_dct[pair[0]])
    distance_dct[pair]['d_size'] = round(size_ratio, 4)

    helper_2code = code_two[code_three.index(pair[1])]
    if helper_2code == 'bs\n':
      helper_2code = 'bs'
    helper_tokens = open('uriel/'+helper_2code+'_toks.txt').readlines()
    helper_tokens = [line.strip('\n') for line in helper_tokens]
    helper_total_tokens = [w for s in ted_in_uriel_dct[pair[1]] for w in s.strip().split()]

    # word overlap
    overlap_distance = len(list(set.intersection(set(child_tokens), set(helper_tokens))))/(len(child_tokens)+len(helper_tokens))
    distance_dct[pair]['d_overlap_w'] = round(overlap_distance, 4)

    # type-token ratio
    ttr_helper = len(helper_tokens)/len(helper_total_tokens)
    ttr_distance = pow(1-(ttr_helper/ttr_child), 2)
    distance_dct[pair]['d_ttr'] = round(ttr_distance,4)

    # sub-word overlap
    distance_dct[pair]['d_overlap_s'] = '-'

  return distance_dct

def data_indep_distances(child_helper_pairs, syntactic_d, phonological_d, inventory_d, family_d, geographical_d, featural_d, distance_dct):
  print('  calculate independent distances ...')
  
  for i,pair in enumerate(child_helper_pairs):
    #print("***",i+1, "/" , len(child_helper_pairs), " ", pair,"***")
    # store syntax distance
    distance_dct[pair]['i_syn'] = syntactic_d[ted_in_uriel.index(pair[0])][ted_in_uriel.index(pair[1])]
    # store phonology distance
    distance_dct[pair]['i_phon'] = phonological_d[ted_in_uriel.index(pair[0])][ted_in_uriel.index(pair[1])]
    # store inventory distance
    distance_dct[pair]['i_inv'] = inventory_d[ted_in_uriel.index(pair[0])][ted_in_uriel.index(pair[1])]
    # store family distance
    distance_dct[pair]['i_fam'] = family_d[ted_in_uriel.index(pair[0])][ted_in_uriel.index(pair[1])]
    # store geo distance
    distance_dct[pair]['i_geo'] = geographical_d[ted_in_uriel.index(pair[0])][ted_in_uriel.index(pair[1])]
    # store featural distance
    distance_dct[pair]['i_fea'] = featural_d[ted_in_uriel.index(pair[0])][ted_in_uriel.index(pair[1])]
    #distance_dct[pair]['i_fea'] = l2v.featural_distance(pair[0], pair[1])
  
  return distance_dct

#Obtain distance dictionary

##Gather typological distance matrices

In [ ]:
syntactic_d = l2v.syntactic_distance(ted_in_uriel)
phonological_d = l2v.phonological_distance(ted_in_uriel)
inventory_d = l2v.inventory_distance(ted_in_uriel)
family_d = l2v.genetic_distance(ted_in_uriel)
geographical_d = l2v.geographic_distance(ted_in_uriel)
featural_d = l2v.featural_distance(ted_in_uriel)

##Create dictionary

In [ ]:
all_lrl_distances = {}
for i,lan in enumerate(lrl_in_uriel.keys()):
  print(i+1,"/", len(lrl_in_uriel.keys()))
  print("LRL under consideration: ", lan[1])
  all_lrl_distances[lan[1]] = {}
  child_helper_pairs = obtain_helper_pairs(lan[0], lan[1])
  dependent_dct = data_dep_distances(child_helper_pairs)
  distance_dct = data_indep_distances(child_helper_pairs, syntactic_d, phonological_d, inventory_d, family_d, geographical_d, featural_d, dependent_dct)
  all_lrl_distances[lan[1]] = distance_dct
  print("\n")

1 / 22
LRL under consideration:  lit
  calculate dependent distances ...
  calculate independent distances ...


2 / 22
LRL under consideration:  slv
  calculate dependent distances ...
  calculate independent distances ...


3 / 22
LRL under consideration:  kaz
  calculate dependent distances ...
  calculate independent distances ...


4 / 22
LRL under consideration:  mkd
  calculate dependent distances ...
  calculate independent distances ...


5 / 22
LRL under consideration:  urd
  calculate dependent distances ...
  calculate independent distances ...


6 / 22
LRL under consideration:  mar
  calculate dependent distances ...
  calculate independent distances ...


7 / 22
LRL under consideration:  mya
  calculate dependent distances ...
  calculate independent distances ...


8 / 22
LRL under consideration:  nob
  calculate dependent distances ...
  calculate independent distances ...


9 / 22
LRL under consideration:  zsm
  calculate dependent distances ...
  calculate independent

In [ ]:
all_lrl_distances =  {'lit': {('lit', 'arb'): {'d_size': 5.1077, 'd_overlap_w': 0.0039, 'd_ttr': 0.2373, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('lit', 'heb'): {'d_size': 5.0531, 'd_overlap_w': 0.0042, 'd_ttr': 0.3177, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6169, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'rus'): {'d_size': 4.9729, 'd_overlap_w': 0.0066, 'd_ttr': 0.3171, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2163, 'i_inv': 0.3825, 'i_fam': 0.5, 'i_geo': 0.2, 'i_fea': 0.5}, ('lit', 'kor'): {'d_size': 4.9057, 'd_overlap_w': 0.0042, 'd_ttr': 0.0001, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5702, 'i_inv': 0.5552, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'ita'): {'d_size': 4.8785, 'd_overlap_w': 0.0245, 'd_ttr': 0.6111, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.641, 'i_inv': 0.4906, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'jpn'): {'d_size': 4.8687, 'd_overlap_w': 0.0013, 'd_ttr': 25.8726, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3824, 'i_inv': 0.5749, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'spa'): {'d_size': 4.6763, 'd_overlap_w': 0.0242, 'd_ttr': 0.6086, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.4278, 'i_inv': 0.5404, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'fra'): {'d_size': 4.5875, 'd_overlap_w': 0.0244, 'd_ttr': 0.6863, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.3918, 'i_inv': 0.4824, 'i_fam': 0.9375, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'nld'): {'d_size': 4.3839, 'd_overlap_w': 0.0214, 'd_ttr': 0.5334, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.641, 'i_inv': 0.4954, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'tur'): {'d_size': 4.3529, 'd_overlap_w': 0.0135, 'd_ttr': 0.1052, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.2952, 'i_inv': 0.4959, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'ron'): {'d_size': 4.3055, 'd_overlap_w': 0.0205, 'd_ttr': 0.4758, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.3217, 'i_inv': 0.4795, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.5}, ('lit', 'pol'): {'d_size': 4.2026, 'd_overlap_w': 0.0166, 'd_ttr': 0.2172, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.2163, 'i_inv': 0.4933, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('lit', 'bul'): {'d_size': 4.1615, 'd_overlap_w': 0.0063, 'd_ttr': 0.4223, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.2163, 'i_inv': 0.3638, 'i_fam': 0.7143, 'i_geo': 0.1, 'i_fea': 0.4}, ('lit', 'vie'): {'d_size': 4.103, 'd_overlap_w': 0.0358, 'd_ttr': 0.8821, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.4934, 'i_inv': 0.5394, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'deu'): {'d_size': 4.0051, 'd_overlap_w': 0.0178, 'd_ttr': 0.4311, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.3918, 'i_inv': 0.5022, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'pes'): {'d_size': 3.6014, 'd_overlap_w': 0.0079, 'd_ttr': 0.642, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.2952, 'i_inv': 0.4795, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.5}, ('lit', 'hun'): {'d_size': 3.512, 'd_overlap_w': 0.0116, 'd_ttr': 0.0525, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.2952, 'i_inv': 0.4287, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('lit', 'srp'): {'d_size': 3.2658, 'd_overlap_w': 0.0128, 'd_ttr': 0.1672, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.8391, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('lit', 'ell'): {'d_size': 3.2044, 'd_overlap_w': 0.009, 'd_ttr': 0.3918, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2952, 'i_inv': 0.5009, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'hrv'): {'d_size': 2.9125, 'd_overlap_w': 0.0196, 'd_ttr': 0.2323, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.3666, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'ukr'): {'d_size': 2.5882, 'd_overlap_w': 0.0064, 'd_ttr': 0.1898, 'd_overlap_s': '-', 'i_syn': 0.44, 'i_phon': 0.641, 'i_inv': 0.4436, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.5}, ('lit', 'ces'): {'d_size': 2.4593, 'd_overlap_w': 0.0162, 'd_ttr': 0.1632, 'd_overlap_s': '-', 'i_syn': 0.29, 'i_phon': 0.641, 'i_inv': 0.4073, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('lit', 'tha'): {'d_size': 2.3394, 'd_overlap_w': 0.0043, 'd_ttr': 0.1234, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.4638, 'i_inv': 0.4887, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'ind'): {'d_size': 2.0851, 'd_overlap_w': 0.027, 'd_ttr': 0.5538, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3498, 'i_inv': 0.4766, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('lit', 'slk'): {'d_size': 1.4664, 'd_overlap_w': 0.0161, 'd_ttr': 0.056, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.6}, ('lit', 'swe'): {'d_size': 1.3513, 'd_overlap_w': 0.0185, 'd_ttr': 0.2409, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.641, 'i_inv': 0.5211, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.6}, ('lit', 'por'): {'d_size': 1.2354, 'd_overlap_w': 0.0202, 'd_ttr': 0.2949, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.454, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'dan'): {'d_size': 1.0721, 'd_overlap_w': 0.0196, 'd_ttr': 0.2777, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.8}, ('lit', 'mkd'): {'d_size': 0.6044, 'd_overlap_w': 0.0053, 'd_ttr': 0.0343, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.641, 'i_inv': 0.392, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'fin'): {'d_size': 0.5778, 'd_overlap_w': 0.0099, 'd_ttr': 0.3885, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3498, 'i_inv': 0.4657, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'mya'): {'d_size': 0.5128, 'd_overlap_w': 0.0086, 'd_ttr': 0.1115, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.4259, 'i_inv': 0.469, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('lit', 'hye'): {'d_size': 0.5096, 'd_overlap_w': 0.0045, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5, 'i_inv': 0.4906, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'slv'): {'d_size': 0.4731, 'd_overlap_w': 0.0149, 'd_ttr': 0.0048, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.641, 'i_inv': 0.429, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'hin'): {'d_size': 0.4484, 'd_overlap_w': 0.004, 'd_ttr': 0.3649, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2952, 'i_inv': 0.5129, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('lit', 'nob'): {'d_size': 0.3775, 'd_overlap_w': 0.013, 'd_ttr': 0.0656, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.4278, 'i_inv': 0.5526, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'est'): {'d_size': 0.2562, 'd_overlap_w': 0.0078, 'd_ttr': 0.3443, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('lit', 'kur'): {'d_size': 0.2474, 'd_overlap_w': 0.001, 'd_ttr': 0.0167, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('lit', 'glg'): {'d_size': 0.239, 'd_overlap_w': 0.0124, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.641, 'i_inv': 0.5421, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'mar'): {'d_size': 0.2347, 'd_overlap_w': 0.0012, 'd_ttr': 0.0323, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5408, 'i_fam': 0.75, 'i_geo': 0.3, 'i_fea': 0.6}, ('lit', 'tam'): {'d_size': 0.1485, 'd_overlap_w': 0.0028, 'd_ttr': 0.2166, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.5698, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('lit', 'urd'): {'d_size': 0.1426, 'd_overlap_w': 0.0035, 'd_ttr': 0.1074, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5551, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('lit', 'azb'): {'d_size': 0.1418, 'd_overlap_w': 0.0061, 'd_ttr': 1.1309, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.2952, 'i_inv': 0.5621, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'bos'): {'d_size': 0.1351, 'd_overlap_w': 0.0088, 'd_ttr': 0.5227, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('lit', 'csl'): {'d_size': 0.132, 'd_overlap_w': 0.0028, 'd_ttr': 19.4487, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'eus'): {'d_size': 0.1236, 'd_overlap_w': 0.0058, 'd_ttr': 0.7273, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3217, 'i_inv': 0.5417, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'ben'): {'d_size': 0.1109, 'd_overlap_w': 0.0006, 'd_ttr': 0.0125, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3824, 'i_inv': 0.4886, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('lit', 'bel'): {'d_size': 0.1076, 'd_overlap_w': 0.0026, 'd_ttr': 0.712, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.8}, ('lit', 'kaz'): {'d_size': 0.0791, 'd_overlap_w': 0.0021, 'd_ttr': 2.4263, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 1.0, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('lit', 'kat'): {'d_size': 0.3147, 'd_overlap_w': 0.0031, 'd_ttr': 0.3364, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.3498, 'i_inv': 0.5404, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'zsm'): {'d_size': 0.1245, 'd_overlap_w': 0.0084, 'd_ttr': 0.0114, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.4766, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}}, 'slv': {('slv', 'arb'): {'d_size': 10.7968, 'd_overlap_w': 0.0031, 'd_ttr': 0.2709, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('slv', 'heb'): {'d_size': 10.6812, 'd_overlap_w': 0.0035, 'd_ttr': 0.3506, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.0002, 'i_inv': 0.572, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'rus'): {'d_size': 10.5117, 'd_overlap_w': 0.0049, 'd_ttr': 0.3499, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.6162, 'i_inv': 0.4254, 'i_fam': 0.4, 'i_geo': 0.2, 'i_fea': 0.5}, ('slv', 'kor'): {'d_size': 10.3696, 'd_overlap_w': 0.0031, 'd_ttr': 0.0034, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.641, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'ita'): {'d_size': 10.3123, 'd_overlap_w': 0.0205, 'd_ttr': 0.6335, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.0002, 'i_inv': 0.4312, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'jpn'): {'d_size': 10.2915, 'd_overlap_w': 0.0009, 'd_ttr': 22.0004, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.6162, 'i_inv': 0.5062, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('slv', 'spa'): {'d_size': 9.8848, 'd_overlap_w': 0.019, 'd_ttr': 0.6311, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5922, 'i_inv': 0.5118, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'fra'): {'d_size': 9.6971, 'd_overlap_w': 0.0199, 'd_ttr': 0.705, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.4688, 'i_fam': 0.9375, 'i_geo': 0.0, 'i_fea': 0.6}, ('slv', 'nld'): {'d_size': 9.2667, 'd_overlap_w': 0.0179, 'd_ttr': 0.5593, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.0002, 'i_inv': 0.4622, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'tur'): {'d_size': 9.2013, 'd_overlap_w': 0.0111, 'd_ttr': 0.1357, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.4627, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'ron'): {'d_size': 9.1011, 'd_overlap_w': 0.0186, 'd_ttr': 0.504, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.641, 'i_inv': 0.3931, 'i_fam': 0.8889, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'pol'): {'d_size': 8.8835, 'd_overlap_w': 0.0174, 'd_ttr': 0.2508, 'd_overlap_s': '-', 'i_syn': 0.4, 'i_phon': 0.6162, 'i_inv': 0.5281, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'bul'): {'d_size': 8.7965, 'd_overlap_w': 0.005, 'd_ttr': 0.4524, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.6162, 'i_inv': 0.4225, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'vie'): {'d_size': 8.673, 'd_overlap_w': 0.0335, 'd_ttr': 0.8896, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5456, 'i_inv': 0.5303, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'deu'): {'d_size': 8.4659, 'd_overlap_w': 0.0143, 'd_ttr': 0.461, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5456, 'i_inv': 0.4255, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'pes'): {'d_size': 7.6126, 'd_overlap_w': 0.0069, 'd_ttr': 0.6629, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.4914, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('slv', 'hun'): {'d_size': 7.4237, 'd_overlap_w': 0.009, 'd_ttr': 0.078, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.438, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'srp'): {'d_size': 6.9032, 'd_overlap_w': 0.0517, 'd_ttr': 0.2001, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.8278, 'i_inv': 0.669, 'i_fam': 0.1667, 'i_geo': 0.0, 'i_fea': 0.7}, ('slv', 'ell'): {'d_size': 6.7736, 'd_overlap_w': 0.0082, 'd_ttr': 0.4228, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5922, 'i_inv': 0.4416, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'hrv'): {'d_size': 6.1566, 'd_overlap_w': 0.0692, 'd_ttr': 0.266, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.3141, 'i_fam': 0.375, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'ukr'): {'d_size': 5.471, 'd_overlap_w': 0.0052, 'd_ttr': 0.2231, 'd_overlap_s': '-', 'i_syn': 0.35, 'i_phon': 0.0002, 'i_inv': 0.404, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'ces'): {'d_size': 5.1986, 'd_overlap_w': 0.0231, 'd_ttr': 0.196, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.4204, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'tha'): {'d_size': 4.945, 'd_overlap_w': 0.0032, 'd_ttr': 0.0694, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5687, 'i_inv': 0.4313, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'ind'): {'d_size': 4.4075, 'd_overlap_w': 0.029, 'd_ttr': 0.5789, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5687, 'i_inv': 0.4413, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('slv', 'slk'): {'d_size': 3.0997, 'd_overlap_w': 0.028, 'd_ttr': 0.082, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('slv', 'swe'): {'d_size': 2.8565, 'd_overlap_w': 0.0208, 'd_ttr': 0.2745, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.5107, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'por'): {'d_size': 2.6113, 'd_overlap_w': 0.0196, 'd_ttr': 0.3281, 'd_overlap_s': '-', 'i_syn': 0.44, 'i_phon': 0.0002, 'i_inv': 0.3356, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.4}, ('slv', 'dan'): {'d_size': 2.2661, 'd_overlap_w': 0.0236, 'd_ttr': 0.311, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('slv', 'lit'): {'d_size': 2.1138, 'd_overlap_w': 0.0149, 'd_ttr': 0.0042, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.641, 'i_inv': 0.429, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'mkd'): {'d_size': 1.2775, 'd_overlap_w': 0.0058, 'd_ttr': 0.0567, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.3753, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'fin'): {'d_size': 1.2214, 'd_overlap_w': 0.011, 'd_ttr': 0.268, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'mya'): {'d_size': 1.084, 'd_overlap_w': 0.0079, 'd_ttr': 0.1423, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5228, 'i_inv': 0.409, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'hye'): {'d_size': 1.0771, 'd_overlap_w': 0.0046, 'd_ttr': 0.0042, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'hin'): {'d_size': 0.9479, 'd_overlap_w': 0.0049, 'd_ttr': 0.3967, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5922, 'i_inv': 0.4816, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('slv', 'nob'): {'d_size': 0.798, 'd_overlap_w': 0.0189, 'd_ttr': 0.0928, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.5922, 'i_inv': 0.5251, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'est'): {'d_size': 0.5415, 'd_overlap_w': 0.011, 'd_ttr': 0.2338, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('slv', 'kur'): {'d_size': 0.523, 'd_overlap_w': 0.0014, 'd_ttr': 0.0031, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('slv', 'glg'): {'d_size': 0.5051, 'd_overlap_w': 0.0154, 'd_ttr': 0.0036, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.0002, 'i_inv': 0.4685, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'mar'): {'d_size': 0.4962, 'd_overlap_w': 0.0016, 'd_ttr': 0.0543, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.4477, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('slv', 'tam'): {'d_size': 0.3139, 'd_overlap_w': 0.0035, 'd_ttr': 0.2502, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.5046, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'urd'): {'d_size': 0.3014, 'd_overlap_w': 0.004, 'd_ttr': 0.1379, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.5088, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('slv', 'azb'): {'d_size': 0.2998, 'd_overlap_w': 0.0102, 'd_ttr': 0.8634, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5922, 'i_inv': 0.557, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'bos'): {'d_size': 0.2856, 'd_overlap_w': 0.0606, 'd_ttr': 0.3731, 'd_overlap_s': '-', 'i_syn': 0.36, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.1667, 'i_geo': 0.0, 'i_fea': 0.6}, ('slv', 'csl'): {'d_size': 0.2791, 'd_overlap_w': 0.0036, 'd_ttr': 16.4676, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('slv', 'eus'): {'d_size': 0.2613, 'd_overlap_w': 0.0089, 'd_ttr': 0.5362, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'ben'): {'d_size': 0.2344, 'd_overlap_w': 0.0007, 'd_ttr': 0.0287, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5332, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('slv', 'bel'): {'d_size': 0.2274, 'd_overlap_w': 0.0038, 'd_ttr': 0.524, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.8}, ('slv', 'kaz'): {'d_size': 0.1673, 'd_overlap_w': 0.003, 'd_ttr': 1.9355, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('slv', 'kat'): {'d_size': 0.6653, 'd_overlap_w': 0.0038, 'd_ttr': 0.2277, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'zsm'): {'d_size': 0.2632, 'd_overlap_w': 0.0135, 'd_ttr': 0.0272, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.4413, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}}, 'kaz': {('kaz', 'arb'): {'d_size': 64.5496, 'd_overlap_w': 0.0009, 'd_ttr': 0.6392, 'd_overlap_s': '-', 'i_syn': 0.93, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'heb'): {'d_size': 63.8586, 'd_overlap_w': 0.001, 'd_ttr': 0.6879, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.879, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'rus'): {'d_size': 62.8453, 'd_overlap_w': 0.0046, 'd_ttr': 0.6875, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 1.0, 'i_inv': 0.688, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'kor'): {'d_size': 61.9958, 'd_overlap_w': 0.0005, 'd_ttr': 0.3675, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.7699, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('kaz', 'ita'): {'d_size': 61.653, 'd_overlap_w': 0.002, 'd_ttr': 0.8366, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'jpn'): {'d_size': 61.5285, 'd_overlap_w': 0.0003, 'd_ttr': 1.9036, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 1.0, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'spa'): {'d_size': 59.0974, 'd_overlap_w': 0.002, 'd_ttr': 0.8354, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 1.0, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'fra'): {'d_size': 57.9753, 'd_overlap_w': 0.0023, 'd_ttr': 0.8704, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 1.0, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'nld'): {'d_size': 55.4016, 'd_overlap_w': 0.0018, 'd_ttr': 0.8002, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.879, 'i_inv': 0.6068, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'tur'): {'d_size': 55.0106, 'd_overlap_w': 0.0009, 'd_ttr': 0.5414, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6196, 'i_fam': 0.4286, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'ron'): {'d_size': 54.4118, 'd_overlap_w': 0.0016, 'd_ttr': 0.7721, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 1.0, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'pol'): {'d_size': 53.1109, 'd_overlap_w': 0.0012, 'd_ttr': 0.6261, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 1.0, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'bul'): {'d_size': 52.5909, 'd_overlap_w': 0.0047, 'd_ttr': 0.7449, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 1.0, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'vie'): {'d_size': 51.8526, 'd_overlap_w': 0.0049, 'd_ttr': 0.953, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.8136, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'deu'): {'d_size': 50.6144, 'd_overlap_w': 0.0015, 'd_ttr': 0.7495, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.8136, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'pes'): {'d_size': 45.5125, 'd_overlap_w': 0.0018, 'd_ttr': 0.8506, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 1.0, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'hun'): {'d_size': 44.3832, 'd_overlap_w': 0.001, 'd_ttr': 0.4881, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'srp'): {'d_size': 41.2716, 'd_overlap_w': 0.0023, 'd_ttr': 0.5912, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 1.0, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ell'): {'d_size': 40.4965, 'd_overlap_w': 0.0017, 'd_ttr': 0.7289, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 1.0, 'i_inv': 0.6929, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'hrv'): {'d_size': 36.8077, 'd_overlap_w': 0.0015, 'd_ttr': 0.636, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.879, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ukr'): {'d_size': 32.7088, 'd_overlap_w': 0.0044, 'd_ttr': 0.6074, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ces'): {'d_size': 31.0802, 'd_overlap_w': 0.0015, 'd_ttr': 0.5882, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.879, 'i_inv': 0.6784, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'tha'): {'d_size': 29.5641, 'd_overlap_w': 0.0005, 'd_ttr': 0.2225, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.805, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ind'): {'d_size': 26.3509, 'd_overlap_w': 0.0037, 'd_ttr': 0.81, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.805, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('kaz', 'slk'): {'d_size': 18.5318, 'd_overlap_w': 0.0019, 'd_ttr': 0.4922, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'swe'): {'d_size': 17.0778, 'd_overlap_w': 0.0027, 'd_ttr': 0.6415, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.879, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'por'): {'d_size': 15.612, 'd_overlap_w': 0.0031, 'd_ttr': 0.6746, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.879, 'i_inv': 0.6328, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'dan'): {'d_size': 13.5484, 'd_overlap_w': 0.0031, 'd_ttr': 0.6643, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'lit'): {'d_size': 12.6376, 'd_overlap_w': 0.0021, 'd_ttr': 0.3709, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 1.0, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'mkd'): {'d_size': 7.6379, 'd_overlap_w': 0.0069, 'd_ttr': 0.4643, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'fin'): {'d_size': 7.3024, 'd_overlap_w': 0.0021, 'd_ttr': 0.1335, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.805, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'mya'): {'d_size': 6.4809, 'd_overlap_w': 0.0013, 'd_ttr': 0.5469, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.8211, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'hye'): {'d_size': 6.4396, 'd_overlap_w': 0.0027, 'd_ttr': 0.3706, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'slv'): {'d_size': 5.9786, 'd_overlap_w': 0.003, 'd_ttr': 0.3385, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'hin'): {'d_size': 5.6672, 'd_overlap_w': 0.0037, 'd_ttr': 0.7144, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 1.0, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'nob'): {'d_size': 4.7709, 'd_overlap_w': 0.0041, 'd_ttr': 0.5029, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 1.0, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'est'): {'d_size': 3.2373, 'd_overlap_w': 0.0032, 'd_ttr': 0.1441, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'kur'): {'d_size': 3.1266, 'd_overlap_w': 0.0014, 'd_ttr': 0.3118, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kaz', 'glg'): {'d_size': 3.0199, 'd_overlap_w': 0.0044, 'd_ttr': 0.3683, 'd_overlap_s': '-', 'i_syn': 0.92, 'i_phon': 0.879, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}, ('kaz', 'mar'): {'d_size': 2.9665, 'd_overlap_w': 0.0013, 'd_ttr': 0.4614, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.879, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'tam'): {'d_size': 1.8764, 'd_overlap_w': 0.004, 'd_ttr': 0.6257, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.879, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'urd'): {'d_size': 1.8019, 'd_overlap_w': 0.0044, 'd_ttr': 0.5433, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'azb'): {'d_size': 1.7926, 'd_overlap_w': 0.0035, 'd_ttr': 0.0373, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 1.0, 'i_inv': 0.669, 'i_fam': 0.3333, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'bos'): {'d_size': 1.7076, 'd_overlap_w': 0.0035, 'd_ttr': 0.1065, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'csl'): {'d_size': 1.6684, 'd_overlap_w': 0.0032, 'd_ttr': 1.2437, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'eus'): {'d_size': 1.5623, 'd_overlap_w': 0.0035, 'd_ttr': 0.076, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'ben'): {'d_size': 1.4016, 'd_overlap_w': 0.0011, 'd_ttr': 0.426, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 1.0, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'bel'): {'d_size': 1.3594, 'd_overlap_w': 0.0073, 'd_ttr': 0.0779, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'kat'): {'d_size': 3.9774, 'd_overlap_w': 0.0028, 'd_ttr': 0.1461, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'zsm'): {'d_size': 1.5737, 'd_overlap_w': 0.0059, 'd_ttr': 0.4235, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}}, 'mkd': {('mkd', 'arb'): {'d_size': 8.4512, 'd_overlap_w': 0.0027, 'd_ttr': 0.1373, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'heb'): {'d_size': 8.3607, 'd_overlap_w': 0.0031, 'd_ttr': 0.2158, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.5654, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'rus'): {'d_size': 8.2281, 'd_overlap_w': 0.0199, 'd_ttr': 0.2151, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.4147, 'i_fam': 0.4, 'i_geo': 0.2, 'i_fea': 0.5}, ('mkd', 'kor'): {'d_size': 8.1168, 'd_overlap_w': 0.0019, 'd_ttr': 0.0557, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.641, 'i_inv': 0.5861, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mkd', 'ita'): {'d_size': 8.072, 'd_overlap_w': 0.0078, 'd_ttr': 0.536, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.4473, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'jpn'): {'d_size': 8.0557, 'd_overlap_w': 0.0007, 'd_ttr': 41.8524, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.6162, 'i_inv': 0.5839, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('mkd', 'spa'): {'d_size': 7.7374, 'd_overlap_w': 0.0077, 'd_ttr': 0.5331, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.479, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'fra'): {'d_size': 7.5904, 'd_overlap_w': 0.0085, 'd_ttr': 0.6233, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.5006, 'i_fam': 0.9375, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'nld'): {'d_size': 7.2535, 'd_overlap_w': 0.0074, 'd_ttr': 0.4476, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.5145, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'tur'): {'d_size': 7.2023, 'd_overlap_w': 0.0041, 'd_ttr': 0.0292, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.5922, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('mkd', 'ron'): {'d_size': 7.1239, 'd_overlap_w': 0.0067, 'd_ttr': 0.3835, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.4086, 'i_fam': 0.8889, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'pol'): {'d_size': 6.9536, 'd_overlap_w': 0.0049, 'd_ttr': 0.1189, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.6162, 'i_inv': 0.3697, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'bul'): {'d_size': 6.8855, 'd_overlap_w': 0.0659, 'd_ttr': 0.3252, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.6162, 'i_inv': 0.4367, 'i_fam': 0.1429, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'vie'): {'d_size': 6.7888, 'd_overlap_w': 0.0148, 'd_ttr': 0.8564, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.5611, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mkd', 'deu'): {'d_size': 6.6267, 'd_overlap_w': 0.0064, 'd_ttr': 0.3348, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5456, 'i_inv': 0.5026, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'pes'): {'d_size': 5.9588, 'd_overlap_w': 0.0051, 'd_ttr': 0.5717, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.5922, 'i_inv': 0.4086, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('mkd', 'hun'): {'d_size': 5.8109, 'd_overlap_w': 0.0036, 'd_ttr': 0.0029, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5922, 'i_inv': 0.4278, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('mkd', 'srp'): {'d_size': 5.4035, 'd_overlap_w': 0.0326, 'd_ttr': 0.0754, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.8278, 'i_inv': 0.6737, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.8}, ('mkd', 'ell'): {'d_size': 5.302, 'd_overlap_w': 0.0058, 'd_ttr': 0.2926, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.4852, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.6}, ('mkd', 'hrv'): {'d_size': 4.8191, 'd_overlap_w': 0.0059, 'd_ttr': 0.1327, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.2983, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('mkd', 'ukr'): {'d_size': 4.2824, 'd_overlap_w': 0.0181, 'd_ttr': 0.0945, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.3926, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'ces'): {'d_size': 4.0692, 'd_overlap_w': 0.0056, 'd_ttr': 0.0721, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.0002, 'i_inv': 0.315, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'tha'): {'d_size': 3.8707, 'd_overlap_w': 0.002, 'd_ttr': 0.4334, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5687, 'i_inv': 0.5531, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mkd', 'ind'): {'d_size': 3.45, 'd_overlap_w': 0.0106, 'd_ttr': 0.4707, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5687, 'i_inv': 0.3444, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('mkd', 'slk'): {'d_size': 2.4263, 'd_overlap_w': 0.0056, 'd_ttr': 0.004, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.7}, ('mkd', 'swe'): {'d_size': 2.2359, 'd_overlap_w': 0.0077, 'd_ttr': 0.1407, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.5797, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'por'): {'d_size': 2.044, 'd_overlap_w': 0.0083, 'd_ttr': 0.193, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.4296, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'dan'): {'d_size': 1.7738, 'd_overlap_w': 0.0088, 'd_ttr': 0.1759, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'lit'): {'d_size': 1.6546, 'd_overlap_w': 0.0053, 'd_ttr': 0.0516, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.641, 'i_inv': 0.392, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'fin'): {'d_size': 0.9561, 'd_overlap_w': 0.0048, 'd_ttr': 0.9843, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'mya'): {'d_size': 0.8485, 'd_overlap_w': 0.0034, 'd_ttr': 0.0333, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5228, 'i_inv': 0.5106, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mkd', 'hye'): {'d_size': 0.8431, 'd_overlap_w': 0.0043, 'd_ttr': 0.0519, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.3926, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'slv'): {'d_size': 0.7828, 'd_overlap_w': 0.0058, 'd_ttr': 0.0977, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.3753, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.4}, ('mkd', 'hin'): {'d_size': 0.742, 'd_overlap_w': 0.0045, 'd_ttr': 0.2643, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.558, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('mkd', 'nob'): {'d_size': 0.6246, 'd_overlap_w': 0.0082, 'd_ttr': 0.0076, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5922, 'i_inv': 0.589, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'est'): {'d_size': 0.4238, 'd_overlap_w': 0.0043, 'd_ttr': 0.8974, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'kur'): {'d_size': 0.4094, 'd_overlap_w': 0.0013, 'd_ttr': 0.149, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('mkd', 'glg'): {'d_size': 0.3954, 'd_overlap_w': 0.0062, 'd_ttr': 0.0547, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.4828, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'mar'): {'d_size': 0.3884, 'd_overlap_w': 0.0012, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.5448, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mkd', 'tam'): {'d_size': 0.2457, 'd_overlap_w': 0.0036, 'd_ttr': 0.1183, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.5366, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mkd', 'urd'): {'d_size': 0.2359, 'd_overlap_w': 0.0033, 'd_ttr': 0.0306, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.5758, 'i_fam': 0.8571, 'i_geo': 0.2, 'i_fea': 0.6}, ('mkd', 'azb'): {'d_size': 0.2347, 'd_overlap_w': 0.004, 'd_ttr': 2.3478, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5922, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('mkd', 'bos'): {'d_size': 0.2236, 'd_overlap_w': 0.0046, 'd_ttr': 1.2419, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.8}, ('mkd', 'csl'): {'d_size': 0.2184, 'd_overlap_w': 0.0027, 'd_ttr': 31.8007, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mkd', 'eus'): {'d_size': 0.2045, 'd_overlap_w': 0.0035, 'd_ttr': 1.6224, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.641, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'ben'): {'d_size': 0.1835, 'd_overlap_w': 0.0007, 'd_ttr': 0.0081, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.6162, 'i_inv': 0.5978, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.7}, ('mkd', 'bel'): {'d_size': 0.178, 'd_overlap_w': 0.0071, 'd_ttr': 1.5945, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'kaz'): {'d_size': 0.1309, 'd_overlap_w': 0.0069, 'd_ttr': 4.5743, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mkd', 'kat'): {'d_size': 0.5207, 'd_overlap_w': 0.0037, 'd_ttr': 0.8817, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5687, 'i_inv': 0.5037, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'zsm'): {'d_size': 0.206, 'd_overlap_w': 0.0055, 'd_ttr': 0.0093, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.3108, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.4}}, 'urd': {('urd', 'arb'): {'d_size': 35.8225, 'd_overlap_w': 0.0059, 'd_ttr': 0.0563, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('urd', 'heb'): {'d_size': 35.439, 'd_overlap_w': 0.0015, 'd_ttr': 0.1232, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.5698, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'rus'): {'d_size': 34.8767, 'd_overlap_w': 0.0016, 'd_ttr': 0.1226, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.6162, 'i_inv': 0.5919, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'kor'): {'d_size': 34.4052, 'd_overlap_w': 0.0008, 'd_ttr': 0.2479, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5935, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'ita'): {'d_size': 34.215, 'd_overlap_w': 0.0037, 'd_ttr': 0.4561, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.5577, 'i_fam': 0.9, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'jpn'): {'d_size': 34.1459, 'd_overlap_w': 0.0004, 'd_ttr': 64.8429, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.6162, 'i_inv': 0.4859, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'spa'): {'d_size': 32.7967, 'd_overlap_w': 0.0035, 'd_ttr': 0.4529, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5922, 'i_inv': 0.5985, 'i_fam': 0.9231, 'i_geo': 0.3, 'i_fea': 0.7}, ('urd', 'fra'): {'d_size': 32.174, 'd_overlap_w': 0.0043, 'd_ttr': 0.5548, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5456, 'i_inv': 0.5194, 'i_fam': 0.9375, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'nld'): {'d_size': 30.7457, 'd_overlap_w': 0.0035, 'd_ttr': 0.3587, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.4971, 'i_fam': 0.9, 'i_geo': 0.3, 'i_fea': 0.5}, ('urd', 'tur'): {'d_size': 30.5287, 'd_overlap_w': 0.0017, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.5459, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'ron'): {'d_size': 30.1964, 'd_overlap_w': 0.0029, 'd_ttr': 0.2901, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.641, 'i_inv': 0.5126, 'i_fam': 0.8889, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'pol'): {'d_size': 29.4745, 'd_overlap_w': 0.0022, 'd_ttr': 0.0424, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.6162, 'i_inv': 0.5772, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'bul'): {'d_size': 29.1859, 'd_overlap_w': 0.0021, 'd_ttr': 0.2296, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.5331, 'i_fam': 0.8571, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'vie'): {'d_size': 28.7761, 'd_overlap_w': 0.0098, 'd_ttr': 0.8274, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5456, 'i_inv': 0.5018, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'deu'): {'d_size': 28.089, 'd_overlap_w': 0.0028, 'd_ttr': 0.2394, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.5018, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'pes'): {'d_size': 25.2577, 'd_overlap_w': 0.0308, 'd_ttr': 0.4962, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5309, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'hun'): {'d_size': 24.6309, 'd_overlap_w': 0.0015, 'd_ttr': 0.0214, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5922, 'i_inv': 0.5612, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'srp'): {'d_size': 22.9041, 'd_overlap_w': 0.0018, 'd_ttr': 0.0146, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.8278, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.9}, ('urd', 'ell'): {'d_size': 22.474, 'd_overlap_w': 0.0026, 'd_ttr': 0.1968, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.566, 'i_fam': 0.8333, 'i_geo': 0.2, 'i_fea': 0.7}, ('urd', 'hrv'): {'d_size': 20.4268, 'd_overlap_w': 0.0026, 'd_ttr': 0.0527, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.0002, 'i_inv': 0.562, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'ukr'): {'d_size': 18.1521, 'd_overlap_w': 0.002, 'd_ttr': 0.0258, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.5006, 'i_fam': 0.8333, 'i_geo': 0.2, 'i_fea': 0.5}, ('urd', 'ces'): {'d_size': 17.2483, 'd_overlap_w': 0.0025, 'd_ttr': 0.0129, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.569, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'tha'): {'d_size': 16.4069, 'd_overlap_w': 0.0009, 'd_ttr': 1.0197, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.4862, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'ind'): {'d_size': 14.6237, 'd_overlap_w': 0.007, 'd_ttr': 0.3838, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.5088, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'slk'): {'d_size': 10.2844, 'd_overlap_w': 0.0031, 'd_ttr': 0.0183, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.7}, ('urd', 'swe'): {'d_size': 9.4775, 'd_overlap_w': 0.0049, 'd_ttr': 0.0589, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.5187, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'por'): {'d_size': 8.664, 'd_overlap_w': 0.0053, 'd_ttr': 0.1027, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.4932, 'i_fam': 0.9333, 'i_geo': 0.3, 'i_fea': 0.5}, ('urd', 'dan'): {'d_size': 7.5188, 'd_overlap_w': 0.0053, 'd_ttr': 0.0879, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('urd', 'lit'): {'d_size': 7.0134, 'd_overlap_w': 0.0035, 'd_ttr': 0.2375, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5551, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'mkd'): {'d_size': 4.2387, 'd_overlap_w': 0.0033, 'd_ttr': 0.0449, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.5758, 'i_fam': 0.8571, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'fin'): {'d_size': 4.0525, 'd_overlap_w': 0.0029, 'd_ttr': 2.0004, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5687, 'i_inv': 0.5551, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'mya'): {'d_size': 3.5966, 'd_overlap_w': 0.0027, 'd_ttr': 0.0001, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5228, 'i_inv': 0.4889, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'hye'): {'d_size': 3.5737, 'd_overlap_w': 0.0035, 'd_ttr': 0.2383, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5577, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'slv'): {'d_size': 3.3179, 'd_overlap_w': 0.004, 'd_ttr': 0.3491, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.5088, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('urd', 'hin'): {'d_size': 3.1451, 'd_overlap_w': 0.0056, 'd_ttr': 0.169, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5922, 'i_inv': 0.3583, 'i_fam': 0.125, 'i_geo': 0.1, 'i_fea': 0.4}, ('urd', 'nob'): {'d_size': 2.6476, 'd_overlap_w': 0.0062, 'd_ttr': 0.0113, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.5197, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'est'): {'d_size': 1.7966, 'd_overlap_w': 0.0039, 'd_ttr': 1.8498, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('urd', 'kur'): {'d_size': 1.7352, 'd_overlap_w': 0.0097, 'd_ttr': 0.462, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('urd', 'glg'): {'d_size': 1.6759, 'd_overlap_w': 0.0063, 'd_ttr': 0.2454, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.5662, 'i_fam': 0.9231, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'mar'): {'d_size': 1.6463, 'd_overlap_w': 0.0016, 'd_ttr': 0.0484, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.4445, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('urd', 'tam'): {'d_size': 1.0413, 'd_overlap_w': 0.0061, 'd_ttr': 0.042, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.4955, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('urd', 'azb'): {'d_size': 0.9948, 'd_overlap_w': 0.0049, 'd_ttr': 4.2807, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5996, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'bos'): {'d_size': 0.9476, 'd_overlap_w': 0.0049, 'd_ttr': 2.4416, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('urd', 'csl'): {'d_size': 0.9259, 'd_overlap_w': 0.0041, 'd_ttr': 49.6521, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('urd', 'eus'): {'d_size': 0.867, 'd_overlap_w': 0.0041, 'd_ttr': 3.0824, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.5996, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'ben'): {'d_size': 0.7778, 'd_overlap_w': 0.0015, 'd_ttr': 0.1032, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.6162, 'i_inv': 0.3603, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('urd', 'bel'): {'d_size': 0.7544, 'd_overlap_w': 0.0042, 'd_ttr': 3.0357, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('urd', 'kaz'): {'d_size': 0.555, 'd_overlap_w': 0.0044, 'd_ttr': 7.8626, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('urd', 'kat'): {'d_size': 2.2073, 'd_overlap_w': 0.0033, 'd_ttr': 1.8225, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.5804, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'zsm'): {'d_size': 0.8733, 'd_overlap_w': 0.0092, 'd_ttr': 0.1079, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.0002, 'i_inv': 0.5464, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}}, 'mar': {('mar', 'arb'): {'d_size': 21.7592, 'd_overlap_w': 0.0005, 'd_ttr': 0.1404, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mar', 'heb'): {'d_size': 21.5263, 'd_overlap_w': 0.0006, 'd_ttr': 0.2191, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.64, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'rus'): {'d_size': 21.1848, 'd_overlap_w': 0.0006, 'd_ttr': 0.2184, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.6018, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'kor'): {'d_size': 20.8984, 'd_overlap_w': 0.0004, 'd_ttr': 0.0519, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.641, 'i_inv': 0.4829, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'ita'): {'d_size': 20.7828, 'd_overlap_w': 0.0014, 'd_ttr': 0.5386, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.5248, 'i_fam': 0.9, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'jpn'): {'d_size': 20.7409, 'd_overlap_w': 0.0002, 'd_ttr': 41.2163, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.5272, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'spa'): {'d_size': 19.9213, 'd_overlap_w': 0.0013, 'd_ttr': 0.5357, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.5499, 'i_fam': 0.9231, 'i_geo': 0.4, 'i_fea': 0.7}, ('mar', 'fra'): {'d_size': 19.5431, 'd_overlap_w': 0.0016, 'd_ttr': 0.6255, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5456, 'i_inv': 0.5174, 'i_fam': 0.9375, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'nld'): {'d_size': 18.6755, 'd_overlap_w': 0.0013, 'd_ttr': 0.4506, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.5625, 'i_fam': 0.9, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'tur'): {'d_size': 18.5437, 'd_overlap_w': 0.0007, 'd_ttr': 0.0311, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.5638, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mar', 'ron'): {'d_size': 18.3419, 'd_overlap_w': 0.0012, 'd_ttr': 0.3866, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.641, 'i_inv': 0.4744, 'i_fam': 0.8889, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'pol'): {'d_size': 17.9034, 'd_overlap_w': 0.0008, 'd_ttr': 0.1219, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5464, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'bul'): {'d_size': 17.728, 'd_overlap_w': 0.0007, 'd_ttr': 0.3284, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.6162, 'i_inv': 0.5533, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'vie'): {'d_size': 17.4792, 'd_overlap_w': 0.0032, 'd_ttr': 0.8573, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.5456, 'i_inv': 0.5522, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'deu'): {'d_size': 17.0618, 'd_overlap_w': 0.0011, 'd_ttr': 0.338, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5456, 'i_inv': 0.5522, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'pes'): {'d_size': 15.342, 'd_overlap_w': 0.0012, 'd_ttr': 0.5741, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.5342, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.7}, ('mar', 'hun'): {'d_size': 14.9613, 'd_overlap_w': 0.0006, 'd_ttr': 0.0036, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.4901, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'srp'): {'d_size': 13.9124, 'd_overlap_w': 0.0007, 'd_ttr': 0.078, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.8278, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'ell'): {'d_size': 13.6511, 'd_overlap_w': 0.0009, 'd_ttr': 0.2959, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5752, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.7}, ('mar', 'hrv'): {'d_size': 12.4076, 'd_overlap_w': 0.001, 'd_ttr': 0.1358, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5295, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'ukr'): {'d_size': 11.0259, 'd_overlap_w': 0.0007, 'd_ttr': 0.0973, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.5248, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'ces'): {'d_size': 10.4769, 'd_overlap_w': 0.001, 'd_ttr': 0.0747, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.5957, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'tha'): {'d_size': 9.9659, 'd_overlap_w': 0.0004, 'd_ttr': 0.4191, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.5687, 'i_inv': 0.3767, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mar', 'ind'): {'d_size': 8.8827, 'd_overlap_w': 0.0024, 'd_ttr': 0.4735, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.5687, 'i_inv': 0.47, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mar', 'slk'): {'d_size': 6.247, 'd_overlap_w': 0.0012, 'd_ttr': 0.0048, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.7}, ('mar', 'swe'): {'d_size': 5.7568, 'd_overlap_w': 0.0017, 'd_ttr': 0.1439, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.5352, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'por'): {'d_size': 5.2627, 'd_overlap_w': 0.0018, 'd_ttr': 0.1962, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.0002, 'i_inv': 0.4726, 'i_fam': 0.9333, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'dan'): {'d_size': 4.5671, 'd_overlap_w': 0.0019, 'd_ttr': 0.1792, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'lit'): {'d_size': 4.2601, 'd_overlap_w': 0.0012, 'd_ttr': 0.048, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5408, 'i_fam': 0.75, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'mkd'): {'d_size': 2.5747, 'd_overlap_w': 0.0012, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.5448, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'fin'): {'d_size': 2.4616, 'd_overlap_w': 0.0012, 'd_ttr': 0.9583, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5687, 'i_inv': 0.4824, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'mya'): {'d_size': 2.1847, 'd_overlap_w': 0.0013, 'd_ttr': 0.0353, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5228, 'i_inv': 0.4475, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('mar', 'hye'): {'d_size': 2.1707, 'd_overlap_w': 0.0011, 'd_ttr': 0.0483, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5448, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('mar', 'slv'): {'d_size': 2.0153, 'd_overlap_w': 0.0016, 'd_ttr': 0.0924, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.4477, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('mar', 'hin'): {'d_size': 1.9104, 'd_overlap_w': 0.0873, 'd_ttr': 0.2676, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.398, 'i_fam': 0.625, 'i_geo': 0.0, 'i_fea': 0.5}, ('mar', 'nob'): {'d_size': 1.6082, 'd_overlap_w': 0.0024, 'd_ttr': 0.0087, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.4265, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('mar', 'est'): {'d_size': 1.0913, 'd_overlap_w': 0.0015, 'd_ttr': 0.8732, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'kur'): {'d_size': 1.054, 'd_overlap_w': 0.0008, 'd_ttr': 0.142, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mar', 'glg'): {'d_size': 1.018, 'd_overlap_w': 0.0023, 'd_ttr': 0.051, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.0002, 'i_inv': 0.5148, 'i_fam': 0.9231, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'tam'): {'d_size': 0.6325, 'd_overlap_w': 0.0017, 'd_ttr': 0.1213, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.0002, 'i_inv': 0.3687, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('mar', 'urd'): {'d_size': 0.6074, 'd_overlap_w': 0.0016, 'd_ttr': 0.0325, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.4445, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('mar', 'azb'): {'d_size': 0.6043, 'd_overlap_w': 0.0018, 'd_ttr': 2.2968, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.6604, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'bos'): {'d_size': 0.5756, 'd_overlap_w': 0.0017, 'd_ttr': 1.211, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'csl'): {'d_size': 0.5624, 'd_overlap_w': 0.0016, 'd_ttr': 31.3079, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'eus'): {'d_size': 0.5266, 'd_overlap_w': 0.0016, 'd_ttr': 1.5844, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.641, 'i_inv': 0.5519, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'ben'): {'d_size': 0.4725, 'd_overlap_w': 0.0011, 'd_ttr': 0.0069, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.6162, 'i_inv': 0.3908, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('mar', 'bel'): {'d_size': 0.4582, 'd_overlap_w': 0.0011, 'd_ttr': 1.557, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'kaz'): {'d_size': 0.3371, 'd_overlap_w': 0.0013, 'd_ttr': 4.486, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.879, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('mar', 'kat'): {'d_size': 1.3408, 'd_overlap_w': 0.001, 'd_ttr': 0.8578, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.6267, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'zsm'): {'d_size': 0.5305, 'd_overlap_w': 0.0026, 'd_ttr': 0.0079, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5324, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}}, 'mya': {('mya', 'arb'): {'d_size': 9.96, 'd_overlap_w': 0.0023, 'd_ttr': 0.0529, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('mya', 'heb'): {'d_size': 9.8534, 'd_overlap_w': 0.0023, 'd_ttr': 0.119, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5228, 'i_inv': 0.544, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('mya', 'rus'): {'d_size': 9.6971, 'd_overlap_w': 0.0039, 'd_ttr': 0.1184, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.3743, 'i_inv': 0.5679, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mya', 'kor'): {'d_size': 9.566, 'd_overlap_w': 0.0029, 'd_ttr': 0.262, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5184, 'i_inv': 0.4682, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('mya', 'ita'): {'d_size': 9.5131, 'd_overlap_w': 0.0154, 'd_ttr': 0.452, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5228, 'i_inv': 0.4682, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mya', 'jpn'): {'d_size': 9.4939, 'd_overlap_w': 0.0009, 'd_ttr': 66.2088, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.47, 'i_inv': 0.4518, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('mya', 'spa'): {'d_size': 9.1188, 'd_overlap_w': 0.0144, 'd_ttr': 0.4488, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.4208, 'i_inv': 0.5148, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'fra'): {'d_size': 8.9456, 'd_overlap_w': 0.0163, 'd_ttr': 0.5513, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.4878, 'i_inv': 0.4708, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'nld'): {'d_size': 8.5485, 'd_overlap_w': 0.0149, 'd_ttr': 0.3542, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.5228, 'i_inv': 0.4643, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'tur'): {'d_size': 8.4882, 'd_overlap_w': 0.0081, 'd_ttr': 0.0002, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.4208, 'i_inv': 0.4815, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('mya', 'ron'): {'d_size': 8.3958, 'd_overlap_w': 0.0128, 'd_ttr': 0.2855, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.4259, 'i_inv': 0.4606, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'pol'): {'d_size': 8.1951, 'd_overlap_w': 0.009, 'd_ttr': 0.0394, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.3743, 'i_inv': 0.5332, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'bul'): {'d_size': 8.1148, 'd_overlap_w': 0.0034, 'd_ttr': 0.225, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.3743, 'i_inv': 0.541, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'vie'): {'d_size': 8.0009, 'd_overlap_w': 0.0288, 'd_ttr': 0.8258, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.3141, 'i_inv': 0.4695, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mya', 'deu'): {'d_size': 7.8098, 'd_overlap_w': 0.0116, 'd_ttr': 0.2347, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.409, 'i_inv': 0.4502, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('mya', 'pes'): {'d_size': 7.0226, 'd_overlap_w': 0.0059, 'd_ttr': 0.4923, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.4208, 'i_inv': 0.5015, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mya', 'hun'): {'d_size': 6.8484, 'd_overlap_w': 0.007, 'd_ttr': 0.0247, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.4208, 'i_inv': 0.4565, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('mya', 'srp'): {'d_size': 6.3682, 'd_overlap_w': 0.005, 'd_ttr': 0.0127, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.8743, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('mya', 'ell'): {'d_size': 6.2486, 'd_overlap_w': 0.0059, 'd_ttr': 0.1922, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.319, 'i_inv': 0.5187, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'hrv'): {'d_size': 5.6794, 'd_overlap_w': 0.0104, 'd_ttr': 0.0495, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5228, 'i_inv': 0.4935, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mya', 'ukr'): {'d_size': 5.047, 'd_overlap_w': 0.0038, 'd_ttr': 0.0233, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5228, 'i_inv': 0.4682, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'ces'): {'d_size': 4.7957, 'd_overlap_w': 0.0096, 'd_ttr': 0.0111, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5228, 'i_inv': 0.523, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mya', 'tha'): {'d_size': 4.5618, 'd_overlap_w': 0.0032, 'd_ttr': 1.0579, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.2566, 'i_inv': 0.3364, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('mya', 'ind'): {'d_size': 4.066, 'd_overlap_w': 0.0205, 'd_ttr': 0.3794, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.2566, 'i_inv': 0.384, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('mya', 'slk'): {'d_size': 2.8595, 'd_overlap_w': 0.0098, 'd_ttr': 0.0213, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'swe'): {'d_size': 2.6351, 'd_overlap_w': 0.0148, 'd_ttr': 0.0555, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5228, 'i_inv': 0.4882, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'por'): {'d_size': 2.4089, 'd_overlap_w': 0.014, 'd_ttr': 0.0987, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5228, 'i_inv': 0.396, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('mya', 'dan'): {'d_size': 2.0905, 'd_overlap_w': 0.0153, 'd_ttr': 0.084, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('mya', 'lit'): {'d_size': 1.95, 'd_overlap_w': 0.0086, 'd_ttr': 0.2512, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.4259, 'i_inv': 0.469, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'mkd'): {'d_size': 1.1785, 'd_overlap_w': 0.0034, 'd_ttr': 0.0498, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5228, 'i_inv': 0.5106, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mya', 'fin'): {'d_size': 1.1268, 'd_overlap_w': 0.0069, 'd_ttr': 2.0645, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.3695, 'i_inv': 0.448, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'hye'): {'d_size': 0.9936, 'd_overlap_w': 0.0022, 'd_ttr': 0.252, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5519, 'i_inv': 0.5308, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mya', 'slv'): {'d_size': 0.9225, 'd_overlap_w': 0.0079, 'd_ttr': 0.3668, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5228, 'i_inv': 0.409, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'hin'): {'d_size': 0.8744, 'd_overlap_w': 0.0028, 'd_ttr': 0.1646, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.4208, 'i_inv': 0.4421, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('mya', 'nob'): {'d_size': 0.7361, 'd_overlap_w': 0.0112, 'd_ttr': 0.0136, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4208, 'i_inv': 0.4892, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'est'): {'d_size': 0.4995, 'd_overlap_w': 0.0055, 'd_ttr': 1.9101, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('mya', 'kur'): {'d_size': 0.4824, 'd_overlap_w': 0.0005, 'd_ttr': 0.4836, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mya', 'glg'): {'d_size': 0.466, 'd_overlap_w': 0.009, 'd_ttr': 0.2594, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.5228, 'i_inv': 0.4165, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('mya', 'mar'): {'d_size': 0.4577, 'd_overlap_w': 0.0013, 'd_ttr': 0.0535, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5228, 'i_inv': 0.4475, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('mya', 'tam'): {'d_size': 0.2895, 'd_overlap_w': 0.0016, 'd_ttr': 0.039, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5228, 'i_inv': 0.5, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mya', 'urd'): {'d_size': 0.278, 'd_overlap_w': 0.0027, 'd_ttr': 0.0001, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5228, 'i_inv': 0.4889, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mya', 'azb'): {'d_size': 0.2766, 'd_overlap_w': 0.0048, 'd_ttr': 4.3999, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.4208, 'i_inv': 0.5762, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mya', 'bos'): {'d_size': 0.2635, 'd_overlap_w': 0.004, 'd_ttr': 2.5168, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('mya', 'csl'): {'d_size': 0.2574, 'd_overlap_w': 0.0024, 'd_ttr': 50.7146, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mya', 'eus'): {'d_size': 0.2411, 'd_overlap_w': 0.0036, 'd_ttr': 3.1732, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4259, 'i_inv': 0.5762, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mya', 'ben'): {'d_size': 0.2163, 'd_overlap_w': 0.0008, 'd_ttr': 0.1113, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3743, 'i_inv': 0.4589, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('mya', 'bel'): {'d_size': 0.2098, 'd_overlap_w': 0.0018, 'd_ttr': 3.1255, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('mya', 'kaz'): {'d_size': 0.1543, 'd_overlap_w': 0.0013, 'd_ttr': 8.0627, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.8211, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mya', 'kat'): {'d_size': 0.6137, 'd_overlap_w': 0.0015, 'd_ttr': 1.8821, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5335, 'i_inv': 0.5555, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mya', 'zsm'): {'d_size': 0.2428, 'd_overlap_w': 0.0074, 'd_ttr': 0.1162, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5228, 'i_inv': 0.4772, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}}, 'nob': {('nob', 'arb'): {'d_size': 13.5299, 'd_overlap_w': 0.0033, 'd_ttr': 0.0964, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('nob', 'heb'): {'d_size': 13.3851, 'd_overlap_w': 0.0037, 'd_ttr': 0.1709, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5922, 'i_inv': 0.5833, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('nob', 'rus'): {'d_size': 13.1727, 'd_overlap_w': 0.0057, 'd_ttr': 0.1703, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.3613, 'i_inv': 0.6556, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('nob', 'kor'): {'d_size': 12.9946, 'd_overlap_w': 0.0036, 'd_ttr': 0.1253, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5319, 'i_inv': 0.4982, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('nob', 'ita'): {'d_size': 12.9228, 'd_overlap_w': 0.0246, 'd_ttr': 0.4992, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5922, 'i_inv': 0.639, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'jpn'): {'d_size': 12.8967, 'd_overlap_w': 0.0011, 'd_ttr': 51.5879, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.4717, 'i_inv': 0.5904, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('nob', 'spa'): {'d_size': 12.3871, 'd_overlap_w': 0.0225, 'd_ttr': 0.4962, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5064, 'i_inv': 0.6445, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'fra'): {'d_size': 12.1519, 'd_overlap_w': 0.0281, 'd_ttr': 0.5919, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.3862, 'i_inv': 0.5505, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('nob', 'nld'): {'d_size': 11.6124, 'd_overlap_w': 0.0283, 'd_ttr': 0.4064, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.5922, 'i_inv': 0.5605, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'tur'): {'d_size': 11.5305, 'd_overlap_w': 0.0121, 'd_ttr': 0.0084, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.2871, 'i_inv': 0.4958, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'ron'): {'d_size': 11.405, 'd_overlap_w': 0.0212, 'd_ttr': 0.3398, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.4278, 'i_inv': 0.5961, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('nob', 'pol'): {'d_size': 11.1323, 'd_overlap_w': 0.0142, 'd_ttr': 0.0796, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3613, 'i_inv': 0.6385, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'bul'): {'d_size': 11.0233, 'd_overlap_w': 0.0059, 'd_ttr': 0.2801, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.3613, 'i_inv': 0.5964, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'vie'): {'d_size': 10.8686, 'd_overlap_w': 0.048, 'd_ttr': 0.8432, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.4788, 'i_inv': 0.61, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('nob', 'deu'): {'d_size': 10.609, 'd_overlap_w': 0.0212, 'd_ttr': 0.2898, 'd_overlap_s': '-', 'i_syn': 0.44, 'i_phon': 0.3862, 'i_inv': 0.5021, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('nob', 'pes'): {'d_size': 9.5397, 'd_overlap_w': 0.0089, 'd_ttr': 0.537, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4097, 'i_inv': 0.5798, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('nob', 'hun'): {'d_size': 9.3029, 'd_overlap_w': 0.0105, 'd_ttr': 0.0013, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.2871, 'i_inv': 0.5249, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('nob', 'srp'): {'d_size': 8.6507, 'd_overlap_w': 0.0101, 'd_ttr': 0.0422, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.8564, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('nob', 'ell'): {'d_size': 8.4883, 'd_overlap_w': 0.0096, 'd_ttr': 0.2471, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4097, 'i_inv': 0.6329, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'hrv'): {'d_size': 7.7151, 'd_overlap_w': 0.0173, 'd_ttr': 0.0922, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5577, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'ukr'): {'d_size': 6.8559, 'd_overlap_w': 0.0058, 'd_ttr': 0.0582, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5922, 'i_inv': 0.6226, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'ces'): {'d_size': 6.5146, 'd_overlap_w': 0.0166, 'd_ttr': 0.0395, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5922, 'i_inv': 0.6169, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'tha'): {'d_size': 6.1968, 'd_overlap_w': 0.0038, 'd_ttr': 0.6669, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.3433, 'i_inv': 0.4859, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('nob', 'ind'): {'d_size': 5.5233, 'd_overlap_w': 0.0398, 'd_ttr': 0.4305, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.4477, 'i_inv': 0.5251, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.6}, ('nob', 'slk'): {'d_size': 3.8844, 'd_overlap_w': 0.0173, 'd_ttr': 0.0007, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'swe'): {'d_size': 3.5796, 'd_overlap_w': 0.0612, 'd_ttr': 0.0995, 'd_overlap_s': '-', 'i_syn': 0.35, 'i_phon': 0.5922, 'i_inv': 0.4852, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('nob', 'por'): {'d_size': 3.2724, 'd_overlap_w': 0.0245, 'd_ttr': 0.1488, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.5757, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'dan'): {'d_size': 2.8398, 'd_overlap_w': 0.1186, 'd_ttr': 0.1325, 'd_overlap_s': '-', 'i_syn': 0.22, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('nob', 'lit'): {'d_size': 2.6489, 'd_overlap_w': 0.013, 'd_ttr': 0.1186, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.4278, 'i_inv': 0.5526, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('nob', 'mkd'): {'d_size': 1.6009, 'd_overlap_w': 0.0082, 'd_ttr': 0.0091, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5922, 'i_inv': 0.589, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'fin'): {'d_size': 1.5306, 'd_overlap_w': 0.0133, 'd_ttr': 1.3978, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.3433, 'i_inv': 0.4626, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('nob', 'mya'): {'d_size': 1.3584, 'd_overlap_w': 0.0112, 'd_ttr': 0.0109, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4208, 'i_inv': 0.4892, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('nob', 'hye'): {'d_size': 1.3498, 'd_overlap_w': 0.0059, 'd_ttr': 0.1191, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5641, 'i_inv': 0.589, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('nob', 'slv'): {'d_size': 1.2531, 'd_overlap_w': 0.0189, 'd_ttr': 0.1918, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.5922, 'i_inv': 0.5251, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'hin'): {'d_size': 1.1879, 'd_overlap_w': 0.0073, 'd_ttr': 0.2188, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.4097, 'i_inv': 0.4505, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('nob', 'est'): {'d_size': 0.6785, 'd_overlap_w': 0.0148, 'd_ttr': 1.2842, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('nob', 'kur'): {'d_size': 0.6554, 'd_overlap_w': 0.0019, 'd_ttr': 0.2686, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('nob', 'glg'): {'d_size': 0.633, 'd_overlap_w': 0.0201, 'd_ttr': 0.1237, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.6121, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'mar'): {'d_size': 0.6218, 'd_overlap_w': 0.0024, 'd_ttr': 0.0106, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.4265, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('nob', 'tam'): {'d_size': 0.3933, 'd_overlap_w': 0.0057, 'd_ttr': 0.0791, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.4785, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('nob', 'urd'): {'d_size': 0.3777, 'd_overlap_w': 0.0062, 'd_ttr': 0.0092, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.5197, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('nob', 'azb'): {'d_size': 0.3757, 'd_overlap_w': 0.013, 'd_ttr': 3.1472, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.2871, 'i_inv': 0.5952, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('nob', 'bos'): {'d_size': 0.3579, 'd_overlap_w': 0.0147, 'd_ttr': 1.7326, 'd_overlap_s': '-', 'i_syn': 0.39, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('nob', 'csl'): {'d_size': 0.3497, 'd_overlap_w': 0.0051, 'd_ttr': 39.3516, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('nob', 'eus'): {'d_size': 0.3275, 'd_overlap_w': 0.0109, 'd_ttr': 2.2226, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.2952, 'i_inv': 0.5952, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('nob', 'ben'): {'d_size': 0.2938, 'd_overlap_w': 0.0011, 'd_ttr': 0.0378, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.2048, 'i_inv': 0.4769, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('nob', 'bel'): {'d_size': 0.2849, 'd_overlap_w': 0.0048, 'd_ttr': 2.1868, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('nob', 'kaz'): {'d_size': 0.2096, 'd_overlap_w': 0.0041, 'd_ttr': 5.9459, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 1.0, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('nob', 'kat'): {'d_size': 0.8337, 'd_overlap_w': 0.0049, 'd_ttr': 1.2637, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5348, 'i_inv': 0.6278, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('nob', 'zsm'): {'d_size': 0.3299, 'd_overlap_w': 0.0204, 'd_ttr': 0.0403, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5783, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}}, 'zsm': {('zsm', 'arb'): {'d_size': 41.0174, 'd_overlap_w': 0.0019, 'd_ttr': 0.1813, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}, ('zsm', 'heb'): {'d_size': 40.5784, 'd_overlap_w': 0.0022, 'd_ttr': 0.2617, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.0002, 'i_inv': 0.5245, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('zsm', 'rus'): {'d_size': 39.9345, 'd_overlap_w': 0.0028, 'd_ttr': 0.261, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.6162, 'i_inv': 0.4783, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('zsm', 'kor'): {'d_size': 39.3946, 'd_overlap_w': 0.0018, 'd_ttr': 0.0162, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.641, 'i_inv': 0.5715, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'ita'): {'d_size': 39.1768, 'd_overlap_w': 0.0116, 'd_ttr': 0.571, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4312, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'jpn'): {'d_size': 39.0977, 'd_overlap_w': 0.0005, 'd_ttr': 33.8011, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.6162, 'i_inv': 0.57, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'spa'): {'d_size': 37.5529, 'd_overlap_w': 0.0115, 'd_ttr': 0.5683, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('zsm', 'fra'): {'d_size': 36.8398, 'd_overlap_w': 0.0128, 'd_ttr': 0.6528, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5456, 'i_inv': 0.4891, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'nld'): {'d_size': 35.2044, 'd_overlap_w': 0.0106, 'd_ttr': 0.4874, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.0002, 'i_inv': 0.4829, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'tur'): {'d_size': 34.9559, 'd_overlap_w': 0.0068, 'd_ttr': 0.0593, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.484, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'ron'): {'d_size': 34.5755, 'd_overlap_w': 0.0103, 'd_ttr': 0.426, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.4445, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'pol'): {'d_size': 33.7489, 'd_overlap_w': 0.0068, 'd_ttr': 0.1618, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.6162, 'i_inv': 0.4094, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'bul'): {'d_size': 33.4184, 'd_overlap_w': 0.0032, 'd_ttr': 0.3696, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.6162, 'i_inv': 0.4697, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'vie'): {'d_size': 32.9492, 'd_overlap_w': 0.0312, 'd_ttr': 0.8685, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5456, 'i_inv': 0.5303, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'deu'): {'d_size': 32.1625, 'd_overlap_w': 0.0081, 'd_ttr': 0.3789, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5456, 'i_inv': 0.4481, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'pes'): {'d_size': 28.9205, 'd_overlap_w': 0.0048, 'd_ttr': 0.6045, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.5922, 'i_inv': 0.4685, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('zsm', 'hun'): {'d_size': 28.2029, 'd_overlap_w': 0.0048, 'd_ttr': 0.0188, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.4617, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'srp'): {'d_size': 26.2257, 'd_overlap_w': 0.0058, 'd_ttr': 0.1144, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.8278, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('zsm', 'ell'): {'d_size': 25.7331, 'd_overlap_w': 0.0049, 'd_ttr': 0.3378, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.5419, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('zsm', 'hrv'): {'d_size': 23.3891, 'd_overlap_w': 0.0092, 'd_ttr': 0.1765, 'd_overlap_s': '-', 'i_syn': 0.0, 'i_phon': 0.0002, 'i_inv': 0.4366, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.4}, ('zsm', 'ukr'): {'d_size': 20.7845, 'd_overlap_w': 0.0034, 'd_ttr': 0.1355, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4312, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'ces'): {'d_size': 19.7496, 'd_overlap_w': 0.0082, 'd_ttr': 0.1107, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.3923, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('zsm', 'tha'): {'d_size': 18.7862, 'd_overlap_w': 0.0019, 'd_ttr': 0.263, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.5687, 'i_inv': 0.5199, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'ind'): {'d_size': 16.7444, 'd_overlap_w': 0.121, 'd_ttr': 0.5093, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5687, 'i_inv': 0.3578, 'i_fam': 0.1111, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'slk'): {'d_size': 11.7759, 'd_overlap_w': 0.0089, 'd_ttr': 0.0211, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('zsm', 'swe'): {'d_size': 10.8519, 'd_overlap_w': 0.0163, 'd_ttr': 0.1849, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.5303, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'por'): {'d_size': 9.9205, 'd_overlap_w': 0.016, 'd_ttr': 0.2386, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4397, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.6}, ('zsm', 'dan'): {'d_size': 8.6092, 'd_overlap_w': 0.018, 'd_ttr': 0.2213, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}, ('zsm', 'lit'): {'d_size': 8.0305, 'd_overlap_w': 0.0084, 'd_ttr': 0.0143, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.4766, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'mkd'): {'d_size': 4.8534, 'd_overlap_w': 0.0055, 'd_ttr': 0.0077, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.3108, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.4}, ('zsm', 'fin'): {'d_size': 4.6402, 'd_overlap_w': 0.0081, 'd_ttr': 0.668, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'mya'): {'d_size': 4.1182, 'd_overlap_w': 0.0074, 'd_ttr': 0.0646, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5228, 'i_inv': 0.4772, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'hye'): {'d_size': 4.092, 'd_overlap_w': 0.0052, 'd_ttr': 0.0144, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('zsm', 'slv'): {'d_size': 3.799, 'd_overlap_w': 0.0135, 'd_ttr': 0.039, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.4413, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'hin'): {'d_size': 3.6011, 'd_overlap_w': 0.008, 'd_ttr': 0.31, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5922, 'i_inv': 0.5489, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'nob'): {'d_size': 3.0316, 'd_overlap_w': 0.0204, 'd_ttr': 0.028, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5783, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'est'): {'d_size': 2.0571, 'd_overlap_w': 0.0108, 'd_ttr': 0.6029, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}, ('zsm', 'kur'): {'d_size': 1.9868, 'd_overlap_w': 0.002, 'd_ttr': 0.0699, 'd_overlap_s': '-', 'i_syn': 0.91, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}, ('zsm', 'glg'): {'d_size': 1.919, 'd_overlap_w': 0.0182, 'd_ttr': 0.0158, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4914, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.6}, ('zsm', 'mar'): {'d_size': 1.8851, 'd_overlap_w': 0.0026, 'd_ttr': 0.0067, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5324, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('zsm', 'tam'): {'d_size': 1.1923, 'd_overlap_w': 0.0068, 'd_ttr': 0.1612, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5439, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'urd'): {'d_size': 1.145, 'd_overlap_w': 0.0092, 'd_ttr': 0.0612, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.0002, 'i_inv': 0.5464, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('zsm', 'azb'): {'d_size': 1.1391, 'd_overlap_w': 0.013, 'd_ttr': 1.7162, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.5922, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('zsm', 'bos'): {'d_size': 1.0851, 'd_overlap_w': 0.0137, 'd_ttr': 0.8628, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}, ('zsm', 'csl'): {'d_size': 1.0602, 'd_overlap_w': 0.0069, 'd_ttr': 25.5691, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'eus'): {'d_size': 0.9927, 'd_overlap_w': 0.0114, 'd_ttr': 1.1539, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.641, 'i_inv': 0.5132, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('zsm', 'ben'): {'d_size': 0.8906, 'd_overlap_w': 0.0018, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.6162, 'i_inv': 0.5867, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('zsm', 'bel'): {'d_size': 0.8638, 'd_overlap_w': 0.006, 'd_ttr': 1.1325, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('zsm', 'kaz'): {'d_size': 0.6354, 'd_overlap_w': 0.0059, 'd_ttr': 3.4719, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('zsm', 'kat'): {'d_size': 2.5274, 'd_overlap_w': 0.0045, 'd_ttr': 0.5911, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5687, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}}, 'dan': {('dan', 'arb'): {'d_size': 4.7644, 'd_overlap_w': 0.0055, 'd_ttr': 0.0071, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('dan', 'heb'): {'d_size': 4.7134, 'd_overlap_w': 0.0059, 'd_ttr': 0.006, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'rus'): {'d_size': 4.6386, 'd_overlap_w': 0.0096, 'd_ttr': 0.0058, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'kor'): {'d_size': 4.5759, 'd_overlap_w': 0.0064, 'd_ttr': 1.2744, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'ita'): {'d_size': 4.5506, 'd_overlap_w': 0.0448, 'd_ttr': 0.29, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.9, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'jpn'): {'d_size': 4.5414, 'd_overlap_w': 0.0018, 'd_ttr': 140.8013, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'spa'): {'d_size': 4.3619, 'd_overlap_w': 0.0411, 'd_ttr': 0.2864, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'fra'): {'d_size': 4.2791, 'd_overlap_w': 0.0516, 'd_ttr': 0.4062, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 0.9375, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'nld'): {'d_size': 4.0892, 'd_overlap_w': 0.0512, 'd_ttr': 0.1849, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 0.7, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'tur'): {'d_size': 4.0603, 'd_overlap_w': 0.0224, 'd_ttr': 0.1834, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'ron'): {'d_size': 4.0161, 'd_overlap_w': 0.0391, 'd_ttr': 0.1185, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 0.8889, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'pol'): {'d_size': 3.9201, 'd_overlap_w': 0.0263, 'd_ttr': 0.0165, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'bul'): {'d_size': 3.8817, 'd_overlap_w': 0.0091, 'd_ttr': 0.0675, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'vie'): {'d_size': 3.8272, 'd_overlap_w': 0.0726, 'd_ttr': 0.7595, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'deu'): {'d_size': 3.7358, 'd_overlap_w': 0.0378, 'd_ttr': 0.0751, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'pes'): {'d_size': 3.3593, 'd_overlap_w': 0.0123, 'd_ttr': 0.3362, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.9}, ('dan', 'hun'): {'d_size': 3.2759, 'd_overlap_w': 0.0187, 'd_ttr': 0.3961, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'srp'): {'d_size': 3.0462, 'd_overlap_w': 0.017, 'd_ttr': 0.0623, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.6}, ('dan', 'ell'): {'d_size': 2.989, 'd_overlap_w': 0.0154, 'd_ttr': 0.0438, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'hrv'): {'d_size': 2.7168, 'd_overlap_w': 0.0298, 'd_ttr': 0.009, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 0.875, 'i_geo': 0.1, 'i_fea': 0.9}, ('dan', 'ukr'): {'d_size': 2.4142, 'd_overlap_w': 0.0089, 'd_ttr': 0.0373, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'ces'): {'d_size': 2.294, 'd_overlap_w': 0.0277, 'd_ttr': 0.0676, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'tha'): {'d_size': 2.1821, 'd_overlap_w': 0.0069, 'd_ttr': 3.4465, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'ind'): {'d_size': 1.9449, 'd_overlap_w': 0.0597, 'd_ttr': 0.2109, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.8}, ('dan', 'slk'): {'d_size': 1.3678, 'd_overlap_w': 0.0258, 'd_ttr': 0.3765, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.8}, ('dan', 'swe'): {'d_size': 1.2605, 'd_overlap_w': 0.0752, 'd_ttr': 0.0058, 'd_overlap_s': '-', 'i_syn': 0.33, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 0.2857, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'por'): {'d_size': 1.1523, 'd_overlap_w': 0.0357, 'd_ttr': 0.0012, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'lit'): {'d_size': 0.9328, 'd_overlap_w': 0.0196, 'd_ttr': 1.2407, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.8}, ('dan', 'mkd'): {'d_size': 0.5638, 'd_overlap_w': 0.0088, 'd_ttr': 0.522, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'fin'): {'d_size': 0.539, 'd_overlap_w': 0.0174, 'd_ttr': 5.9118, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'mya'): {'d_size': 0.4783, 'd_overlap_w': 0.0153, 'd_ttr': 0.1666, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'hye'): {'d_size': 0.4753, 'd_overlap_w': 0.0068, 'd_ttr': 1.2434, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'slv'): {'d_size': 0.4413, 'd_overlap_w': 0.0236, 'd_ttr': 1.5901, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'hin'): {'d_size': 0.4183, 'd_overlap_w': 0.0066, 'd_ttr': 0.0266, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('dan', 'nob'): {'d_size': 0.3521, 'd_overlap_w': 0.1186, 'd_ttr': 0.3276, 'd_overlap_s': '-', 'i_syn': 0.22, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'est'): {'d_size': 0.2389, 'd_overlap_w': 0.0145, 'd_ttr': 5.5426, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('dan', 'kur'): {'d_size': 0.2308, 'd_overlap_w': 0.0016, 'd_ttr': 1.9247, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('dan', 'glg'): {'d_size': 0.2229, 'd_overlap_w': 0.0226, 'd_ttr': 1.2665, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'mar'): {'d_size': 0.219, 'd_overlap_w': 0.0019, 'd_ttr': 0.5387, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('dan', 'tam'): {'d_size': 0.1385, 'd_overlap_w': 0.0042, 'd_ttr': 0.0169, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'urd'): {'d_size': 0.133, 'd_overlap_w': 0.0053, 'd_ttr': 0.1775, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('dan', 'azb'): {'d_size': 0.1323, 'd_overlap_w': 0.012, 'd_ttr': 11.3019, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('dan', 'bos'): {'d_size': 0.126, 'd_overlap_w': 0.014, 'd_ttr': 6.9806, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.875, 'i_geo': 0.1, 'i_fea': 0.5}, ('dan', 'csl'): {'d_size': 0.1231, 'd_overlap_w': 0.0043, 'd_ttr': 108.9115, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'eus'): {'d_size': 0.1153, 'd_overlap_w': 0.0104, 'd_ttr': 8.5061, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'ben'): {'d_size': 0.1034, 'd_overlap_w': 0.001, 'd_ttr': 0.7707, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'bel'): {'d_size': 0.1003, 'd_overlap_w': 0.0039, 'd_ttr': 8.3959, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.8, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'kaz'): {'d_size': 0.0738, 'd_overlap_w': 0.0031, 'd_ttr': 19.4174, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('dan', 'kat'): {'d_size': 0.2936, 'd_overlap_w': 0.005, 'd_ttr': 5.4754, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'zsm'): {'d_size': 0.1162, 'd_overlap_w': 0.018, 'd_ttr': 0.7889, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}}, 'eus': {('eus', 'arb'): {'d_size': 41.3182, 'd_overlap_w': 0.0013, 'd_ttr': 0.523, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('eus', 'heb'): {'d_size': 40.8759, 'd_overlap_w': 0.0013, 'd_ttr': 0.5845, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.641, 'i_inv': 0.6169, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('eus', 'rus'): {'d_size': 40.2273, 'd_overlap_w': 0.0019, 'd_ttr': 0.584, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.2163, 'i_inv': 0.5499, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('eus', 'kor'): {'d_size': 39.6835, 'd_overlap_w': 0.0011, 'd_ttr': 0.2083, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.4601, 'i_inv': 0.6522, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('eus', 'ita'): {'d_size': 39.4641, 'd_overlap_w': 0.0088, 'd_ttr': 0.7782, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'jpn'): {'d_size': 39.3844, 'd_overlap_w': 0.0003, 'd_ttr': 5.2214, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5, 'i_inv': 0.57, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('eus', 'spa'): {'d_size': 37.8283, 'd_overlap_w': 0.0095, 'd_ttr': 0.7767, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.4278, 'i_inv': 0.5118, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('eus', 'fra'): {'d_size': 37.11, 'd_overlap_w': 0.0082, 'd_ttr': 0.8234, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.3918, 'i_inv': 0.5822, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('eus', 'nld'): {'d_size': 35.4626, 'd_overlap_w': 0.0069, 'd_ttr': 0.7301, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5776, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'tur'): {'d_size': 35.2123, 'd_overlap_w': 0.0039, 'd_ttr': 0.4036, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.2952, 'i_inv': 0.5245, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'ron'): {'d_size': 34.829, 'd_overlap_w': 0.0073, 'd_ttr': 0.6932, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.3217, 'i_inv': 0.5136, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'pol'): {'d_size': 33.9963, 'd_overlap_w': 0.0048, 'd_ttr': 0.5067, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.2163, 'i_inv': 0.5494, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'bul'): {'d_size': 33.6635, 'd_overlap_w': 0.0019, 'd_ttr': 0.6577, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.2163, 'i_inv': 0.5929, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'vie'): {'d_size': 33.1909, 'd_overlap_w': 0.0167, 'd_ttr': 0.9355, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.4934, 'i_inv': 0.6038, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('eus', 'deu'): {'d_size': 32.3983, 'd_overlap_w': 0.0054, 'd_ttr': 0.6637, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.3918, 'i_inv': 0.6038, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'pes'): {'d_size': 29.1326, 'd_overlap_w': 0.003, 'd_ttr': 0.797, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.2952, 'i_inv': 0.5136, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('eus', 'hun'): {'d_size': 28.4097, 'd_overlap_w': 0.0033, 'd_ttr': 0.341, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2952, 'i_inv': 0.568, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('eus', 'srp'): {'d_size': 26.418, 'd_overlap_w': 0.0041, 'd_ttr': 0.4637, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8391, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('eus', 'ell'): {'d_size': 25.9218, 'd_overlap_w': 0.0032, 'd_ttr': 0.637, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.2952, 'i_inv': 0.5648, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('eus', 'hrv'): {'d_size': 23.5606, 'd_overlap_w': 0.0063, 'd_ttr': 0.519, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.641, 'i_inv': 0.4627, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('eus', 'ukr'): {'d_size': 20.9369, 'd_overlap_w': 0.0021, 'd_ttr': 0.4836, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.641, 'i_inv': 0.5715, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'ces'): {'d_size': 19.8944, 'd_overlap_w': 0.0052, 'd_ttr': 0.4601, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.641, 'i_inv': 0.4967, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'tha'): {'d_size': 18.924, 'd_overlap_w': 0.0012, 'd_ttr': 0.0733, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.3498, 'i_inv': 0.5792, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('eus', 'ind'): {'d_size': 16.8672, 'd_overlap_w': 0.0134, 'd_ttr': 0.7429, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.3498, 'i_inv': 0.5132, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.5}, ('eus', 'slk'): {'d_size': 11.8622, 'd_overlap_w': 0.0055, 'd_ttr': 0.3457, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'swe'): {'d_size': 10.9315, 'd_overlap_w': 0.0095, 'd_ttr': 0.5259, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.641, 'i_inv': 0.6212, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('eus', 'por'): {'d_size': 9.9932, 'd_overlap_w': 0.0115, 'd_ttr': 0.5676, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.641, 'i_inv': 0.5462, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('eus', 'dan'): {'d_size': 8.6723, 'd_overlap_w': 0.0104, 'd_ttr': 0.5545, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('eus', 'lit'): {'d_size': 8.0893, 'd_overlap_w': 0.0058, 'd_ttr': 0.2119, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3217, 'i_inv': 0.5417, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'mkd'): {'d_size': 4.889, 'd_overlap_w': 0.0035, 'd_ttr': 0.3138, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.641, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'fin'): {'d_size': 4.6743, 'd_overlap_w': 0.0058, 'd_ttr': 0.0153, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.3498, 'i_inv': 0.5621, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'mya'): {'d_size': 4.1484, 'd_overlap_w': 0.0036, 'd_ttr': 0.4102, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4259, 'i_inv': 0.5762, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('eus', 'hye'): {'d_size': 4.122, 'd_overlap_w': 0.003, 'd_ttr': 0.2116, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5, 'i_inv': 0.4815, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('eus', 'slv'): {'d_size': 3.8269, 'd_overlap_w': 0.0089, 'd_ttr': 0.1787, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'hin'): {'d_size': 3.6276, 'd_overlap_w': 0.0039, 'd_ttr': 0.6183, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.2952, 'i_inv': 0.5327, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('eus', 'nob'): {'d_size': 3.0538, 'd_overlap_w': 0.0109, 'd_ttr': 0.3582, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.2952, 'i_inv': 0.5952, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('eus', 'est'): {'d_size': 2.0722, 'd_overlap_w': 0.0066, 'd_ttr': 0.0206, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('eus', 'kur'): {'d_size': 2.0014, 'd_overlap_w': 0.0015, 'd_ttr': 0.1524, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('eus', 'glg'): {'d_size': 1.933, 'd_overlap_w': 0.0139, 'd_ttr': 0.2092, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.535, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('eus', 'mar'): {'d_size': 1.8989, 'd_overlap_w': 0.0016, 'd_ttr': 0.3106, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.641, 'i_inv': 0.5519, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('eus', 'tam'): {'d_size': 1.2011, 'd_overlap_w': 0.0036, 'd_ttr': 0.5062, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5439, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('eus', 'urd'): {'d_size': 1.1534, 'd_overlap_w': 0.0041, 'd_ttr': 0.4059, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.5996, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('eus', 'azb'): {'d_size': 1.1474, 'd_overlap_w': 0.0074, 'd_ttr': 0.0129, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.2952, 'i_inv': 0.6572, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('eus', 'bos'): {'d_size': 1.093, 'd_overlap_w': 0.0088, 'd_ttr': 0.0049, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('eus', 'csl'): {'d_size': 1.0679, 'd_overlap_w': 0.0038, 'd_ttr': 3.6862, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('eus', 'ben'): {'d_size': 0.8971, 'd_overlap_w': 0.001, 'd_ttr': 0.2709, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.2163, 'i_inv': 0.5514, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('eus', 'bel'): {'d_size': 0.8701, 'd_overlap_w': 0.0035, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('eus', 'kaz'): {'d_size': 0.6401, 'd_overlap_w': 0.0035, 'd_ttr': 0.1447, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('eus', 'kat'): {'d_size': 2.5459, 'd_overlap_w': 0.0027, 'd_ttr': 0.0217, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.4638, 'i_inv': 0.535, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('eus', 'zsm'): {'d_size': 1.0073, 'd_overlap_w': 0.0114, 'd_ttr': 0.2682, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.641, 'i_inv': 0.5132, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}}, 'fin': {('fin', 'arb'): {'d_size': 8.8395, 'd_overlap_w': 0.0028, 'd_ttr': 0.4679, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('fin', 'heb'): {'d_size': 8.7449, 'd_overlap_w': 0.0032, 'd_ttr': 0.5347, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5687, 'i_inv': 0.552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('fin', 'rus'): {'d_size': 8.6061, 'd_overlap_w': 0.0048, 'd_ttr': 0.5341, 'd_overlap_s': '-', 'i_syn': 0.38, 'i_phon': 0.2804, 'i_inv': 0.5099, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'kor'): {'d_size': 8.4898, 'd_overlap_w': 0.003, 'd_ttr': 0.1441, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.4638, 'i_inv': 0.3638, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('fin', 'ita'): {'d_size': 8.4429, 'd_overlap_w': 0.0166, 'd_ttr': 0.7491, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.4906, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'jpn'): {'d_size': 8.4258, 'd_overlap_w': 0.001, 'd_ttr': 7.5596, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5032, 'i_inv': 0.5146, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'spa'): {'d_size': 8.0929, 'd_overlap_w': 0.0155, 'd_ttr': 0.7474, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4477, 'i_inv': 0.5617, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'fra'): {'d_size': 7.9392, 'd_overlap_w': 0.017, 'd_ttr': 0.7998, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3277, 'i_inv': 0.3996, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'nld'): {'d_size': 7.5868, 'd_overlap_w': 0.015, 'd_ttr': 0.6954, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5687, 'i_inv': 0.4561, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'tur'): {'d_size': 7.5332, 'd_overlap_w': 0.0091, 'd_ttr': 0.3408, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.195, 'i_inv': 0.3885, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'ron'): {'d_size': 7.4512, 'd_overlap_w': 0.014, 'd_ttr': 0.6543, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.3498, 'i_inv': 0.4795, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'pol'): {'d_size': 7.2731, 'd_overlap_w': 0.0102, 'd_ttr': 0.4504, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.2804, 'i_inv': 0.5146, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'bul'): {'d_size': 7.2019, 'd_overlap_w': 0.0049, 'd_ttr': 0.6151, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.2804, 'i_inv': 0.4599, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'vie'): {'d_size': 7.1008, 'd_overlap_w': 0.0261, 'd_ttr': 0.9265, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.427, 'i_inv': 0.5211, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'deu'): {'d_size': 6.9312, 'd_overlap_w': 0.0128, 'd_ttr': 0.6217, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.1864, 'i_inv': 0.4201, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'pes'): {'d_size': 6.2326, 'd_overlap_w': 0.0063, 'd_ttr': 0.7701, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3433, 'i_inv': 0.4572, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'hun'): {'d_size': 6.0779, 'd_overlap_w': 0.0082, 'd_ttr': 0.2758, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.195, 'i_inv': 0.3235, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'srp'): {'d_size': 5.6518, 'd_overlap_w': 0.007, 'd_ttr': 0.4043, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8632, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('fin', 'ell'): {'d_size': 5.5457, 'd_overlap_w': 0.0069, 'd_ttr': 0.5922, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.3433, 'i_inv': 0.5463, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'hrv'): {'d_size': 5.0405, 'd_overlap_w': 0.012, 'd_ttr': 0.4636, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.448, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'ukr'): {'d_size': 4.4792, 'd_overlap_w': 0.0047, 'd_ttr': 0.4255, 'd_overlap_s': '-', 'i_syn': 0.48, 'i_phon': 0.5687, 'i_inv': 0.4906, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'ces'): {'d_size': 4.2562, 'd_overlap_w': 0.0114, 'd_ttr': 0.4004, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.4818, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'tha'): {'d_size': 4.0486, 'd_overlap_w': 0.0033, 'd_ttr': 0.0281, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.39, 'i_inv': 0.3995, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('fin', 'ind'): {'d_size': 3.6085, 'd_overlap_w': 0.0211, 'd_ttr': 0.7097, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2736, 'i_inv': 0.4034, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('fin', 'slk'): {'d_size': 2.5378, 'd_overlap_w': 0.0114, 'd_ttr': 0.2806, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'swe'): {'d_size': 2.3387, 'd_overlap_w': 0.0162, 'd_ttr': 0.4711, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5687, 'i_inv': 0.4418, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('fin', 'por'): {'d_size': 2.1379, 'd_overlap_w': 0.0141, 'd_ttr': 0.5163, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.454, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'dan'): {'d_size': 1.8553, 'd_overlap_w': 0.0174, 'd_ttr': 0.5021, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('fin', 'lit'): {'d_size': 1.7306, 'd_overlap_w': 0.0099, 'd_ttr': 0.1474, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3498, 'i_inv': 0.4657, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'mkd'): {'d_size': 1.0459, 'd_overlap_w': 0.0048, 'd_ttr': 0.248, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'mya'): {'d_size': 0.8875, 'd_overlap_w': 0.0069, 'd_ttr': 0.3477, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.3695, 'i_inv': 0.448, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'hye'): {'d_size': 0.8818, 'd_overlap_w': 0.0036, 'd_ttr': 0.1472, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5032, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'slv'): {'d_size': 0.8187, 'd_overlap_w': 0.011, 'd_ttr': 0.1163, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'hin'): {'d_size': 0.7761, 'd_overlap_w': 0.0042, 'd_ttr': 0.5717, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3433, 'i_inv': 0.5129, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('fin', 'nob'): {'d_size': 0.6533, 'd_overlap_w': 0.0133, 'd_ttr': 0.2935, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.3433, 'i_inv': 0.4626, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('fin', 'est'): {'d_size': 0.4433, 'd_overlap_w': 0.0113, 'd_ttr': 0.0005, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('fin', 'kur'): {'d_size': 0.4282, 'd_overlap_w': 0.0011, 'd_ttr': 0.0926, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('fin', 'glg'): {'d_size': 0.4135, 'd_overlap_w': 0.01, 'd_ttr': 0.1449, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5687, 'i_inv': 0.5011, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('fin', 'mar'): {'d_size': 0.4062, 'd_overlap_w': 0.0012, 'd_ttr': 0.2447, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5687, 'i_inv': 0.4824, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'tam'): {'d_size': 0.257, 'd_overlap_w': 0.003, 'd_ttr': 0.4498, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5687, 'i_inv': 0.5338, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'urd'): {'d_size': 0.2468, 'd_overlap_w': 0.0029, 'd_ttr': 0.3432, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5687, 'i_inv': 0.5551, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'azb'): {'d_size': 0.2455, 'd_overlap_w': 0.0059, 'd_ttr': 0.0735, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.195, 'i_inv': 0.5207, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('fin', 'bos'): {'d_size': 0.2338, 'd_overlap_w': 0.0062, 'd_ttr': 0.0038, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('fin', 'csl'): {'d_size': 0.2285, 'd_overlap_w': 0.0029, 'd_ttr': 5.4418, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'eus'): {'d_size': 0.2139, 'd_overlap_w': 0.0058, 'd_ttr': 0.02, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.3498, 'i_inv': 0.5621, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'ben'): {'d_size': 0.1919, 'd_overlap_w': 0.0005, 'd_ttr': 0.205, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4054, 'i_inv': 0.5071, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'bel'): {'d_size': 0.1862, 'd_overlap_w': 0.0025, 'd_ttr': 0.0185, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('fin', 'kaz'): {'d_size': 0.1369, 'd_overlap_w': 0.0021, 'd_ttr': 0.3313, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.805, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('fin', 'kat'): {'d_size': 0.5447, 'd_overlap_w': 0.0032, 'd_ttr': 0.0007, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4816, 'i_inv': 0.5823, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'zsm'): {'d_size': 0.2155, 'd_overlap_w': 0.0081, 'd_ttr': 0.2023, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}}, 'bos': {('bos', 'arb'): {'d_size': 37.8021, 'd_overlap_w': 0.0015, 'd_ttr': 0.4933, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('bos', 'heb'): {'d_size': 37.3974, 'd_overlap_w': 0.0017, 'd_ttr': 0.5577, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'rus'): {'d_size': 36.804, 'd_overlap_w': 0.0024, 'd_ttr': 0.5571, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 0.4, 'i_geo': 0.2, 'i_fea': 0.7}, ('bos', 'kor'): {'d_size': 36.3065, 'd_overlap_w': 0.0015, 'd_ttr': 0.1726, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'ita'): {'d_size': 36.1058, 'd_overlap_w': 0.0106, 'd_ttr': 0.7627, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'jpn'): {'d_size': 36.0328, 'd_overlap_w': 0.0005, 'd_ttr': 6.414, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.8}, ('bos', 'spa'): {'d_size': 34.6091, 'd_overlap_w': 0.0094, 'd_ttr': 0.761, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'fra'): {'d_size': 33.952, 'd_overlap_w': 0.0101, 'd_ttr': 0.8108, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 0.9375, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'nld'): {'d_size': 32.4447, 'd_overlap_w': 0.0085, 'd_ttr': 0.7115, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 0.9, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'tur'): {'d_size': 32.2157, 'd_overlap_w': 0.0053, 'd_ttr': 0.3695, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'ron'): {'d_size': 31.8651, 'd_overlap_w': 0.0099, 'd_ttr': 0.6723, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 0.8889, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'pol'): {'d_size': 31.1033, 'd_overlap_w': 0.0086, 'd_ttr': 0.4763, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'bul'): {'d_size': 30.7987, 'd_overlap_w': 0.0029, 'd_ttr': 0.6348, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'vie'): {'d_size': 30.3663, 'd_overlap_w': 0.0201, 'd_ttr': 0.9307, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'deu'): {'d_size': 29.6412, 'd_overlap_w': 0.0065, 'd_ttr': 0.6411, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'pes'): {'d_size': 26.6534, 'd_overlap_w': 0.0042, 'd_ttr': 0.7826, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('bos', 'hun'): {'d_size': 25.9921, 'd_overlap_w': 0.0041, 'd_ttr': 0.3054, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'srp'): {'d_size': 24.1698, 'd_overlap_w': 0.0763, 'd_ttr': 0.4315, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 0.125, 'i_geo': 0.0, 'i_fea': 0.6}, ('bos', 'ell'): {'d_size': 23.7159, 'd_overlap_w': 0.0041, 'd_ttr': 0.6129, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'hrv'): {'d_size': 21.5556, 'd_overlap_w': 0.1028, 'd_ttr': 0.4891, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 0.125, 'i_geo': 0.0, 'i_fea': 0.9}, ('bos', 'ukr'): {'d_size': 19.1552, 'd_overlap_w': 0.0028, 'd_ttr': 0.4522, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'ces'): {'d_size': 18.2014, 'd_overlap_w': 0.012, 'd_ttr': 0.4278, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'tha'): {'d_size': 17.3136, 'd_overlap_w': 0.0016, 'd_ttr': 0.0465, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'ind'): {'d_size': 15.4319, 'd_overlap_w': 0.0175, 'd_ttr': 0.7251, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.8}, ('bos', 'slk'): {'d_size': 10.8528, 'd_overlap_w': 0.0155, 'd_ttr': 0.3102, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.625, 'i_geo': 0.0, 'i_fea': 0.8}, ('bos', 'swe'): {'d_size': 10.0012, 'd_overlap_w': 0.0119, 'd_ttr': 0.4963, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'por'): {'d_size': 9.1428, 'd_overlap_w': 0.0118, 'd_ttr': 0.5399, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'dan'): {'d_size': 7.9343, 'd_overlap_w': 0.014, 'd_ttr': 0.5263, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.875, 'i_geo': 0.1, 'i_fea': 0.5}, ('bos', 'lit'): {'d_size': 7.401, 'd_overlap_w': 0.0088, 'd_ttr': 0.1761, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'mkd'): {'d_size': 4.473, 'd_overlap_w': 0.0046, 'd_ttr': 0.2778, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.8}, ('bos', 'fin'): {'d_size': 4.2765, 'd_overlap_w': 0.0062, 'd_ttr': 0.0033, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'mya'): {'d_size': 3.7954, 'd_overlap_w': 0.004, 'd_ttr': 0.3762, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('bos', 'hye'): {'d_size': 3.7712, 'd_overlap_w': 0.0039, 'd_ttr': 0.1758, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'slv'): {'d_size': 3.5012, 'd_overlap_w': 0.0606, 'd_ttr': 0.1438, 'd_overlap_s': '-', 'i_syn': 0.36, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.1667, 'i_geo': 0.0, 'i_fea': 0.6}, ('bos', 'hin'): {'d_size': 3.3189, 'd_overlap_w': 0.0052, 'd_ttr': 0.5932, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'nob'): {'d_size': 2.794, 'd_overlap_w': 0.0147, 'd_ttr': 0.3229, 'd_overlap_s': '-', 'i_syn': 0.39, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'est'): {'d_size': 1.8958, 'd_overlap_w': 0.0093, 'd_ttr': 0.0062, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('bos', 'kur'): {'d_size': 1.831, 'd_overlap_w': 0.0016, 'd_ttr': 0.1187, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'glg'): {'d_size': 1.7685, 'd_overlap_w': 0.0128, 'd_ttr': 0.1734, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'mar'): {'d_size': 1.7373, 'd_overlap_w': 0.0017, 'd_ttr': 0.2745, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'tam'): {'d_size': 1.0989, 'd_overlap_w': 0.0041, 'd_ttr': 0.4757, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'urd'): {'d_size': 1.0553, 'd_overlap_w': 0.0049, 'd_ttr': 0.3718, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('bos', 'azb'): {'d_size': 1.0498, 'd_overlap_w': 0.0095, 'd_ttr': 0.0391, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'csl'): {'d_size': 0.977, 'd_overlap_w': 0.0037, 'd_ttr': 4.5795, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'eus'): {'d_size': 0.9149, 'd_overlap_w': 0.0088, 'd_ttr': 0.0057, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'ben'): {'d_size': 0.8208, 'd_overlap_w': 0.0011, 'd_ttr': 0.2346, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'bel'): {'d_size': 0.7961, 'd_overlap_w': 0.0038, 'd_ttr': 0.0049, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.4, 'i_geo': 0.1, 'i_fea': 0.6}, ('bos', 'kaz'): {'d_size': 0.5856, 'd_overlap_w': 0.0035, 'd_ttr': 0.2347, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('bos', 'kat'): {'d_size': 2.3293, 'd_overlap_w': 0.0031, 'd_ttr': 0.0069, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'zsm'): {'d_size': 0.9216, 'd_overlap_w': 0.0137, 'd_ttr': 0.2319, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}}, 'est': {('est', 'arb'): {'d_size': 19.9396, 'd_overlap_w': 0.002, 'd_ttr': 0.458, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('est', 'heb'): {'d_size': 19.7261, 'd_overlap_w': 0.0023, 'd_ttr': 0.5257, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('est', 'rus'): {'d_size': 19.4131, 'd_overlap_w': 0.0032, 'd_ttr': 0.5251, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('est', 'kor'): {'d_size': 19.1507, 'd_overlap_w': 0.002, 'd_ttr': 0.1335, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('est', 'ita'): {'d_size': 19.0448, 'd_overlap_w': 0.0118, 'd_ttr': 0.7438, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'jpn'): {'d_size': 19.0063, 'd_overlap_w': 0.0006, 'd_ttr': 8.0411, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'spa'): {'d_size': 18.2554, 'd_overlap_w': 0.0114, 'd_ttr': 0.742, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'fra'): {'d_size': 17.9087, 'd_overlap_w': 0.0129, 'd_ttr': 0.7955, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'nld'): {'d_size': 17.1137, 'd_overlap_w': 0.0113, 'd_ttr': 0.689, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'tur'): {'d_size': 16.9929, 'd_overlap_w': 0.0063, 'd_ttr': 0.3297, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'ron'): {'d_size': 16.808, 'd_overlap_w': 0.0101, 'd_ttr': 0.6472, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'pol'): {'d_size': 16.4061, 'd_overlap_w': 0.0075, 'd_ttr': 0.4403, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'bul'): {'d_size': 16.2455, 'd_overlap_w': 0.0036, 'd_ttr': 0.6073, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'vie'): {'d_size': 16.0174, 'd_overlap_w': 0.0219, 'd_ttr': 0.9249, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'deu'): {'d_size': 15.6349, 'd_overlap_w': 0.0092, 'd_ttr': 0.614, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'pes'): {'d_size': 14.0589, 'd_overlap_w': 0.005, 'd_ttr': 0.7652, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('est', 'hun'): {'d_size': 13.7101, 'd_overlap_w': 0.0056, 'd_ttr': 0.2645, 'd_overlap_s': '-', 'i_syn': 0.48, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'srp'): {'d_size': 12.7489, 'd_overlap_w': 0.0062, 'd_ttr': 0.3937, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('est', 'ell'): {'d_size': 12.5095, 'd_overlap_w': 0.0052, 'd_ttr': 0.5841, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'hrv'): {'d_size': 11.37, 'd_overlap_w': 0.0098, 'd_ttr': 0.4537, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('est', 'ukr'): {'d_size': 10.1038, 'd_overlap_w': 0.0032, 'd_ttr': 0.4152, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'ces'): {'d_size': 9.6008, 'd_overlap_w': 0.0087, 'd_ttr': 0.3898, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'tha'): {'d_size': 9.1324, 'd_overlap_w': 0.002, 'd_ttr': 0.022, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'ind'): {'d_size': 8.1399, 'd_overlap_w': 0.0181, 'd_ttr': 0.7036, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('est', 'slk'): {'d_size': 5.7245, 'd_overlap_w': 0.0094, 'd_ttr': 0.2693, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'swe'): {'d_size': 5.2754, 'd_overlap_w': 0.0137, 'd_ttr': 0.4612, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'por'): {'d_size': 4.8226, 'd_overlap_w': 0.0118, 'd_ttr': 0.507, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'dan'): {'d_size': 4.1851, 'd_overlap_w': 0.0145, 'd_ttr': 0.4926, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('est', 'lit'): {'d_size': 3.9038, 'd_overlap_w': 0.0078, 'd_ttr': 0.1368, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'mkd'): {'d_size': 2.3594, 'd_overlap_w': 0.0043, 'd_ttr': 0.2367, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'fin'): {'d_size': 2.2557, 'd_overlap_w': 0.0113, 'd_ttr': 0.0005, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'mya'): {'d_size': 2.002, 'd_overlap_w': 0.0055, 'd_ttr': 0.3366, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('est', 'hye'): {'d_size': 1.9892, 'd_overlap_w': 0.0038, 'd_ttr': 0.1365, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'slv'): {'d_size': 1.8468, 'd_overlap_w': 0.011, 'd_ttr': 0.1062, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'hin'): {'d_size': 1.7506, 'd_overlap_w': 0.0043, 'd_ttr': 0.5632, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'nob'): {'d_size': 1.4737, 'd_overlap_w': 0.0148, 'd_ttr': 0.2822, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('est', 'kur'): {'d_size': 0.9658, 'd_overlap_w': 0.0013, 'd_ttr': 0.0831, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('est', 'glg'): {'d_size': 0.9329, 'd_overlap_w': 0.0117, 'd_ttr': 0.1342, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('est', 'mar'): {'d_size': 0.9164, 'd_overlap_w': 0.0015, 'd_ttr': 0.2333, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'tam'): {'d_size': 0.5796, 'd_overlap_w': 0.0035, 'd_ttr': 0.4397, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'urd'): {'d_size': 0.5566, 'd_overlap_w': 0.0039, 'd_ttr': 0.3321, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'azb'): {'d_size': 0.5537, 'd_overlap_w': 0.0077, 'd_ttr': 0.0902, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('est', 'bos'): {'d_size': 0.5275, 'd_overlap_w': 0.0093, 'd_ttr': 0.0074, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('est', 'csl'): {'d_size': 0.5154, 'd_overlap_w': 0.0036, 'd_ttr': 5.8052, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'eus'): {'d_size': 0.4826, 'd_overlap_w': 0.0066, 'd_ttr': 0.0281, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'ben'): {'d_size': 0.4329, 'd_overlap_w': 0.0008, 'd_ttr': 0.1937, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'bel'): {'d_size': 0.4199, 'd_overlap_w': 0.0033, 'd_ttr': 0.0262, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('est', 'kaz'): {'d_size': 0.3089, 'd_overlap_w': 0.0032, 'd_ttr': 0.3743, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('est', 'kat'): {'d_size': 1.2286, 'd_overlap_w': 0.0032, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'zsm'): {'d_size': 0.4861, 'd_overlap_w': 0.0108, 'd_ttr': 0.191, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}}, 'azb': {('azb', 'arb'): {'d_size': 36.0092, 'd_overlap_w': 0.0016, 'd_ttr': 0.5647, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('azb', 'heb'): {'d_size': 35.6238, 'd_overlap_w': 0.0017, 'd_ttr': 0.6218, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5922, 'i_inv': 0.4734, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'rus'): {'d_size': 35.0585, 'd_overlap_w': 0.0025, 'd_ttr': 0.6213, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.2048, 'i_inv': 0.5722, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'kor'): {'d_size': 34.5846, 'd_overlap_w': 0.0016, 'd_ttr': 0.2621, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5319, 'i_inv': 0.5715, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('azb', 'ita'): {'d_size': 34.3934, 'd_overlap_w': 0.0088, 'd_ttr': 0.7996, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.5922, 'i_inv': 0.5924, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'jpn'): {'d_size': 34.3239, 'd_overlap_w': 0.0005, 'd_ttr': 3.8012, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4717, 'i_inv': 0.6099, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('azb', 'spa'): {'d_size': 32.9677, 'd_overlap_w': 0.0088, 'd_ttr': 0.7982, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.4097, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'fra'): {'d_size': 32.3417, 'd_overlap_w': 0.0096, 'd_ttr': 0.8406, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.2677, 'i_inv': 0.5822, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'nld'): {'d_size': 30.906, 'd_overlap_w': 0.0081, 'd_ttr': 0.7557, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.4622, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'tur'): {'d_size': 30.6879, 'd_overlap_w': 0.0196, 'd_ttr': 0.4523, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.0, 'i_inv': 0.4175, 'i_fam': 0.3333, 'i_geo': 0.1, 'i_fea': 0.4}, ('azb', 'ron'): {'d_size': 30.3539, 'd_overlap_w': 0.0078, 'd_ttr': 0.7219, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.2952, 'i_inv': 0.5761, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'pol'): {'d_size': 29.6282, 'd_overlap_w': 0.0055, 'd_ttr': 0.5494, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.2048, 'i_inv': 0.6291, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'bul'): {'d_size': 29.338, 'd_overlap_w': 0.0027, 'd_ttr': 0.6894, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.2048, 'i_inv': 0.5541, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'vie'): {'d_size': 28.9262, 'd_overlap_w': 0.0188, 'd_ttr': 0.942, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.4788, 'i_inv': 0.4906, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('azb', 'deu'): {'d_size': 28.2355, 'd_overlap_w': 0.0065, 'd_ttr': 0.6949, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.2677, 'i_inv': 0.4481, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('azb', 'pes'): {'d_size': 25.3893, 'd_overlap_w': 0.0041, 'd_ttr': 0.8166, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.2871, 'i_inv': 0.535, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'hun'): {'d_size': 24.7593, 'd_overlap_w': 0.0039, 'd_ttr': 0.3924, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0, 'i_inv': 0.5275, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('azb', 'srp'): {'d_size': 23.0235, 'd_overlap_w': 0.0051, 'd_ttr': 0.5091, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.8564, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('azb', 'ell'): {'d_size': 22.5912, 'd_overlap_w': 0.0041, 'd_ttr': 0.6703, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.2871, 'i_inv': 0.587, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'hrv'): {'d_size': 20.5333, 'd_overlap_w': 0.0075, 'd_ttr': 0.561, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.5922, 'i_inv': 0.5793, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('azb', 'ukr'): {'d_size': 18.2467, 'd_overlap_w': 0.0026, 'd_ttr': 0.5278, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5922, 'i_inv': 0.528, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'ces'): {'d_size': 17.3382, 'd_overlap_w': 0.0063, 'd_ttr': 0.5057, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.4967, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'tha'): {'d_size': 16.4924, 'd_overlap_w': 0.0017, 'd_ttr': 0.1191, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.4477, 'i_inv': 0.56, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('azb', 'ind'): {'d_size': 14.7, 'd_overlap_w': 0.0159, 'd_ttr': 0.7674, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.3433, 'i_inv': 0.578, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('azb', 'slk'): {'d_size': 10.338, 'd_overlap_w': 0.0067, 'd_ttr': 0.397, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('azb', 'swe'): {'d_size': 9.5269, 'd_overlap_w': 0.0109, 'd_ttr': 0.5674, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.5922, 'i_inv': 0.5493, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'por'): {'d_size': 8.7092, 'd_overlap_w': 0.0099, 'd_ttr': 0.6062, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5922, 'i_inv': 0.5656, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'dan'): {'d_size': 7.558, 'd_overlap_w': 0.012, 'd_ttr': 0.594, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('azb', 'lit'): {'d_size': 7.0499, 'd_overlap_w': 0.0061, 'd_ttr': 0.2656, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.2952, 'i_inv': 0.5621, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'mkd'): {'d_size': 4.2608, 'd_overlap_w': 0.004, 'd_ttr': 0.3661, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5922, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('azb', 'fin'): {'d_size': 4.0737, 'd_overlap_w': 0.0059, 'd_ttr': 0.0455, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.195, 'i_inv': 0.5207, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'mya'): {'d_size': 3.6154, 'd_overlap_w': 0.0048, 'd_ttr': 0.4586, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.4208, 'i_inv': 0.5762, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'hye'): {'d_size': 3.5923, 'd_overlap_w': 0.0036, 'd_ttr': 0.2653, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.4717, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('azb', 'slv'): {'d_size': 3.3352, 'd_overlap_w': 0.0102, 'd_ttr': 0.232, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5922, 'i_inv': 0.557, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'hin'): {'d_size': 3.1615, 'd_overlap_w': 0.0046, 'd_ttr': 0.6531, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2871, 'i_inv': 0.6107, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'nob'): {'d_size': 2.6615, 'd_overlap_w': 0.013, 'd_ttr': 0.409, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.2871, 'i_inv': 0.5952, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'est'): {'d_size': 1.8059, 'd_overlap_w': 0.0077, 'd_ttr': 0.0534, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('azb', 'kur'): {'d_size': 1.7442, 'd_overlap_w': 0.0017, 'd_ttr': 0.2049, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.9}, ('azb', 'glg'): {'d_size': 1.6847, 'd_overlap_w': 0.0119, 'd_ttr': 0.2629, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.5922, 'i_inv': 0.6341, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'mar'): {'d_size': 1.6549, 'd_overlap_w': 0.0018, 'd_ttr': 0.363, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.6604, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'tam'): {'d_size': 1.0468, 'd_overlap_w': 0.0039, 'd_ttr': 0.549, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5922, 'i_inv': 0.668, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'urd'): {'d_size': 1.0052, 'd_overlap_w': 0.0049, 'd_ttr': 0.4545, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5996, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('azb', 'bos'): {'d_size': 0.9526, 'd_overlap_w': 0.0095, 'd_ttr': 0.0272, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('azb', 'csl'): {'d_size': 0.9307, 'd_overlap_w': 0.0039, 'd_ttr': 2.6304, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('azb', 'eus'): {'d_size': 0.8715, 'd_overlap_w': 0.0074, 'd_ttr': 0.0104, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.2952, 'i_inv': 0.6572, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('azb', 'ben'): {'d_size': 0.7819, 'd_overlap_w': 0.0012, 'd_ttr': 0.3243, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.3613, 'i_inv': 0.6532, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('azb', 'bel'): {'d_size': 0.7583, 'd_overlap_w': 0.0034, 'd_ttr': 0.0113, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('azb', 'kaz'): {'d_size': 0.5579, 'd_overlap_w': 0.0035, 'd_ttr': 0.0574, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 1.0, 'i_inv': 0.669, 'i_fam': 0.3333, 'i_geo': 0.1, 'i_fea': 0.9}, ('azb', 'kat'): {'d_size': 2.2188, 'd_overlap_w': 0.003, 'd_ttr': 0.0549, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.4477, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('azb', 'zsm'): {'d_size': 0.8779, 'd_overlap_w': 0.013, 'd_ttr': 0.3216, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.5922, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}}, 'hye': {('hye', 'arb'): {'d_size': 10.0239, 'd_overlap_w': 0.0022, 'd_ttr': 0.2376, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('hye', 'heb'): {'d_size': 9.9166, 'd_overlap_w': 0.0025, 'd_ttr': 0.318, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.4911, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'rus'): {'d_size': 9.7593, 'd_overlap_w': 0.0031, 'd_ttr': 0.3173, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.4327, 'i_inv': 0.4693, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'kor'): {'d_size': 9.6273, 'd_overlap_w': 0.0015, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5, 'i_inv': 0.5429, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hye', 'ita'): {'d_size': 9.5741, 'd_overlap_w': 0.0056, 'd_ttr': 0.6113, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.6162, 'i_inv': 0.5203, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'jpn'): {'d_size': 9.5548, 'd_overlap_w': 0.0006, 'd_ttr': 25.8367, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5354, 'i_inv': 0.5204, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('hye', 'spa'): {'d_size': 9.1772, 'd_overlap_w': 0.0056, 'd_ttr': 0.6088, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.4717, 'i_inv': 0.5037, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'fra'): {'d_size': 9.003, 'd_overlap_w': 0.0062, 'd_ttr': 0.6865, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.4407, 'i_inv': 0.5006, 'i_fam': 0.9375, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'nld'): {'d_size': 8.6033, 'd_overlap_w': 0.0051, 'd_ttr': 0.5336, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.6162, 'i_inv': 0.5339, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'tur'): {'d_size': 8.5426, 'd_overlap_w': 0.0031, 'd_ttr': 0.1055, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.4717, 'i_inv': 0.4963, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'ron'): {'d_size': 8.4496, 'd_overlap_w': 0.0049, 'd_ttr': 0.476, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.3824, 'i_inv': 0.3812, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.4}, ('hye', 'pol'): {'d_size': 8.2476, 'd_overlap_w': 0.0037, 'd_ttr': 0.2175, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4327, 'i_inv': 0.5204, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'bul'): {'d_size': 8.1669, 'd_overlap_w': 0.0035, 'd_ttr': 0.4225, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.4327, 'i_inv': 0.5052, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'vie'): {'d_size': 8.0522, 'd_overlap_w': 0.0108, 'd_ttr': 0.8822, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5295, 'i_inv': 0.5422, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'deu'): {'d_size': 7.8599, 'd_overlap_w': 0.0046, 'd_ttr': 0.4314, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4407, 'i_inv': 0.4819, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'pes'): {'d_size': 7.0676, 'd_overlap_w': 0.0041, 'd_ttr': 0.6422, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.4717, 'i_inv': 0.3203, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'hun'): {'d_size': 6.8923, 'd_overlap_w': 0.0028, 'd_ttr': 0.0527, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.4717, 'i_inv': 0.4756, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'srp'): {'d_size': 6.4091, 'd_overlap_w': 0.003, 'd_ttr': 0.1675, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 1.0, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.8}, ('hye', 'ell'): {'d_size': 6.2887, 'd_overlap_w': 0.0043, 'd_ttr': 0.392, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4717, 'i_inv': 0.5346, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'hrv'): {'d_size': 5.7159, 'd_overlap_w': 0.0043, 'd_ttr': 0.2326, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.6162, 'i_inv': 0.4263, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'ukr'): {'d_size': 5.0794, 'd_overlap_w': 0.0037, 'd_ttr': 0.19, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.4726, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'ces'): {'d_size': 4.8265, 'd_overlap_w': 0.0042, 'd_ttr': 0.1634, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.6162, 'i_inv': 0.4882, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'tha'): {'d_size': 4.591, 'd_overlap_w': 0.0017, 'd_ttr': 0.1229, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5879, 'i_inv': 0.5531, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'ind'): {'d_size': 4.092, 'd_overlap_w': 0.0084, 'd_ttr': 0.554, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5032, 'i_inv': 0.404, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('hye', 'slk'): {'d_size': 2.8778, 'd_overlap_w': 0.0044, 'd_ttr': 0.0562, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'swe'): {'d_size': 2.652, 'd_overlap_w': 0.0061, 'd_ttr': 0.2412, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.5797, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'por'): {'d_size': 2.4244, 'd_overlap_w': 0.0058, 'd_ttr': 0.2952, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.4975, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'dan'): {'d_size': 2.1039, 'd_overlap_w': 0.0068, 'd_ttr': 0.2779, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.7}, ('hye', 'lit'): {'d_size': 1.9625, 'd_overlap_w': 0.0045, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5, 'i_inv': 0.4906, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'mkd'): {'d_size': 1.1861, 'd_overlap_w': 0.0043, 'd_ttr': 0.0345, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.3926, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'fin'): {'d_size': 1.134, 'd_overlap_w': 0.0036, 'd_ttr': 0.3873, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5032, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hye', 'mya'): {'d_size': 1.0064, 'd_overlap_w': 0.0022, 'd_ttr': 0.1117, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5519, 'i_inv': 0.5308, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'slv'): {'d_size': 0.9284, 'd_overlap_w': 0.0046, 'd_ttr': 0.0048, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'hin'): {'d_size': 0.8801, 'd_overlap_w': 0.0039, 'd_ttr': 0.3652, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.4717, 'i_inv': 0.5081, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.5}, ('hye', 'nob'): {'d_size': 0.7409, 'd_overlap_w': 0.0059, 'd_ttr': 0.0658, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5641, 'i_inv': 0.589, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'est'): {'d_size': 0.5027, 'd_overlap_w': 0.0038, 'd_ttr': 0.3433, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('hye', 'kur'): {'d_size': 0.4855, 'd_overlap_w': 0.0012, 'd_ttr': 0.0166, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('hye', 'glg'): {'d_size': 0.469, 'd_overlap_w': 0.0049, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5056, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'mar'): {'d_size': 0.4607, 'd_overlap_w': 0.0011, 'd_ttr': 0.0325, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5448, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'tam'): {'d_size': 0.2914, 'd_overlap_w': 0.0034, 'd_ttr': 0.2169, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.5747, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'urd'): {'d_size': 0.2798, 'd_overlap_w': 0.0035, 'd_ttr': 0.1076, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5577, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'azb'): {'d_size': 0.2784, 'd_overlap_w': 0.0036, 'd_ttr': 1.1284, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.4717, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('hye', 'bos'): {'d_size': 0.2652, 'd_overlap_w': 0.0039, 'd_ttr': 0.5212, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.7}, ('hye', 'csl'): {'d_size': 0.2591, 'd_overlap_w': 0.0027, 'd_ttr': 19.421, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'eus'): {'d_size': 0.2426, 'd_overlap_w': 0.003, 'd_ttr': 0.7254, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5, 'i_inv': 0.4815, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hye', 'ben'): {'d_size': 0.2176, 'd_overlap_w': 0.0006, 'd_ttr': 0.0126, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5354, 'i_inv': 0.5625, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'bel'): {'d_size': 0.2111, 'd_overlap_w': 0.003, 'd_ttr': 0.7102, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.8}, ('hye', 'kaz'): {'d_size': 0.1553, 'd_overlap_w': 0.0027, 'd_ttr': 2.4217, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('hye', 'kat'): {'d_size': 0.6176, 'd_overlap_w': 0.0035, 'd_ttr': 0.3354, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4054, 'i_inv': 0.3673, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('hye', 'zsm'): {'d_size': 0.2444, 'd_overlap_w': 0.0052, 'd_ttr': 0.0115, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}}, 'kur': {('kur', 'arb'): {'d_size': 20.6452, 'd_overlap_w': 0.0027, 'd_ttr': 0.298, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('kur', 'heb'): {'d_size': 20.4242, 'd_overlap_w': 0.0006, 'd_ttr': 0.3766, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'rus'): {'d_size': 20.1001, 'd_overlap_w': 0.0005, 'd_ttr': 0.3759, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'kor'): {'d_size': 19.8284, 'd_overlap_w': 0.0003, 'd_ttr': 0.0117, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('kur', 'ita'): {'d_size': 19.7187, 'd_overlap_w': 0.001, 'd_ttr': 0.6508, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'jpn'): {'d_size': 19.6789, 'd_overlap_w': 0.0002, 'd_ttr': 19.2657, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('kur', 'spa'): {'d_size': 18.9014, 'd_overlap_w': 0.0011, 'd_ttr': 0.6485, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'fra'): {'d_size': 18.5425, 'd_overlap_w': 0.0011, 'd_ttr': 0.7193, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'nld'): {'d_size': 17.7193, 'd_overlap_w': 0.001, 'd_ttr': 0.5795, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'tur'): {'d_size': 17.5943, 'd_overlap_w': 0.0006, 'd_ttr': 0.1614, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kur', 'ron'): {'d_size': 17.4028, 'd_overlap_w': 0.0009, 'd_ttr': 0.5261, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'pol'): {'d_size': 16.9867, 'd_overlap_w': 0.0007, 'd_ttr': 0.278, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'bul'): {'d_size': 16.8204, 'd_overlap_w': 0.0007, 'd_ttr': 0.476, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'vie'): {'d_size': 16.5842, 'd_overlap_w': 0.0023, 'd_ttr': 0.8953, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kur', 'deu'): {'d_size': 16.1882, 'd_overlap_w': 0.0009, 'd_ttr': 0.4843, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'pes'): {'d_size': 14.5565, 'd_overlap_w': 0.0144, 'd_ttr': 0.679, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'hun'): {'d_size': 14.1953, 'd_overlap_w': 0.0005, 'd_ttr': 0.1008, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'srp'): {'d_size': 13.2001, 'd_overlap_w': 0.0007, 'd_ttr': 0.2271, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kur', 'ell'): {'d_size': 12.9522, 'd_overlap_w': 0.0008, 'd_ttr': 0.4473, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'hrv'): {'d_size': 11.7723, 'd_overlap_w': 0.0008, 'd_ttr': 0.2931, 'd_overlap_s': '-', 'i_syn': 0.91, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 1.0}, ('kur', 'ukr'): {'d_size': 10.4614, 'd_overlap_w': 0.0007, 'd_ttr': 0.2503, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'ces'): {'d_size': 9.9405, 'd_overlap_w': 0.0008, 'd_ttr': 0.223, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'tha'): {'d_size': 9.4556, 'd_overlap_w': 0.0003, 'd_ttr': 0.0386, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kur', 'ind'): {'d_size': 8.4279, 'd_overlap_w': 0.0016, 'd_ttr': 0.5983, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('kur', 'slk'): {'d_size': 5.9271, 'd_overlap_w': 0.0009, 'd_ttr': 0.105, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kur', 'swe'): {'d_size': 5.4621, 'd_overlap_w': 0.0014, 'd_ttr': 0.3016, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'por'): {'d_size': 4.9933, 'd_overlap_w': 0.0016, 'd_ttr': 0.3545, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'dan'): {'d_size': 4.3332, 'd_overlap_w': 0.0016, 'd_ttr': 0.3377, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'lit'): {'d_size': 4.0419, 'd_overlap_w': 0.001, 'd_ttr': 0.0131, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'mkd'): {'d_size': 2.4429, 'd_overlap_w': 0.0013, 'd_ttr': 0.0776, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kur', 'fin'): {'d_size': 2.3356, 'd_overlap_w': 0.0011, 'd_ttr': 0.1913, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kur', 'mya'): {'d_size': 2.0728, 'd_overlap_w': 0.0005, 'd_ttr': 0.1682, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kur', 'hye'): {'d_size': 2.0596, 'd_overlap_w': 0.0012, 'd_ttr': 0.013, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('kur', 'slv'): {'d_size': 1.9122, 'd_overlap_w': 0.0014, 'd_ttr': 0.0028, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kur', 'hin'): {'d_size': 1.8126, 'd_overlap_w': 0.0017, 'd_ttr': 0.4218, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kur', 'nob'): {'d_size': 1.5259, 'd_overlap_w': 0.0019, 'd_ttr': 0.1165, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kur', 'est'): {'d_size': 1.0354, 'd_overlap_w': 0.0013, 'd_ttr': 0.1641, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'glg'): {'d_size': 0.9659, 'd_overlap_w': 0.0019, 'd_ttr': 0.012, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'mar'): {'d_size': 0.9488, 'd_overlap_w': 0.0008, 'd_ttr': 0.0749, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kur', 'tam'): {'d_size': 0.6001, 'd_overlap_w': 0.0016, 'd_ttr': 0.2774, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kur', 'urd'): {'d_size': 0.5763, 'd_overlap_w': 0.0097, 'd_ttr': 0.1638, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kur', 'azb'): {'d_size': 0.5733, 'd_overlap_w': 0.0017, 'd_ttr': 0.6841, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.9}, ('kur', 'bos'): {'d_size': 0.5461, 'd_overlap_w': 0.0016, 'd_ttr': 0.2762, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kur', 'csl'): {'d_size': 0.5336, 'd_overlap_w': 0.0015, 'd_ttr': 14.3665, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kur', 'eus'): {'d_size': 0.4997, 'd_overlap_w': 0.0015, 'd_ttr': 0.4103, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'ben'): {'d_size': 0.4483, 'd_overlap_w': 0.0005, 'd_ttr': 0.0455, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('kur', 'bel'): {'d_size': 0.4348, 'd_overlap_w': 0.0013, 'd_ttr': 0.4002, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kur', 'kaz'): {'d_size': 0.3198, 'd_overlap_w': 0.0014, 'd_ttr': 1.5994, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kur', 'kat'): {'d_size': 1.2721, 'd_overlap_w': 0.0014, 'd_ttr': 0.1592, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('kur', 'zsm'): {'d_size': 0.5033, 'd_overlap_w': 0.002, 'd_ttr': 0.0437, 'd_overlap_s': '-', 'i_syn': 0.91, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}}, 'hin': {('hin', 'arb'): {'d_size': 11.3901, 'd_overlap_w': 0.0018, 'd_ttr': 0.0873, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('hin', 'heb'): {'d_size': 11.2682, 'd_overlap_w': 0.002, 'd_ttr': 0.0104, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.5629, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hin', 'rus'): {'d_size': 11.0894, 'd_overlap_w': 0.0023, 'd_ttr': 0.0107, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.2048, 'i_inv': 0.6015, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'kor'): {'d_size': 10.9395, 'd_overlap_w': 0.0014, 'd_ttr': 2.3832, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.5319, 'i_inv': 0.558, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'ita'): {'d_size': 10.879, 'd_overlap_w': 0.0051, 'd_ttr': 0.2013, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.5418, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'jpn'): {'d_size': 10.857, 'd_overlap_w': 0.0005, 'd_ttr': 206.5918, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.4717, 'i_inv': 0.498, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'spa'): {'d_size': 10.428, 'd_overlap_w': 0.0051, 'd_ttr': 0.1977, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4097, 'i_inv': 0.5922, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('hin', 'fra'): {'d_size': 10.23, 'd_overlap_w': 0.0058, 'd_ttr': 0.3211, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2677, 'i_inv': 0.4618, 'i_fam': 0.9375, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'nld'): {'d_size': 9.7759, 'd_overlap_w': 0.005, 'd_ttr': 0.1017, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5922, 'i_inv': 0.4861, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'tur'): {'d_size': 9.7069, 'd_overlap_w': 0.0025, 'd_ttr': 0.4992, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.2871, 'i_inv': 0.5418, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'ron'): {'d_size': 9.6012, 'd_overlap_w': 0.0043, 'd_ttr': 0.0468, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.2952, 'i_inv': 0.489, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'pol'): {'d_size': 9.3717, 'd_overlap_w': 0.003, 'd_ttr': 0.1215, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2048, 'i_inv': 0.5925, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'bul'): {'d_size': 9.2799, 'd_overlap_w': 0.0026, 'd_ttr': 0.0133, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.2048, 'i_inv': 0.5418, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'vie'): {'d_size': 9.1496, 'd_overlap_w': 0.0119, 'd_ttr': 0.7165, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.4788, 'i_inv': 0.5186, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hin', 'deu'): {'d_size': 8.9312, 'd_overlap_w': 0.004, 'd_ttr': 0.0176, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.3862, 'i_inv': 0.5037, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'pes'): {'d_size': 8.0309, 'd_overlap_w': 0.0041, 'd_ttr': 0.248, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.2871, 'i_inv': 0.489, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hin', 'hun'): {'d_size': 7.8316, 'd_overlap_w': 0.0023, 'd_ttr': 0.8966, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.2871, 'i_inv': 0.5039, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'srp'): {'d_size': 7.2826, 'd_overlap_w': 0.0024, 'd_ttr': 0.2431, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.8564, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'ell'): {'d_size': 7.1458, 'd_overlap_w': 0.0035, 'd_ttr': 0.003, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2871, 'i_inv': 0.6108, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'hrv'): {'d_size': 6.4949, 'd_overlap_w': 0.0037, 'd_ttr': 0.0951, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5922, 'i_inv': 0.5263, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'ukr'): {'d_size': 5.7716, 'd_overlap_w': 0.0026, 'd_ttr': 0.1811, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5922, 'i_inv': 0.5252, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'ces'): {'d_size': 5.4843, 'd_overlap_w': 0.0038, 'd_ttr': 0.2556, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5509, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'tha'): {'d_size': 5.2167, 'd_overlap_w': 0.0014, 'd_ttr': 5.8233, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4477, 'i_inv': 0.4526, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('hin', 'ind'): {'d_size': 4.6497, 'd_overlap_w': 0.0089, 'd_ttr': 0.1252, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3433, 'i_inv': 0.4816, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'slk'): {'d_size': 3.27, 'd_overlap_w': 0.0042, 'd_ttr': 0.8613, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'swe'): {'d_size': 3.0135, 'd_overlap_w': 0.0061, 'd_ttr': 0.0818, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5473, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'por'): {'d_size': 2.7548, 'd_overlap_w': 0.0065, 'd_ttr': 0.0237, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.477, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.5}, ('hin', 'dan'): {'d_size': 2.3907, 'd_overlap_w': 0.0066, 'd_ttr': 0.038, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'lit'): {'d_size': 2.23, 'd_overlap_w': 0.004, 'd_ttr': 2.3281, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2952, 'i_inv': 0.5129, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'mkd'): {'d_size': 1.3477, 'd_overlap_w': 0.0045, 'd_ttr': 1.1198, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.558, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'fin'): {'d_size': 1.2885, 'd_overlap_w': 0.0042, 'd_ttr': 9.6108, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3433, 'i_inv': 0.5129, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'mya'): {'d_size': 1.1436, 'd_overlap_w': 0.0028, 'd_ttr': 0.4659, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.4208, 'i_inv': 0.4421, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hin', 'hye'): {'d_size': 1.1363, 'd_overlap_w': 0.0039, 'd_ttr': 2.3325, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.4717, 'i_inv': 0.5081, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'slv'): {'d_size': 1.055, 'd_overlap_w': 0.0049, 'd_ttr': 2.8954, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5922, 'i_inv': 0.4816, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'nob'): {'d_size': 0.8418, 'd_overlap_w': 0.0073, 'd_ttr': 0.7723, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.4097, 'i_inv': 0.4505, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'est'): {'d_size': 0.5712, 'd_overlap_w': 0.0043, 'd_ttr': 9.0478, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'kur'): {'d_size': 0.5517, 'd_overlap_w': 0.0017, 'd_ttr': 3.432, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('hin', 'glg'): {'d_size': 0.5329, 'd_overlap_w': 0.0065, 'd_ttr': 2.3703, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.5219, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('hin', 'mar'): {'d_size': 0.5235, 'd_overlap_w': 0.0873, 'd_ttr': 1.1489, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.398, 'i_fam': 0.625, 'i_geo': 0.0, 'i_fea': 0.5}, ('hin', 'tam'): {'d_size': 0.3311, 'd_overlap_w': 0.0054, 'd_ttr': 0.1227, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.4661, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hin', 'urd'): {'d_size': 0.318, 'd_overlap_w': 0.0056, 'd_ttr': 0.4875, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5922, 'i_inv': 0.3583, 'i_fam': 0.125, 'i_geo': 0.1, 'i_fea': 0.4}, ('hin', 'azb'): {'d_size': 0.3163, 'd_overlap_w': 0.0046, 'd_ttr': 17.7398, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2871, 'i_inv': 0.6107, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('hin', 'bos'): {'d_size': 0.3013, 'd_overlap_w': 0.0052, 'd_ttr': 11.2349, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'csl'): {'d_size': 0.2944, 'd_overlap_w': 0.0036, 'd_ttr': 160.3955, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hin', 'eus'): {'d_size': 0.2757, 'd_overlap_w': 0.0039, 'd_ttr': 13.5408, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.2952, 'i_inv': 0.5327, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('hin', 'ben'): {'d_size': 0.2473, 'd_overlap_w': 0.0014, 'd_ttr': 1.5472, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.3613, 'i_inv': 0.3481, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.4}, ('hin', 'bel'): {'d_size': 0.2399, 'd_overlap_w': 0.0038, 'd_ttr': 13.3747, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'kaz'): {'d_size': 0.1765, 'd_overlap_w': 0.0037, 'd_ttr': 29.8132, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 1.0, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('hin', 'kat'): {'d_size': 0.7018, 'd_overlap_w': 0.0037, 'd_ttr': 8.9452, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.4477, 'i_inv': 0.5088, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'zsm'): {'d_size': 0.2777, 'd_overlap_w': 0.008, 'd_ttr': 1.578, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5922, 'i_inv': 0.5489, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}}, 'ben': {('ben', 'arb'): {'d_size': 46.0553, 'd_overlap_w': 0.0003, 'd_ttr': 0.1787, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('ben', 'heb'): {'d_size': 45.5623, 'd_overlap_w': 0.0003, 'd_ttr': 0.2589, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5936, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('ben', 'rus'): {'d_size': 44.8393, 'd_overlap_w': 0.0003, 'd_ttr': 0.2582, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.303, 'i_inv': 0.5962, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'kor'): {'d_size': 44.2332, 'd_overlap_w': 0.0002, 'd_ttr': 0.0179, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5, 'i_inv': 0.5256, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('ben', 'ita'): {'d_size': 43.9886, 'd_overlap_w': 0.0007, 'd_ttr': 0.5689, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.6162, 'i_inv': 0.5625, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.7}, ('ben', 'jpn'): {'d_size': 43.8998, 'd_overlap_w': 0.0001, 'd_ttr': 34.2373, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.4327, 'i_inv': 0.4724, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('ben', 'spa'): {'d_size': 42.1652, 'd_overlap_w': 0.0006, 'd_ttr': 0.5662, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4717, 'i_inv': 0.5849, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'fra'): {'d_size': 41.3646, 'd_overlap_w': 0.0008, 'd_ttr': 0.6511, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4407, 'i_inv': 0.4753, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.5}, ('ben', 'nld'): {'d_size': 39.5283, 'd_overlap_w': 0.0006, 'd_ttr': 0.485, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5193, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'tur'): {'d_size': 39.2493, 'd_overlap_w': 0.0003, 'd_ttr': 0.0573, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.3613, 'i_inv': 0.5667, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'ron'): {'d_size': 38.8221, 'd_overlap_w': 0.0006, 'd_ttr': 0.4235, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.3824, 'i_inv': 0.4997, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'pol'): {'d_size': 37.894, 'd_overlap_w': 0.0004, 'd_ttr': 0.1592, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.303, 'i_inv': 0.5472, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'bul'): {'d_size': 37.5229, 'd_overlap_w': 0.0004, 'd_ttr': 0.367, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.303, 'i_inv': 0.5383, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'vie'): {'d_size': 36.9961, 'd_overlap_w': 0.0018, 'd_ttr': 0.8678, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.4407, 'i_inv': 0.5242, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('ben', 'deu'): {'d_size': 36.1127, 'd_overlap_w': 0.0005, 'd_ttr': 0.3763, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.4407, 'i_inv': 0.5404, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'pes'): {'d_size': 32.4726, 'd_overlap_w': 0.0006, 'd_ttr': 0.6026, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.3613, 'i_inv': 0.5182, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'hun'): {'d_size': 31.6668, 'd_overlap_w': 0.0003, 'd_ttr': 0.0175, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.3613, 'i_inv': 0.5319, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'srp'): {'d_size': 29.4468, 'd_overlap_w': 0.0004, 'd_ttr': 0.112, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.8485, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('ben', 'ell'): {'d_size': 28.8937, 'd_overlap_w': 0.0005, 'd_ttr': 0.3351, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.3613, 'i_inv': 0.5893, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'hrv'): {'d_size': 26.2618, 'd_overlap_w': 0.0006, 'd_ttr': 0.1738, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.6162, 'i_inv': 0.5667, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.7}, ('ben', 'ukr'): {'d_size': 23.3373, 'd_overlap_w': 0.0004, 'd_ttr': 0.133, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5256, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'ces'): {'d_size': 22.1753, 'd_overlap_w': 0.0005, 'd_ttr': 0.1083, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.6162, 'i_inv': 0.5737, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'tha'): {'d_size': 21.0936, 'd_overlap_w': 0.0002, 'd_ttr': 0.2716, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.2804, 'i_inv': 0.4147, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'ind'): {'d_size': 18.801, 'd_overlap_w': 0.0013, 'd_ttr': 0.507, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.4054, 'i_inv': 0.5144, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'slk'): {'d_size': 13.2222, 'd_overlap_w': 0.0006, 'd_ttr': 0.0198, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'swe'): {'d_size': 12.1848, 'd_overlap_w': 0.0008, 'd_ttr': 0.1822, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.4906, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'por'): {'d_size': 11.139, 'd_overlap_w': 0.0008, 'd_ttr': 0.2359, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.6162, 'i_inv': 0.4991, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'dan'): {'d_size': 9.6666, 'd_overlap_w': 0.001, 'd_ttr': 0.2185, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.8}, ('ben', 'lit'): {'d_size': 9.0168, 'd_overlap_w': 0.0006, 'd_ttr': 0.0158, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3824, 'i_inv': 0.4886, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('ben', 'mkd'): {'d_size': 5.4496, 'd_overlap_w': 0.0007, 'd_ttr': 0.0068, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.6162, 'i_inv': 0.5978, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.7}, ('ben', 'fin'): {'d_size': 5.2102, 'd_overlap_w': 0.0005, 'd_ttr': 0.6844, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4054, 'i_inv': 0.5071, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'mya'): {'d_size': 4.624, 'd_overlap_w': 0.0008, 'd_ttr': 0.0626, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3743, 'i_inv': 0.4589, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('ben', 'hye'): {'d_size': 4.5945, 'd_overlap_w': 0.0006, 'd_ttr': 0.016, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5354, 'i_inv': 0.5625, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'slv'): {'d_size': 4.2656, 'd_overlap_w': 0.0007, 'd_ttr': 0.0416, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5332, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'hin'): {'d_size': 4.0435, 'd_overlap_w': 0.0014, 'd_ttr': 0.3073, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.3613, 'i_inv': 0.3481, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.4}, ('ben', 'nob'): {'d_size': 3.404, 'd_overlap_w': 0.0011, 'd_ttr': 0.0265, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.2048, 'i_inv': 0.4769, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('ben', 'est'): {'d_size': 2.3097, 'd_overlap_w': 0.0008, 'd_ttr': 0.6181, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('ben', 'kur'): {'d_size': 2.2308, 'd_overlap_w': 0.0005, 'd_ttr': 0.0736, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('ben', 'glg'): {'d_size': 2.1547, 'd_overlap_w': 0.0011, 'd_ttr': 0.0174, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.6162, 'i_inv': 0.5182, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'mar'): {'d_size': 2.1166, 'd_overlap_w': 0.0011, 'd_ttr': 0.0059, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.6162, 'i_inv': 0.3908, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'tam'): {'d_size': 1.3388, 'd_overlap_w': 0.0013, 'd_ttr': 0.1586, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4661, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'urd'): {'d_size': 1.2857, 'd_overlap_w': 0.0015, 'd_ttr': 0.0591, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.6162, 'i_inv': 0.3603, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'azb'): {'d_size': 1.279, 'd_overlap_w': 0.0012, 'd_ttr': 1.7496, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.3613, 'i_inv': 0.6532, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'bos'): {'d_size': 1.2183, 'd_overlap_w': 0.0011, 'd_ttr': 0.8826, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('ben', 'csl'): {'d_size': 1.1904, 'd_overlap_w': 0.001, 'd_ttr': 25.9063, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'eus'): {'d_size': 1.1146, 'd_overlap_w': 0.001, 'd_ttr': 1.1785, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.2163, 'i_inv': 0.5514, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'bel'): {'d_size': 0.9699, 'd_overlap_w': 0.001, 'd_ttr': 1.1567, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.9}, ('ben', 'kaz'): {'d_size': 0.7135, 'd_overlap_w': 0.0011, 'd_ttr': 3.5307, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 1.0, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('ben', 'kat'): {'d_size': 2.8378, 'd_overlap_w': 0.0007, 'd_ttr': 0.6062, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5032, 'i_inv': 0.5849, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'zsm'): {'d_size': 1.1228, 'd_overlap_w': 0.0018, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.6162, 'i_inv': 0.5867, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}}, 'tam': {('tam', 'arb'): {'d_size': 34.4009, 'd_overlap_w': 0.0013, 'd_ttr': 0.0017, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('tam', 'heb'): {'d_size': 34.0326, 'd_overlap_w': 0.0014, 'd_ttr': 0.0338, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.6293, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('tam', 'rus'): {'d_size': 33.4926, 'd_overlap_w': 0.0015, 'd_ttr': 0.0334, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.6162, 'i_inv': 0.611, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('tam', 'kor'): {'d_size': 33.0398, 'd_overlap_w': 0.0008, 'd_ttr': 0.7812, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.641, 'i_inv': 0.5366, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'ita'): {'d_size': 32.8572, 'd_overlap_w': 0.003, 'd_ttr': 0.3501, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.5747, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('tam', 'jpn'): {'d_size': 32.7908, 'd_overlap_w': 0.0004, 'd_ttr': 107.8516, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.5575, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'spa'): {'d_size': 31.4952, 'd_overlap_w': 0.0029, 'd_ttr': 0.3465, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5922, 'i_inv': 0.5, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'fra'): {'d_size': 30.8972, 'd_overlap_w': 0.0035, 'd_ttr': 0.4612, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5456, 'i_inv': 0.5776, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('tam', 'nld'): {'d_size': 29.5255, 'd_overlap_w': 0.0026, 'd_ttr': 0.2456, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.5729, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'tur'): {'d_size': 29.3172, 'd_overlap_w': 0.0015, 'd_ttr': 0.0696, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5922, 'i_inv': 0.5903, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'ron'): {'d_size': 28.9981, 'd_overlap_w': 0.0025, 'd_ttr': 0.1761, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.641, 'i_inv': 0.527, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'pol'): {'d_size': 28.3048, 'd_overlap_w': 0.0017, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.5003, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'bul'): {'d_size': 28.0276, 'd_overlap_w': 0.0019, 'd_ttr': 0.119, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.598, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('tam', 'vie'): {'d_size': 27.6342, 'd_overlap_w': 0.0078, 'd_ttr': 0.7855, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.5456, 'i_inv': 0.5792, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('tam', 'deu'): {'d_size': 26.9743, 'd_overlap_w': 0.0023, 'd_ttr': 0.1279, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5456, 'i_inv': 0.595, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'pes'): {'d_size': 24.2553, 'd_overlap_w': 0.0027, 'd_ttr': 0.3946, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.5456, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('tam', 'hun'): {'d_size': 23.6534, 'd_overlap_w': 0.0013, 'd_ttr': 0.1952, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.5403, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'srp'): {'d_size': 21.9952, 'd_overlap_w': 0.0017, 'd_ttr': 0.0112, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.8278, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('tam', 'ell'): {'d_size': 21.5821, 'd_overlap_w': 0.0024, 'd_ttr': 0.0902, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.5922, 'i_inv': 0.5656, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('tam', 'hrv'): {'d_size': 19.6162, 'd_overlap_w': 0.0022, 'd_ttr': 0.001, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.4784, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'ukr'): {'d_size': 17.4317, 'd_overlap_w': 0.002, 'd_ttr': 0.0031, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.5559, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'ces'): {'d_size': 16.5638, 'd_overlap_w': 0.0022, 'd_ttr': 0.0132, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.568, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'tha'): {'d_size': 15.7558, 'd_overlap_w': 0.0009, 'd_ttr': 2.3339, 'd_overlap_s': '-', 'i_syn': 0.91, 'i_phon': 0.5687, 'i_inv': 0.4792, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('tam', 'ind'): {'d_size': 14.0434, 'd_overlap_w': 0.0057, 'd_ttr': 0.272, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.5687, 'i_inv': 0.4627, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('tam', 'slk'): {'d_size': 9.8763, 'd_overlap_w': 0.0027, 'd_ttr': 0.1831, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('tam', 'swe'): {'d_size': 9.1014, 'd_overlap_w': 0.0038, 'd_ttr': 0.0023, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.5792, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'por'): {'d_size': 8.3202, 'd_overlap_w': 0.0041, 'd_ttr': 0.0211, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4865, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'dan'): {'d_size': 7.2204, 'd_overlap_w': 0.0042, 'd_ttr': 0.0132, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('tam', 'lit'): {'d_size': 6.7351, 'd_overlap_w': 0.0028, 'd_ttr': 0.7579, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.5698, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'mkd'): {'d_size': 4.0705, 'd_overlap_w': 0.0036, 'd_ttr': 0.2748, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.5366, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('tam', 'fin'): {'d_size': 3.8917, 'd_overlap_w': 0.003, 'd_ttr': 4.1472, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5687, 'i_inv': 0.5338, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'mya'): {'d_size': 3.4539, 'd_overlap_w': 0.0016, 'd_ttr': 0.0605, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5228, 'i_inv': 0.5, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('tam', 'hye'): {'d_size': 3.4319, 'd_overlap_w': 0.0034, 'd_ttr': 0.7598, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.5747, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('tam', 'slv'): {'d_size': 3.1862, 'd_overlap_w': 0.0035, 'd_ttr': 1.0015, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.5046, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'hin'): {'d_size': 3.0202, 'd_overlap_w': 0.0054, 'd_ttr': 0.0673, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.4661, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('tam', 'nob'): {'d_size': 2.5426, 'd_overlap_w': 0.0057, 'd_ttr': 0.1532, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.4785, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'est'): {'d_size': 1.7253, 'd_overlap_w': 0.0035, 'd_ttr': 3.8739, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('tam', 'kur'): {'d_size': 1.6663, 'd_overlap_w': 0.0016, 'd_ttr': 1.2378, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('tam', 'glg'): {'d_size': 1.6094, 'd_overlap_w': 0.005, 'd_ttr': 0.7757, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.0002, 'i_inv': 0.5079, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'mar'): {'d_size': 1.581, 'd_overlap_w': 0.0017, 'd_ttr': 0.2856, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.0002, 'i_inv': 0.3687, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('tam', 'urd'): {'d_size': 0.9603, 'd_overlap_w': 0.0061, 'd_ttr': 0.0664, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.4955, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('tam', 'azb'): {'d_size': 0.9553, 'd_overlap_w': 0.0039, 'd_ttr': 8.1785, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5922, 'i_inv': 0.668, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('tam', 'bos'): {'d_size': 0.91, 'd_overlap_w': 0.0041, 'd_ttr': 4.9413, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('tam', 'csl'): {'d_size': 0.8891, 'd_overlap_w': 0.0034, 'd_ttr': 83.1713, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('tam', 'eus'): {'d_size': 0.8326, 'd_overlap_w': 0.0036, 'd_ttr': 6.08, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5439, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('tam', 'ben'): {'d_size': 0.7469, 'd_overlap_w': 0.0013, 'd_ttr': 0.4379, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4661, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('tam', 'bel'): {'d_size': 0.7245, 'd_overlap_w': 0.0038, 'd_ttr': 5.9975, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('tam', 'kaz'): {'d_size': 0.5329, 'd_overlap_w': 0.004, 'd_ttr': 14.3206, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.879, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('tam', 'kat'): {'d_size': 2.1197, 'd_overlap_w': 0.0034, 'd_ttr': 3.8241, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.6529, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('tam', 'zsm'): {'d_size': 0.8387, 'd_overlap_w': 0.0068, 'd_ttr': 0.4501, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5439, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}}, 'kat': {('kat', 'arb'): {'d_size': 16.2291, 'd_overlap_w': 0.0016, 'd_ttr': 0.4562, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kat', 'heb'): {'d_size': 16.0554, 'd_overlap_w': 0.0019, 'd_ttr': 0.524, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.4445, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'rus'): {'d_size': 15.8007, 'd_overlap_w': 0.0021, 'd_ttr': 0.5234, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.4054, 'i_inv': 0.597, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'kor'): {'d_size': 15.5871, 'd_overlap_w': 0.001, 'd_ttr': 0.1315, 'd_overlap_s': '-', 'i_syn': 0.48, 'i_phon': 0.5582, 'i_inv': 0.6156, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('kat', 'ita'): {'d_size': 15.5009, 'd_overlap_w': 0.0037, 'd_ttr': 0.7428, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6363, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'jpn'): {'d_size': 15.4696, 'd_overlap_w': 0.0004, 'd_ttr': 8.1347, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.4054, 'i_inv': 0.5704, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('kat', 'spa'): {'d_size': 14.8583, 'd_overlap_w': 0.0037, 'd_ttr': 0.741, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5348, 'i_inv': 0.6035, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'fra'): {'d_size': 14.5762, 'd_overlap_w': 0.0041, 'd_ttr': 0.7946, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.427, 'i_inv': 0.5985, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'nld'): {'d_size': 13.9291, 'd_overlap_w': 0.0034, 'd_ttr': 0.6878, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5687, 'i_inv': 0.6121, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'tur'): {'d_size': 13.8308, 'd_overlap_w': 0.0019, 'd_ttr': 0.3276, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.4477, 'i_inv': 0.5608, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'ron'): {'d_size': 13.6803, 'd_overlap_w': 0.0031, 'd_ttr': 0.6459, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.4638, 'i_inv': 0.5114, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'pol'): {'d_size': 13.3532, 'd_overlap_w': 0.0023, 'd_ttr': 0.4384, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.4054, 'i_inv': 0.6118, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'bul'): {'d_size': 13.2225, 'd_overlap_w': 0.0026, 'd_ttr': 0.6059, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.4054, 'i_inv': 0.5733, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'vie'): {'d_size': 13.0368, 'd_overlap_w': 0.0075, 'd_ttr': 0.9245, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5829, 'i_inv': 0.566, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('kat', 'deu'): {'d_size': 12.7255, 'd_overlap_w': 0.0029, 'd_ttr': 0.6126, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.427, 'i_inv': 0.5466, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'pes'): {'d_size': 11.4428, 'd_overlap_w': 0.0031, 'd_ttr': 0.7642, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3433, 'i_inv': 0.439, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'hun'): {'d_size': 11.1589, 'd_overlap_w': 0.0018, 'd_ttr': 0.2623, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.4477, 'i_inv': 0.5879, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'srp'): {'d_size': 10.3766, 'd_overlap_w': 0.0022, 'd_ttr': 0.3917, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.8632, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kat', 'ell'): {'d_size': 10.1817, 'd_overlap_w': 0.003, 'd_ttr': 0.5825, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.4477, 'i_inv': 0.656, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'hrv'): {'d_size': 9.2542, 'd_overlap_w': 0.0028, 'd_ttr': 0.4518, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5687, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kat', 'ukr'): {'d_size': 8.2237, 'd_overlap_w': 0.0027, 'd_ttr': 0.4132, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.5505, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'ces'): {'d_size': 7.8142, 'd_overlap_w': 0.0029, 'd_ttr': 0.3878, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.5881, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'tha'): {'d_size': 7.433, 'd_overlap_w': 0.0011, 'd_ttr': 0.021, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5609, 'i_inv': 0.5984, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('kat', 'ind'): {'d_size': 6.6252, 'd_overlap_w': 0.0055, 'd_ttr': 0.7024, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.4816, 'i_inv': 0.4878, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('kat', 'slk'): {'d_size': 4.6593, 'd_overlap_w': 0.0033, 'd_ttr': 0.2672, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'swe'): {'d_size': 4.2937, 'd_overlap_w': 0.0047, 'd_ttr': 0.4593, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.6393, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('kat', 'por'): {'d_size': 3.9252, 'd_overlap_w': 0.0045, 'd_ttr': 0.5053, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5687, 'i_inv': 0.6034, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'dan'): {'d_size': 3.4064, 'd_overlap_w': 0.005, 'd_ttr': 0.4908, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('kat', 'lit'): {'d_size': 3.1774, 'd_overlap_w': 0.0031, 'd_ttr': 0.1348, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.3498, 'i_inv': 0.5404, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('kat', 'mkd'): {'d_size': 1.9203, 'd_overlap_w': 0.0037, 'd_ttr': 0.2345, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5687, 'i_inv': 0.5037, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'fin'): {'d_size': 1.836, 'd_overlap_w': 0.0032, 'd_ttr': 0.0007, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4816, 'i_inv': 0.5823, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('kat', 'mya'): {'d_size': 1.6294, 'd_overlap_w': 0.0015, 'd_ttr': 0.3345, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5335, 'i_inv': 0.5555, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'hye'): {'d_size': 1.619, 'd_overlap_w': 0.0035, 'd_ttr': 0.1345, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4054, 'i_inv': 0.3673, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('kat', 'slv'): {'d_size': 1.5031, 'd_overlap_w': 0.0038, 'd_ttr': 0.1044, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'hin'): {'d_size': 1.4248, 'd_overlap_w': 0.0037, 'd_ttr': 0.5616, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.4477, 'i_inv': 0.5088, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('kat', 'nob'): {'d_size': 1.1995, 'd_overlap_w': 0.0049, 'd_ttr': 0.2801, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5348, 'i_inv': 0.6278, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'est'): {'d_size': 0.8139, 'd_overlap_w': 0.0032, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kat', 'kur'): {'d_size': 0.7861, 'd_overlap_w': 0.0014, 'd_ttr': 0.0814, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('kat', 'glg'): {'d_size': 0.7593, 'd_overlap_w': 0.0042, 'd_ttr': 0.1322, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5687, 'i_inv': 0.5765, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'mar'): {'d_size': 0.7459, 'd_overlap_w': 0.001, 'd_ttr': 0.2312, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.6267, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('kat', 'tam'): {'d_size': 0.4718, 'd_overlap_w': 0.0034, 'd_ttr': 0.4378, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.6529, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('kat', 'urd'): {'d_size': 0.453, 'd_overlap_w': 0.0033, 'd_ttr': 0.33, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.5804, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('kat', 'azb'): {'d_size': 0.4507, 'd_overlap_w': 0.003, 'd_ttr': 0.0936, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.4477, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('kat', 'bos'): {'d_size': 0.4293, 'd_overlap_w': 0.0031, 'd_ttr': 0.0082, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kat', 'csl'): {'d_size': 0.4195, 'd_overlap_w': 0.0025, 'd_ttr': 5.8759, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('kat', 'eus'): {'d_size': 0.3928, 'd_overlap_w': 0.0027, 'd_ttr': 0.0298, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.4638, 'i_inv': 0.535, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('kat', 'ben'): {'d_size': 0.3524, 'd_overlap_w': 0.0007, 'd_ttr': 0.1916, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5032, 'i_inv': 0.5849, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('kat', 'bel'): {'d_size': 0.3418, 'd_overlap_w': 0.003, 'd_ttr': 0.0279, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('kat', 'kaz'): {'d_size': 0.2514, 'd_overlap_w': 0.0028, 'd_ttr': 0.3828, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kat', 'zsm'): {'d_size': 0.3957, 'd_overlap_w': 0.0045, 'd_ttr': 0.1889, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5687, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}}, 'csl': {('csl', 'arb'): {'d_size': 38.6901, 'd_overlap_w': 0.001, 'd_ttr': 0.8194, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('csl', 'heb'): {'d_size': 38.2759, 'd_overlap_w': 0.0011, 'd_ttr': 0.8452, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('csl', 'rus'): {'d_size': 37.6686, 'd_overlap_w': 0.0013, 'd_ttr': 0.845, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('csl', 'kor'): {'d_size': 37.1594, 'd_overlap_w': 0.0008, 'd_ttr': 0.6623, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('csl', 'ita'): {'d_size': 36.9539, 'd_overlap_w': 0.0032, 'd_ttr': 0.9209, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('csl', 'jpn'): {'d_size': 36.8793, 'd_overlap_w': 0.0004, 'd_ttr': 0.0156, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('csl', 'spa'): {'d_size': 35.4221, 'd_overlap_w': 0.0033, 'd_ttr': 0.9204, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('csl', 'fra'): {'d_size': 34.7495, 'd_overlap_w': 0.0036, 'd_ttr': 0.9376, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('csl', 'nld'): {'d_size': 33.2069, 'd_overlap_w': 0.0031, 'd_ttr': 0.9028, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'tur'): {'d_size': 32.9725, 'd_overlap_w': 0.0016, 'd_ttr': 0.7658, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'ron'): {'d_size': 32.6137, 'd_overlap_w': 0.0027, 'd_ttr': 0.8886, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'pol'): {'d_size': 31.8339, 'd_overlap_w': 0.0019, 'd_ttr': 0.8124, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'bul'): {'d_size': 31.5222, 'd_overlap_w': 0.0014, 'd_ttr': 0.8747, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'vie'): {'d_size': 31.0797, 'd_overlap_w': 0.0084, 'd_ttr': 0.9777, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('csl', 'deu'): {'d_size': 30.3375, 'd_overlap_w': 0.0025, 'd_ttr': 0.8771, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'pes'): {'d_size': 27.2795, 'd_overlap_w': 0.0021, 'd_ttr': 0.9279, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('csl', 'hun'): {'d_size': 26.6026, 'd_overlap_w': 0.0014, 'd_ttr': 0.7354, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'srp'): {'d_size': 24.7376, 'd_overlap_w': 0.0014, 'd_ttr': 0.7934, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('csl', 'ell'): {'d_size': 24.273, 'd_overlap_w': 0.002, 'd_ttr': 0.8665, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'hrv'): {'d_size': 22.062, 'd_overlap_w': 0.0022, 'd_ttr': 0.8177, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('csl', 'ukr'): {'d_size': 19.6052, 'd_overlap_w': 0.0015, 'd_ttr': 0.8022, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('csl', 'ces'): {'d_size': 18.629, 'd_overlap_w': 0.0024, 'd_ttr': 0.7918, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('csl', 'tha'): {'d_size': 17.7203, 'd_overlap_w': 0.0008, 'd_ttr': 0.5628, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('csl', 'ind'): {'d_size': 15.7944, 'd_overlap_w': 0.0056, 'd_ttr': 0.9077, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('csl', 'slk'): {'d_size': 11.1077, 'd_overlap_w': 0.0026, 'd_ttr': 0.7377, 'd_overlap_s': '-', 'i_syn': 0.0, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.0}, ('csl', 'swe'): {'d_size': 10.2362, 'd_overlap_w': 0.0043, 'd_ttr': 0.8206, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'por'): {'d_size': 9.3576, 'd_overlap_w': 0.0049, 'd_ttr': 0.8382, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('csl', 'dan'): {'d_size': 8.1207, 'd_overlap_w': 0.0043, 'd_ttr': 0.8328, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('csl', 'lit'): {'d_size': 7.5748, 'd_overlap_w': 0.0028, 'd_ttr': 0.6645, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'mkd'): {'d_size': 4.5781, 'd_overlap_w': 0.0027, 'd_ttr': 0.7214, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('csl', 'fin'): {'d_size': 4.3769, 'd_overlap_w': 0.0029, 'd_ttr': 0.4899, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'mya'): {'d_size': 3.8845, 'd_overlap_w': 0.0024, 'd_ttr': 0.7689, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('csl', 'hye'): {'d_size': 3.8598, 'd_overlap_w': 0.0027, 'd_ttr': 0.6643, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('csl', 'slv'): {'d_size': 3.5835, 'd_overlap_w': 0.0036, 'd_ttr': 0.6437, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('csl', 'hin'): {'d_size': 3.3968, 'd_overlap_w': 0.0036, 'd_ttr': 0.859, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('csl', 'nob'): {'d_size': 2.8596, 'd_overlap_w': 0.0051, 'd_ttr': 0.7439, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('csl', 'est'): {'d_size': 1.9404, 'd_overlap_w': 0.0036, 'd_ttr': 0.4994, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('csl', 'kur'): {'d_size': 1.8741, 'd_overlap_w': 0.0015, 'd_ttr': 0.6261, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('csl', 'glg'): {'d_size': 1.8101, 'd_overlap_w': 0.0056, 'd_ttr': 0.6628, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('csl', 'mar'): {'d_size': 1.7781, 'd_overlap_w': 0.0016, 'd_ttr': 0.7197, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('csl', 'tam'): {'d_size': 1.1247, 'd_overlap_w': 0.0034, 'd_ttr': 0.8121, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('csl', 'urd'): {'d_size': 1.0801, 'd_overlap_w': 0.0041, 'd_ttr': 0.7669, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('csl', 'azb'): {'d_size': 1.0744, 'd_overlap_w': 0.0039, 'd_ttr': 0.3827, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('csl', 'bos'): {'d_size': 1.0235, 'd_overlap_w': 0.0037, 'd_ttr': 0.4645, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('csl', 'eus'): {'d_size': 0.9364, 'd_overlap_w': 0.0038, 'd_ttr': 0.4323, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('csl', 'ben'): {'d_size': 0.8401, 'd_overlap_w': 0.001, 'd_ttr': 0.6985, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('csl', 'bel'): {'d_size': 0.8148, 'd_overlap_w': 0.0032, 'd_ttr': 0.4345, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('csl', 'kaz'): {'d_size': 0.5994, 'd_overlap_w': 0.0032, 'd_ttr': 0.278, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('csl', 'kat'): {'d_size': 2.384, 'd_overlap_w': 0.0025, 'd_ttr': 0.5012, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('csl', 'zsm'): {'d_size': 0.9433, 'd_overlap_w': 0.0069, 'd_ttr': 0.697, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}}}

#Add sub-word overlap to dictionary

In [ ]:
all_helpers_3code =  ['arb','swe', 'hun', 'deu', 'por', 'tur', 'bel', 'pes', 'kur', 'srp', 'kor', 'pol', 'kaz', 'ind', 'zsm', 'mya', 'slv', 'hin', 'ron', 'rus', 'mar', 'vie', 'hrv']
all_helpers_2code=['ar', 'sv', 'hu', 'de', 'pt', 'tr', 'be', 'fa', 'ku', 'sr', 'ko', 'pl', 'kk', 'id', 'ms', 'my', 'sl', 'hi', 'ro', 'ru', 'mr', 'vi', 'hr']

In [ ]:
!pip install --upgrade pip
!pip install pyonmttok
!pip install --upgrade OpenNMT-py==2.0.0rc1
!pip install subword-nmt

     |████████████████████████████████| 1.6MB 20.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 14.3 MB 224 kB/s 
     |████████████████████████████████| 219 kB 16.4 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-py3-none-any.whl size=19639 sha256=c68e3e141f6c9074e392795c1cd334a942f72eaf027c425330273d3493fa1b74
  Stored in directory: /root/.cache/pip/wheels/ad/ec/5c/2fb8013e264db5ff728101a261c406b942ab4740221c02d3e8
Successfully built configargparse
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


##Lithuanian

In [ ]:
lit_3code_help = ['rus', 'tur', 'pol', 'vie', 'deu', 'srp', 'swe', 'por', 'slv', 'bel']
lit_2code_help = ['ru', 'tr', 'pl', 'vi', 'de', 'sr', 'sv', 'pt', 'sl', 'be']  

child = []
for line in open('ted_talks/data_per_src/train/lt_train.txt', 'r').readlines():
    child.append(line)

###Learn BPEs

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/ru_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-ru_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+ru_c.vocab.txt baseline_childs/universal/bpe_apply/lt+ru_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/tr_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-tr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+tr_c.vocab.txt baseline_childs/universal/bpe_apply/lt+tr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/pl_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-pl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+pl_c.vocab.txt baseline_childs/universal/bpe_apply/lt+pl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/vi_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-vi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+vi_c.vocab.txt baseline_childs/universal/bpe_apply/lt+vi_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/de_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-de_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+de_c.vocab.txt baseline_childs/universal/bpe_apply/lt+de_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/sr_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-sr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+sr_c.vocab.txt baseline_childs/universal/bpe_apply/lt+sr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/sv_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-sv_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+sv_c.vocab.txt baseline_childs/universal/bpe_apply/lt+sv_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/pt_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-pt_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+pt_c.vocab.txt baseline_childs/universal/bpe_apply/lt+pt_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/sl_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-sl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+sl_c.vocab.txt baseline_childs/universal/bpe_apply/lt+sl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/be_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-be_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+be_c.vocab.txt baseline_childs/universal/bpe_apply/lt+be_h.vocab.txt

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/ar_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-ar_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+ar_c.vocab.txt baseline_childs/universal/bpe_apply/lt+ar_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/ko_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-ko_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+ko_c.vocab.txt baseline_childs/universal/bpe_apply/lt+ko_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/ro_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-ro_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+ro_c.vocab.txt baseline_childs/universal/bpe_apply/lt+ro_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/fa_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-fa_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+fa_c.vocab.txt baseline_childs/universal/bpe_apply/lt+fa_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/hu_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-hu_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+hu_c.vocab.txt baseline_childs/universal/bpe_apply/lt+hu_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/hr_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-hr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+hr_c.vocab.txt baseline_childs/universal/bpe_apply/lt+hr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/id_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-id_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+id_c.vocab.txt baseline_childs/universal/bpe_apply/lt+id_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/my_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-my_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+my_c.vocab.txt baseline_childs/universal/bpe_apply/lt+my_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/hi_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-hi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+hi_c.vocab.txt baseline_childs/universal/bpe_apply/lt+hi_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/ku_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-ku_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+ku_c.vocab.txt baseline_childs/universal/bpe_apply/lt+ku_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/kk_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-kk_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+kk_c.vocab.txt baseline_childs/universal/bpe_apply/lt+kk_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/mr_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-mr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+mr_c.vocab.txt baseline_childs/universal/bpe_apply/lt+mr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/ms_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-ms_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+ms_c.vocab.txt baseline_childs/universal/bpe_apply/lt+ms_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/lt_train.txt ted_talks/data_per_src/train/be_train.txt \
  -o baseline_childs/universal/bpe_learn/lt-be_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/lt+be_c.vocab.txt baseline_childs/universal/bpe_apply/lt+be_h.vocab.txt


###Calculate overlap

In [ ]:
for lan in all_helpers_2code:
  subunits_c = []
  for token in open('baseline_childs/universal/bpe_apply/lt+'+lan+'_c.vocab.txt', 'r').readlines():
    subunits_c.append(token.split()[0])

  subunits_h = []
  for token in open('baseline_childs/universal/bpe_apply/lt+'+lan+'_h.vocab.txt', 'r').readlines():
    subunits_h.append(token.split()[0])
  overlap = len(set(subunits_c).intersection(set(subunits_h))) / (len(subunits_c)+len(subunits_h))
  print(lan, round(overlap, 4))
  filtered_lrl_distances['lit'][('lit', all_helpers_3code[all_helpers_2code.index(lan)])]['d_overlap_s'] = round(overlap,4)

for pair, dct in filtered_lrl_distances['lit'].items():
  if pair[1] in all_helpers_3code:
    print(pair, dct)

ar 0.0751
sv 0.2391
hu 0.2123
de 0.2478
pt 0.2375
tr 0.2143
be 0.0407
fa 0.0835
ku 0.0347
sr 0.2397
ko 0.0741
pl 0.2235
kk 0.0345
id 0.2345
ms 0.1713
my 0.1129
sl 0.229
hi 0.0691
ro 0.2486
ru 0.0859
mr 0.0341
vi 0.2067
hr 0.2576
('lit', 'arb') {'d_size': 5.1077, 'd_overlap_w': 0.0039, 'd_ttr': 0.2373, 'd_overlap_s': 0.0751, 'i_syn': 0.66, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}
('lit', 'rus') {'d_size': 4.9729, 'd_overlap_w': 0.0066, 'd_ttr': 0.3171, 'd_overlap_s': 0.0859, 'i_syn': 0.55, 'i_phon': 0.2163, 'i_inv': 0.3825, 'i_fam': 0.5, 'i_geo': 0.2, 'i_fea': 0.5}
('lit', 'kor') {'d_size': 4.9057, 'd_overlap_w': 0.0042, 'd_ttr': 0.0001, 'd_overlap_s': 0.0741, 'i_syn': 0.68, 'i_phon': 0.5702, 'i_inv': 0.5552, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}
('lit', 'tur') {'d_size': 4.3529, 'd_overlap_w': 0.0135, 'd_ttr': 0.1052, 'd_overlap_s': 0.2143, 'i_syn': 0.56, 'i_phon': 0.2952, 'i_inv': 0.4959, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}
('lit', 'ron') {

In [ ]:
for k,v in filtered_lrl_distances.items():
  print(k)
  for k2,v in v.items():
    print(k2)
    print(v['d_overlap_s'])

lit
('lit', 'arb')
0.0751
('lit', 'heb')
-
('lit', 'rus')
0.0859
('lit', 'kor')
0.0741
('lit', 'ita')
-
('lit', 'jpn')
-
('lit', 'spa')
-
('lit', 'fra')
-
('lit', 'nld')
-
('lit', 'tur')
0.2143
('lit', 'ron')
0.2486
('lit', 'pol')
0.2235
('lit', 'bul')
-
('lit', 'vie')
0.2067
('lit', 'deu')
0.2478
('lit', 'pes')
0.0835
('lit', 'hun')
0.2123
('lit', 'srp')
0.2397
('lit', 'ell')
-
('lit', 'hrv')
0.2576
('lit', 'ukr')
-
('lit', 'ces')
-
('lit', 'tha')
-
('lit', 'ind')
0.2345
('lit', 'slk')
-
('lit', 'swe')
0.2391
('lit', 'por')
0.2375
('lit', 'dan')
-
('lit', 'mkd')
-
('lit', 'fin')
-
('lit', 'mya')
0.1129
('lit', 'hye')
-
('lit', 'slv')
0.229
('lit', 'hin')
0.0691
('lit', 'nob')
-
('lit', 'est')
-
('lit', 'kur')
0.0347
('lit', 'glg')
-
('lit', 'mar')
0.0341
('lit', 'tam')
-
('lit', 'urd')
-
('lit', 'azb')
-
('lit', 'bos')
-
('lit', 'csl')
-
('lit', 'eus')
-
('lit', 'ben')
-
('lit', 'bel')
0.0407
('lit', 'kaz')
0.0345
('lit', 'kat')
-
('lit', 'zsm')
0.1713
slv
('slv', 'arb')
-
('slv', 'he

##Armenian

In [ ]:
hye_3code_help = ['rus', 'tur', 'ron', 'vie', 'hun', 'hrv', 'ind', 'kur', 'bel', 'arb']
hye_2code_help = ['ru', 'tr', 'ro', 'vi', 'hu', 'hr', 'id', 'ku', 'be', 'ar']  

child = []
for line in open('baseline_childs/universal/bpe_apply/lt+be_c_train_bpe.txt', 'r').readlines():
    child.append(line)

###Learn BPEs

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/ru_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-ru_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+ru_c.vocab.txt baseline_childs/universal/bpe_apply/hy+ru_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/tr_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-tr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+tr_c.vocab.txt baseline_childs/universal/bpe_apply/hy+tr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/ro_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-ro_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+ro_c.vocab.txt baseline_childs/universal/bpe_apply/hy+ro_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/vi_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-vi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+vi_c.vocab.txt baseline_childs/universal/bpe_apply/hy+vi_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/hu_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-hu_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+hu_c.vocab.txt baseline_childs/universal/bpe_apply/hy+hu_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/hr_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-hr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+hr_c.vocab.txt baseline_childs/universal/bpe_apply/hy+hr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/id_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-id_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+id_c.vocab.txt baseline_childs/universal/bpe_apply/hy+id_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/ku_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-ku_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+ku_c.vocab.txt baseline_childs/universal/bpe_apply/hy+ku_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/be_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-be_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+be_c.vocab.txt baseline_childs/universal/bpe_apply/hy+be_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/ar_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-ar_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+ar_c.vocab.txt baseline_childs/universal/bpe_apply/hy+ar_h.vocab.txt


In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/ko_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-ko_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+ko_c.vocab.txt baseline_childs/universal/bpe_apply/hy+ko_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/pl_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-pl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+pl_c.vocab.txt baseline_childs/universal/bpe_apply/hy+pl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/de_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-de_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+de_c.vocab.txt baseline_childs/universal/bpe_apply/hy+de_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/fa_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-fa_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+fa_c.vocab.txt baseline_childs/universal/bpe_apply/hy+fa_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/sr_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-sr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+sr_c.vocab.txt baseline_childs/universal/bpe_apply/hy+sr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/sv_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-sv_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+sv_c.vocab.txt baseline_childs/universal/bpe_apply/hy+sv_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/pt_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-pt_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+pt_c.vocab.txt baseline_childs/universal/bpe_apply/hy+pt_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/my_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-my_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+my_c.vocab.txt baseline_childs/universal/bpe_apply/hy+my_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/sl_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-sl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/sl+sl_c.vocab.txt baseline_childs/universal/bpe_apply/hy+sl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/hi_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-hi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+hi_c.vocab.txt baseline_childs/universal/bpe_apply/hy+hi_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/mr_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-mr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+mr_c.vocab.txt baseline_childs/universal/bpe_apply/hy+mr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/ms_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-ms_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+ms_c.vocab.txt baseline_childs/universal/bpe_apply/hy+ms_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/kk_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-kk_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+kk_c.vocab.txt baseline_childs/universal/bpe_apply/hy+kk_h.vocab.txt

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/sl_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-sl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+sl_c.vocab.txt baseline_childs/universal/bpe_apply/hy+sl_h.vocab.txt

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/hy_train.txt ted_talks/data_per_src/train/pt_train.txt \
  -o baseline_childs/universal/bpe_learn/hy-pt_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/hy+pt_c.vocab.txt baseline_childs/universal/bpe_apply/hy+pt_h.vocab.txt


###Calculate overlap

In [ ]:
for lan in all_helpers_2code:
  subunits_c = []
  for token in open('baseline_childs/universal/bpe_apply/hy+'+lan+'_c.vocab.txt', 'r').readlines():
    subunits_c.append(token.split()[0])

  subunits_h = []
  for token in open('baseline_childs/universal/bpe_apply/hy+'+lan+'_h.vocab.txt', 'r').readlines():
    subunits_h.append(token.split()[0])
  overlap = len(set(subunits_c).intersection(set(subunits_h))) / (len(subunits_c)+len(subunits_h))
  print(lan, round(overlap, 4))
  filtered_lrl_distances['hye'][('hye', all_helpers_3code[all_helpers_2code.index(lan)])]['d_overlap_s'] = round(overlap,4)

for pair, dct in filtered_lrl_distances['hye'].items():
  if pair[1] in all_helpers_3code:
    print(pair, dct)

ar 0.0267
sv 0.0788
hu 0.0714
de 0.0885
pt 0.0742
tr 0.074
be 0.0277
fa 0.0258
ku 0.0207
sr 0.0615
ko 0.0262
pl 0.0716
kk 0.0267
id 0.081
ms 0.0579
my 0.0419
sl 0.0642
hi 0.0291
ro 0.0804
ru 0.0323
mr 0.021
vi 0.089
hr 0.0691
('hye', 'arb') {'d_size': 10.0239, 'd_overlap_w': 0.0022, 'd_ttr': 0.2376, 'd_overlap_s': 0.0267, 'i_syn': 0.71, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}
('hye', 'rus') {'d_size': 9.7593, 'd_overlap_w': 0.0031, 'd_ttr': 0.3173, 'd_overlap_s': 0.0323, 'i_syn': 0.58, 'i_phon': 0.4327, 'i_inv': 0.4693, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}
('hye', 'kor') {'d_size': 9.6273, 'd_overlap_w': 0.0015, 'd_ttr': 0.0, 'd_overlap_s': 0.0262, 'i_syn': 0.52, 'i_phon': 0.5, 'i_inv': 0.5429, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}
('hye', 'tur') {'d_size': 8.5426, 'd_overlap_w': 0.0031, 'd_ttr': 0.1055, 'd_overlap_s': 0.074, 'i_syn': 0.42, 'i_phon': 0.4717, 'i_inv': 0.4963, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}
('hye', 'ron') {'d_size

##Urdu

In [ ]:
urd_3code_help = ['arb', 'kor', 'vie', 'pes', 'hin', 'kur', 'mar', 'zsm', 'kaz', 'mya']
urd_2code_help = ['ar', 'ko', 'vi', 'fa', 'hi', 'ku', 'mr', 'ms', 'kk', 'my']  

child = []
for line in open('ted_talks/data_per_src/train/ur_train.txt', 'r').readlines():
    child.append(line)

###Learn BPEs

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/ar_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-ar_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+ar_c.vocab.txt baseline_childs/universal/bpe_apply/ur+ar_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/ko_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-ko_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+ko_c.vocab.txt baseline_childs/universal/bpe_apply/ur+ko_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/vi_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-vi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+vi_c.vocab.txt baseline_childs/universal/bpe_apply/ur+vi_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/fa_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-fa_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+fa_c.vocab.txt baseline_childs/universal/bpe_apply/ur+fa_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/hi_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-hi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+hi_c.vocab.txt baseline_childs/universal/bpe_apply/ur+hi_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/ku_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-ku_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+ku_c.vocab.txt baseline_childs/universal/bpe_apply/ur+ku_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/mr_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-mr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+mr_c.vocab.txt baseline_childs/universal/bpe_apply/ur+mr_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/ms_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-ms_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+ms_c.vocab.txt baseline_childs/universal/bpe_apply/ur+ms_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/kk_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-kk_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+kk_c.vocab.txt baseline_childs/universal/bpe_apply/ur+kk_h.vocab.txt


!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/my_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-my_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+my_c.vocab.txt baseline_childs/universal/bpe_apply/ur+my_h.vocab.txt

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/ru_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-ru_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+ru_c.vocab.txt baseline_childs/universal/bpe_apply/ur+ru_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/tr_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-tr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+tr_c.vocab.txt baseline_childs/universal/bpe_apply/ur+tr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/ro_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-ro_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+ro_c.vocab.txt baseline_childs/universal/bpe_apply/ur+ro_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/pl_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-pl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+pl_c.vocab.txt baseline_childs/universal/bpe_apply/ur+pl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/de_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-de_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+de_c.vocab.txt baseline_childs/universal/bpe_apply/ur+de_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/hu_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-hu_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+hu_c.vocab.txt baseline_childs/universal/bpe_apply/ur+hu_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/sr_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-sr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+sr_c.vocab.txt baseline_childs/universal/bpe_apply/ur+sr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/hr_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-hr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+hr_c.vocab.txt baseline_childs/universal/bpe_apply/ur+hr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/id_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-id_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+id_c.vocab.txt baseline_childs/universal/bpe_apply/ur+id_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/sv_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-sv_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+sv_c.vocab.txt baseline_childs/universal/bpe_apply/ur+sv_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/pt_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-pt_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+pt_c.vocab.txt baseline_childs/universal/bpe_apply/ur+pt_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/sl_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-sl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+sl_c.vocab.txt baseline_childs/universal/bpe_apply/ur+sl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/ur_train.txt ted_talks/data_per_src/train/be_train.txt \
  -o baseline_childs/universal/bpe_learn/ur-be_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/ur+be_c.vocab.txt baseline_childs/universal/bpe_apply/ur+be_h.vocab.txt

###Calculate overlap

In [ ]:
for lan in all_helpers_2code:
  subunits_c = []
  for token in open('baseline_childs/universal/bpe_apply/ur+'+lan+'_c.vocab.txt', 'r').readlines():
    subunits_c.append(token.split()[0])

  subunits_h = []
  for token in open('baseline_childs/universal/bpe_apply/ur+'+lan+'_h.vocab.txt', 'r').readlines():
    subunits_h.append(token.split()[0])
  overlap = len(set(subunits_c).intersection(set(subunits_h))) / (len(subunits_c)+len(subunits_h))
  print(lan, round(overlap, 4))
  filtered_lrl_distances['urd'][('urd', all_helpers_3code[all_helpers_2code.index(lan)])]['d_overlap_s'] = round(overlap,4)

for pair, dct in filtered_lrl_distances['urd'].items():
  if pair[1] in all_helpers_3code:
    print(pair, dct)

ar 0.1666
sv 0.0628
hu 0.0554
de 0.0633
pt 0.0608
tr 0.0589
be 0.0277
fa 0.2626
ku 0.1381
sr 0.0507
ko 0.0238
pl 0.0578
kk 0.0258
id 0.0623
ms 0.0578
my 0.0435
sl 0.0556
hi 0.0294
ro 0.0614
ru 0.0292
mr 0.0212
vi 0.0713
hr 0.0561
('urd', 'arb') {'d_size': 35.8225, 'd_overlap_w': 0.0059, 'd_ttr': 0.0563, 'd_overlap_s': 0.1666, 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}
('urd', 'rus') {'d_size': 34.8767, 'd_overlap_w': 0.0016, 'd_ttr': 0.1226, 'd_overlap_s': 0.0292, 'i_syn': 0.62, 'i_phon': 0.6162, 'i_inv': 0.5919, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.6}
('urd', 'kor') {'d_size': 34.4052, 'd_overlap_w': 0.0008, 'd_ttr': 0.2479, 'd_overlap_s': 0.0238, 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5935, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}
('urd', 'tur') {'d_size': 30.5287, 'd_overlap_w': 0.0017, 'd_ttr': 0.0, 'd_overlap_s': 0.0589, 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.5459, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}
('urd', 'ron')

##Bengali

In [ ]:
ben_3code_help = ['ind','kor', 'vie', 'deu', 'mya', 'hin', 'kur', 'mar', 'zsm', 'kaz']
ben_2code_help = ['id', 'ko', 'vi', 'de', 'my', 'hi', 'ku', 'mr', 'ms', 'kk']  

child = []
for line in open('ted_talks/data_per_src/train/bn_train.txt', 'r').readlines():
    child.append(line)

###Learn BPEs

In [ ]:
ben_2code_help = ['id', 'ko', 'vi', 'de', 'my', 'hi', 'ku', 'mr', 'ms', 'kk']  

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/id_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-id_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+id_c.vocab.txt baseline_childs/universal/bpe_apply/bn+id_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/ko_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-ko_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+ko_c.vocab.txt baseline_childs/universal/bpe_apply/bn+ko_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/vi_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-vi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+vi_c.vocab.txt baseline_childs/universal/bpe_apply/bn+vi_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/de_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-de_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+de_c.vocab.txt baseline_childs/universal/bpe_apply/bn+de_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/my_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-my_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+my_c.vocab.txt baseline_childs/universal/bpe_apply/bn+my_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/hi_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-hi_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+hi_c.vocab.txt baseline_childs/universal/bpe_apply/bn+hi_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/ku_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-ku_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+ku_c.vocab.txt baseline_childs/universal/bpe_apply/bn+ku_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/mr_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-mr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+mr_c.vocab.txt baseline_childs/universal/bpe_apply/bn+mr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/ms_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-ms_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+ms_c.vocab.txt baseline_childs/universal/bpe_apply/bn+ms_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/kk_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-kk_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+kk_c.vocab.txt baseline_childs/universal/bpe_apply/bn+kk_h.vocab.txt

In [ ]:
!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/ar_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-ar_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+ar_c.vocab.txt baseline_childs/universal/bpe_apply/bn+ar_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/ru_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-ru_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+ru_c.vocab.txt baseline_childs/universal/bpe_apply/bn+ru_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/ro_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-ro_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+ro_c.vocab.txt baseline_childs/universal/bpe_apply/bn+ro_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/tr_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-tr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+tr_c.vocab.txt baseline_childs/universal/bpe_apply/bn+tr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/pl_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-pl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+pl_c.vocab.txt baseline_childs/universal/bpe_apply/bn+pl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/fa_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-fa_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+fa_c.vocab.txt baseline_childs/universal/bpe_apply/bn+fa_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/hu_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-hu_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+hu_c.vocab.txt baseline_childs/universal/bpe_apply/bn+hu_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/sr_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-sr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+sr_c.vocab.txt baseline_childs/universal/bpe_apply/bn+sr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/hr_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-hr_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+hr_c.vocab.txt baseline_childs/universal/bpe_apply/bn+hr_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/sv_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-sv_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+sv_c.vocab.txt baseline_childs/universal/bpe_apply/bn+sv_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/pt_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-pt_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+pt_c.vocab.txt baseline_childs/universal/bpe_apply/bn+pt_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/sl_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-sl_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+sl_c.vocab.txt baseline_childs/universal/bpe_apply/bn+sl_h.vocab.txt

!subword-nmt learn-joint-bpe-and-vocab \
  -s 8000 \
  -i ted_talks/data_per_src/train/bn_train.txt ted_talks/data_per_src/train/be_train.txt \
  -o baseline_childs/universal/bpe_learn/bn-be_bpe.code \
  --write-vocabulary baseline_childs/universal/bpe_apply/bn+be_c.vocab.txt baseline_childs/universal/bpe_apply/bn+be_h.vocab.txt

###Calculate overlap

In [ ]:
for lan in all_helpers_2code:
  subunits_c = []
  for token in open('baseline_childs/universal/bpe_apply/bn+'+lan+'_c.vocab.txt', 'r').readlines():
    subunits_c.append(token.split()[0])

  subunits_h = []
  for token in open('baseline_childs/universal/bpe_apply/bn+'+lan+'_h.vocab.txt', 'r').readlines():
    subunits_h.append(token.split()[0])
  overlap = len(set(subunits_c).intersection(set(subunits_h))) / (len(subunits_c)+len(subunits_h))
  filtered_lrl_distances['ben'][('ben', all_helpers_3code[all_helpers_2code.index(lan)])]['d_overlap_s'] = round(overlap,4)


#Analyze heuristic ranking

In [ ]:
filtered_lrl_distances =  {'lit': {('lit', 'arb'): {'d_size': 5.1077, 'd_overlap_w': 0.0039, 'd_ttr': 0.2373, 'd_overlap_s': 0.0751, 'i_syn': 0.66, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('lit', 'heb'): {'d_size': 5.0531, 'd_overlap_w': 0.0042, 'd_ttr': 0.3177, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6169, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'rus'): {'d_size': 4.9729, 'd_overlap_w': 0.0066, 'd_ttr': 0.3171, 'd_overlap_s': 0.0859, 'i_syn': 0.55, 'i_phon': 0.2163, 'i_inv': 0.3825, 'i_fam': 0.5, 'i_geo': 0.2, 'i_fea': 0.5}, ('lit', 'kor'): {'d_size': 4.9057, 'd_overlap_w': 0.0042, 'd_ttr': 0.0001, 'd_overlap_s': 0.0741, 'i_syn': 0.68, 'i_phon': 0.5702, 'i_inv': 0.5552, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'ita'): {'d_size': 4.8785, 'd_overlap_w': 0.0245, 'd_ttr': 0.6111, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.641, 'i_inv': 0.4906, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'jpn'): {'d_size': 4.8687, 'd_overlap_w': 0.0013, 'd_ttr': 25.8726, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3824, 'i_inv': 0.5749, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'spa'): {'d_size': 4.6763, 'd_overlap_w': 0.0242, 'd_ttr': 0.6086, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.4278, 'i_inv': 0.5404, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'fra'): {'d_size': 4.5875, 'd_overlap_w': 0.0244, 'd_ttr': 0.6863, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.3918, 'i_inv': 0.4824, 'i_fam': 0.9375, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'nld'): {'d_size': 4.3839, 'd_overlap_w': 0.0214, 'd_ttr': 0.5334, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.641, 'i_inv': 0.4954, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'tur'): {'d_size': 4.3529, 'd_overlap_w': 0.0135, 'd_ttr': 0.1052, 'd_overlap_s': 0.2143, 'i_syn': 0.56, 'i_phon': 0.2952, 'i_inv': 0.4959, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'ron'): {'d_size': 4.3055, 'd_overlap_w': 0.0205, 'd_ttr': 0.4758, 'd_overlap_s': 0.2486, 'i_syn': 0.55, 'i_phon': 0.3217, 'i_inv': 0.4795, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.5}, ('lit', 'pol'): {'d_size': 4.2026, 'd_overlap_w': 0.0166, 'd_ttr': 0.2172, 'd_overlap_s': 0.2235, 'i_syn': 0.42, 'i_phon': 0.2163, 'i_inv': 0.4933, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('lit', 'bul'): {'d_size': 4.1615, 'd_overlap_w': 0.0063, 'd_ttr': 0.4223, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.2163, 'i_inv': 0.3638, 'i_fam': 0.7143, 'i_geo': 0.1, 'i_fea': 0.4}, ('lit', 'vie'): {'d_size': 4.103, 'd_overlap_w': 0.0358, 'd_ttr': 0.8821, 'd_overlap_s': 0.2067, 'i_syn': 0.75, 'i_phon': 0.4934, 'i_inv': 0.5394, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'deu'): {'d_size': 4.0051, 'd_overlap_w': 0.0178, 'd_ttr': 0.4311, 'd_overlap_s': 0.2478, 'i_syn': 0.56, 'i_phon': 0.3918, 'i_inv': 0.5022, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'pes'): {'d_size': 3.6014, 'd_overlap_w': 0.0079, 'd_ttr': 0.642, 'd_overlap_s': 0.0835, 'i_syn': 0.59, 'i_phon': 0.2952, 'i_inv': 0.4795, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.5}, ('lit', 'hun'): {'d_size': 3.512, 'd_overlap_w': 0.0116, 'd_ttr': 0.0525, 'd_overlap_s': 0.2123, 'i_syn': 0.52, 'i_phon': 0.2952, 'i_inv': 0.4287, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('lit', 'srp'): {'d_size': 3.2658, 'd_overlap_w': 0.0128, 'd_ttr': 0.1672, 'd_overlap_s': 0.2397, 'i_syn': 0.54, 'i_phon': 0.8391, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('lit', 'ell'): {'d_size': 3.2044, 'd_overlap_w': 0.009, 'd_ttr': 0.3918, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2952, 'i_inv': 0.5009, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'hrv'): {'d_size': 2.9125, 'd_overlap_w': 0.0196, 'd_ttr': 0.2323, 'd_overlap_s': 0.2576, 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.3666, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'ukr'): {'d_size': 2.5882, 'd_overlap_w': 0.0064, 'd_ttr': 0.1898, 'd_overlap_s': '-', 'i_syn': 0.44, 'i_phon': 0.641, 'i_inv': 0.4436, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.5}, ('lit', 'ces'): {'d_size': 2.4593, 'd_overlap_w': 0.0162, 'd_ttr': 0.1632, 'd_overlap_s': '-', 'i_syn': 0.29, 'i_phon': 0.641, 'i_inv': 0.4073, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('lit', 'tha'): {'d_size': 2.3394, 'd_overlap_w': 0.0043, 'd_ttr': 0.1234, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.4638, 'i_inv': 0.4887, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'ind'): {'d_size': 2.0851, 'd_overlap_w': 0.027, 'd_ttr': 0.5538, 'd_overlap_s': 0.2345, 'i_syn': 0.66, 'i_phon': 0.3498, 'i_inv': 0.4766, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('lit', 'slk'): {'d_size': 1.4664, 'd_overlap_w': 0.0161, 'd_ttr': 0.056, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.6}, ('lit', 'swe'): {'d_size': 1.3513, 'd_overlap_w': 0.0185, 'd_ttr': 0.2409, 'd_overlap_s': 0.2391, 'i_syn': 0.59, 'i_phon': 0.641, 'i_inv': 0.5211, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.6}, ('lit', 'por'): {'d_size': 1.2354, 'd_overlap_w': 0.0202, 'd_ttr': 0.2949, 'd_overlap_s': 0.2375, 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.454, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'dan'): {'d_size': 1.0721, 'd_overlap_w': 0.0196, 'd_ttr': 0.2777, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.8}, ('lit', 'mkd'): {'d_size': 0.6044, 'd_overlap_w': 0.0053, 'd_ttr': 0.0343, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.641, 'i_inv': 0.392, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'fin'): {'d_size': 0.5778, 'd_overlap_w': 0.0099, 'd_ttr': 0.3885, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3498, 'i_inv': 0.4657, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'mya'): {'d_size': 0.5128, 'd_overlap_w': 0.0086, 'd_ttr': 0.1115, 'd_overlap_s': 0.1129, 'i_syn': 0.74, 'i_phon': 0.4259, 'i_inv': 0.469, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('lit', 'hye'): {'d_size': 0.5096, 'd_overlap_w': 0.0045, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5, 'i_inv': 0.4906, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'slv'): {'d_size': 0.4731, 'd_overlap_w': 0.0149, 'd_ttr': 0.0048, 'd_overlap_s': 0.229, 'i_syn': 0.47, 'i_phon': 0.641, 'i_inv': 0.429, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'hin'): {'d_size': 0.4484, 'd_overlap_w': 0.004, 'd_ttr': 0.3649, 'd_overlap_s': 0.0691, 'i_syn': 0.58, 'i_phon': 0.2952, 'i_inv': 0.5129, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('lit', 'nob'): {'d_size': 0.3775, 'd_overlap_w': 0.013, 'd_ttr': 0.0656, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.4278, 'i_inv': 0.5526, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'est'): {'d_size': 0.2562, 'd_overlap_w': 0.0078, 'd_ttr': 0.3443, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('lit', 'kur'): {'d_size': 0.2474, 'd_overlap_w': 0.001, 'd_ttr': 0.0167, 'd_overlap_s': 0.0347, 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('lit', 'glg'): {'d_size': 0.239, 'd_overlap_w': 0.0124, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.641, 'i_inv': 0.5421, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'mar'): {'d_size': 0.2347, 'd_overlap_w': 0.0012, 'd_ttr': 0.0323, 'd_overlap_s': 0.0341, 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5408, 'i_fam': 0.75, 'i_geo': 0.3, 'i_fea': 0.6}, ('lit', 'tam'): {'d_size': 0.1485, 'd_overlap_w': 0.0028, 'd_ttr': 0.2166, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.5698, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('lit', 'urd'): {'d_size': 0.1426, 'd_overlap_w': 0.0035, 'd_ttr': 0.1074, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5551, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('lit', 'azb'): {'d_size': 0.1418, 'd_overlap_w': 0.0061, 'd_ttr': 1.1309, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.2952, 'i_inv': 0.5621, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'bos'): {'d_size': 0.1351, 'd_overlap_w': 0.0088, 'd_ttr': 0.5227, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('lit', 'csl'): {'d_size': 0.132, 'd_overlap_w': 0.0028, 'd_ttr': 19.4487, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('lit', 'eus'): {'d_size': 0.1236, 'd_overlap_w': 0.0058, 'd_ttr': 0.7273, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3217, 'i_inv': 0.5417, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('lit', 'ben'): {'d_size': 0.1109, 'd_overlap_w': 0.0006, 'd_ttr': 0.0125, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3824, 'i_inv': 0.4886, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('lit', 'bel'): {'d_size': 0.1076, 'd_overlap_w': 0.0026, 'd_ttr': 0.712, 'd_overlap_s': 0.0407, 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.8}, ('lit', 'kaz'): {'d_size': 0.0791, 'd_overlap_w': 0.0021, 'd_ttr': 2.4263, 'd_overlap_s': 0.0345, 'i_syn': 0.77, 'i_phon': 1.0, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('lit', 'kat'): {'d_size': 0.3147, 'd_overlap_w': 0.0031, 'd_ttr': 0.3364, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.3498, 'i_inv': 0.5404, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('lit', 'zsm'): {'d_size': 0.1245, 'd_overlap_w': 0.0084, 'd_ttr': 0.0114, 'd_overlap_s': 0.1713, 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.4766, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}}, 'slv': {('slv', 'arb'): {'d_size': 10.7968, 'd_overlap_w': 0.0031, 'd_ttr': 0.2709, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('slv', 'heb'): {'d_size': 10.6812, 'd_overlap_w': 0.0035, 'd_ttr': 0.3506, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.0002, 'i_inv': 0.572, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'rus'): {'d_size': 10.5117, 'd_overlap_w': 0.0049, 'd_ttr': 0.3499, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.6162, 'i_inv': 0.4254, 'i_fam': 0.4, 'i_geo': 0.2, 'i_fea': 0.5}, ('slv', 'kor'): {'d_size': 10.3696, 'd_overlap_w': 0.0031, 'd_ttr': 0.0034, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.641, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'ita'): {'d_size': 10.3123, 'd_overlap_w': 0.0205, 'd_ttr': 0.6335, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.0002, 'i_inv': 0.4312, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'jpn'): {'d_size': 10.2915, 'd_overlap_w': 0.0009, 'd_ttr': 22.0004, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.6162, 'i_inv': 0.5062, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('slv', 'spa'): {'d_size': 9.8848, 'd_overlap_w': 0.019, 'd_ttr': 0.6311, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5922, 'i_inv': 0.5118, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'fra'): {'d_size': 9.6971, 'd_overlap_w': 0.0199, 'd_ttr': 0.705, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.4688, 'i_fam': 0.9375, 'i_geo': 0.0, 'i_fea': 0.6}, ('slv', 'nld'): {'d_size': 9.2667, 'd_overlap_w': 0.0179, 'd_ttr': 0.5593, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.0002, 'i_inv': 0.4622, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'tur'): {'d_size': 9.2013, 'd_overlap_w': 0.0111, 'd_ttr': 0.1357, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.4627, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'ron'): {'d_size': 9.1011, 'd_overlap_w': 0.0186, 'd_ttr': 0.504, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.641, 'i_inv': 0.3931, 'i_fam': 0.8889, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'pol'): {'d_size': 8.8835, 'd_overlap_w': 0.0174, 'd_ttr': 0.2508, 'd_overlap_s': '-', 'i_syn': 0.4, 'i_phon': 0.6162, 'i_inv': 0.5281, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'bul'): {'d_size': 8.7965, 'd_overlap_w': 0.005, 'd_ttr': 0.4524, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.6162, 'i_inv': 0.4225, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'vie'): {'d_size': 8.673, 'd_overlap_w': 0.0335, 'd_ttr': 0.8896, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5456, 'i_inv': 0.5303, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'deu'): {'d_size': 8.4659, 'd_overlap_w': 0.0143, 'd_ttr': 0.461, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5456, 'i_inv': 0.4255, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'pes'): {'d_size': 7.6126, 'd_overlap_w': 0.0069, 'd_ttr': 0.6629, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.4914, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('slv', 'hun'): {'d_size': 7.4237, 'd_overlap_w': 0.009, 'd_ttr': 0.078, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.438, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'srp'): {'d_size': 6.9032, 'd_overlap_w': 0.0517, 'd_ttr': 0.2001, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.8278, 'i_inv': 0.669, 'i_fam': 0.1667, 'i_geo': 0.0, 'i_fea': 0.7}, ('slv', 'ell'): {'d_size': 6.7736, 'd_overlap_w': 0.0082, 'd_ttr': 0.4228, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5922, 'i_inv': 0.4416, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'hrv'): {'d_size': 6.1566, 'd_overlap_w': 0.0692, 'd_ttr': 0.266, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.3141, 'i_fam': 0.375, 'i_geo': 0.0, 'i_fea': 0.5}, ('slv', 'ukr'): {'d_size': 5.471, 'd_overlap_w': 0.0052, 'd_ttr': 0.2231, 'd_overlap_s': '-', 'i_syn': 0.35, 'i_phon': 0.0002, 'i_inv': 0.404, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'ces'): {'d_size': 5.1986, 'd_overlap_w': 0.0231, 'd_ttr': 0.196, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.4204, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'tha'): {'d_size': 4.945, 'd_overlap_w': 0.0032, 'd_ttr': 0.0694, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5687, 'i_inv': 0.4313, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'ind'): {'d_size': 4.4075, 'd_overlap_w': 0.029, 'd_ttr': 0.5789, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5687, 'i_inv': 0.4413, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('slv', 'slk'): {'d_size': 3.0997, 'd_overlap_w': 0.028, 'd_ttr': 0.082, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('slv', 'swe'): {'d_size': 2.8565, 'd_overlap_w': 0.0208, 'd_ttr': 0.2745, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.5107, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'por'): {'d_size': 2.6113, 'd_overlap_w': 0.0196, 'd_ttr': 0.3281, 'd_overlap_s': '-', 'i_syn': 0.44, 'i_phon': 0.0002, 'i_inv': 0.3356, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.4}, ('slv', 'dan'): {'d_size': 2.2661, 'd_overlap_w': 0.0236, 'd_ttr': 0.311, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('slv', 'lit'): {'d_size': 2.1138, 'd_overlap_w': 0.0149, 'd_ttr': 0.0042, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.641, 'i_inv': 0.429, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'mkd'): {'d_size': 1.2775, 'd_overlap_w': 0.0058, 'd_ttr': 0.0567, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.3753, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.4}, ('slv', 'fin'): {'d_size': 1.2214, 'd_overlap_w': 0.011, 'd_ttr': 0.268, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'mya'): {'d_size': 1.084, 'd_overlap_w': 0.0079, 'd_ttr': 0.1423, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5228, 'i_inv': 0.409, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'hye'): {'d_size': 1.0771, 'd_overlap_w': 0.0046, 'd_ttr': 0.0042, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'hin'): {'d_size': 0.9479, 'd_overlap_w': 0.0049, 'd_ttr': 0.3967, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5922, 'i_inv': 0.4816, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('slv', 'nob'): {'d_size': 0.798, 'd_overlap_w': 0.0189, 'd_ttr': 0.0928, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.5922, 'i_inv': 0.5251, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'est'): {'d_size': 0.5415, 'd_overlap_w': 0.011, 'd_ttr': 0.2338, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('slv', 'kur'): {'d_size': 0.523, 'd_overlap_w': 0.0014, 'd_ttr': 0.0031, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('slv', 'glg'): {'d_size': 0.5051, 'd_overlap_w': 0.0154, 'd_ttr': 0.0036, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.0002, 'i_inv': 0.4685, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.5}, ('slv', 'mar'): {'d_size': 0.4962, 'd_overlap_w': 0.0016, 'd_ttr': 0.0543, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.4477, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('slv', 'tam'): {'d_size': 0.3139, 'd_overlap_w': 0.0035, 'd_ttr': 0.2502, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.5046, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('slv', 'urd'): {'d_size': 0.3014, 'd_overlap_w': 0.004, 'd_ttr': 0.1379, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.5088, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('slv', 'azb'): {'d_size': 0.2998, 'd_overlap_w': 0.0102, 'd_ttr': 0.8634, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5922, 'i_inv': 0.557, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'bos'): {'d_size': 0.2856, 'd_overlap_w': 0.0606, 'd_ttr': 0.3731, 'd_overlap_s': '-', 'i_syn': 0.36, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.1667, 'i_geo': 0.0, 'i_fea': 0.6}, ('slv', 'csl'): {'d_size': 0.2791, 'd_overlap_w': 0.0036, 'd_ttr': 16.4676, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('slv', 'eus'): {'d_size': 0.2613, 'd_overlap_w': 0.0089, 'd_ttr': 0.5362, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.641, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'ben'): {'d_size': 0.2344, 'd_overlap_w': 0.0007, 'd_ttr': 0.0287, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5332, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('slv', 'bel'): {'d_size': 0.2274, 'd_overlap_w': 0.0038, 'd_ttr': 0.524, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.8}, ('slv', 'kaz'): {'d_size': 0.1673, 'd_overlap_w': 0.003, 'd_ttr': 1.9355, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('slv', 'kat'): {'d_size': 0.6653, 'd_overlap_w': 0.0038, 'd_ttr': 0.2277, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('slv', 'zsm'): {'d_size': 0.2632, 'd_overlap_w': 0.0135, 'd_ttr': 0.0272, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.4413, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}}, 'kaz': {('kaz', 'arb'): {'d_size': 64.5496, 'd_overlap_w': 0.0009, 'd_ttr': 0.6392, 'd_overlap_s': '-', 'i_syn': 0.93, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'heb'): {'d_size': 63.8586, 'd_overlap_w': 0.001, 'd_ttr': 0.6879, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.879, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'rus'): {'d_size': 62.8453, 'd_overlap_w': 0.0046, 'd_ttr': 0.6875, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 1.0, 'i_inv': 0.688, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'kor'): {'d_size': 61.9958, 'd_overlap_w': 0.0005, 'd_ttr': 0.3675, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.7699, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('kaz', 'ita'): {'d_size': 61.653, 'd_overlap_w': 0.002, 'd_ttr': 0.8366, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'jpn'): {'d_size': 61.5285, 'd_overlap_w': 0.0003, 'd_ttr': 1.9036, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 1.0, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'spa'): {'d_size': 59.0974, 'd_overlap_w': 0.002, 'd_ttr': 0.8354, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 1.0, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'fra'): {'d_size': 57.9753, 'd_overlap_w': 0.0023, 'd_ttr': 0.8704, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 1.0, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'nld'): {'d_size': 55.4016, 'd_overlap_w': 0.0018, 'd_ttr': 0.8002, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.879, 'i_inv': 0.6068, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'tur'): {'d_size': 55.0106, 'd_overlap_w': 0.0009, 'd_ttr': 0.5414, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6196, 'i_fam': 0.4286, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'ron'): {'d_size': 54.4118, 'd_overlap_w': 0.0016, 'd_ttr': 0.7721, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 1.0, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'pol'): {'d_size': 53.1109, 'd_overlap_w': 0.0012, 'd_ttr': 0.6261, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 1.0, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'bul'): {'d_size': 52.5909, 'd_overlap_w': 0.0047, 'd_ttr': 0.7449, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 1.0, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'vie'): {'d_size': 51.8526, 'd_overlap_w': 0.0049, 'd_ttr': 0.953, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.8136, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'deu'): {'d_size': 50.6144, 'd_overlap_w': 0.0015, 'd_ttr': 0.7495, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.8136, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'pes'): {'d_size': 45.5125, 'd_overlap_w': 0.0018, 'd_ttr': 0.8506, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 1.0, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'hun'): {'d_size': 44.3832, 'd_overlap_w': 0.001, 'd_ttr': 0.4881, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'srp'): {'d_size': 41.2716, 'd_overlap_w': 0.0023, 'd_ttr': 0.5912, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 1.0, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ell'): {'d_size': 40.4965, 'd_overlap_w': 0.0017, 'd_ttr': 0.7289, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 1.0, 'i_inv': 0.6929, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'hrv'): {'d_size': 36.8077, 'd_overlap_w': 0.0015, 'd_ttr': 0.636, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.879, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ukr'): {'d_size': 32.7088, 'd_overlap_w': 0.0044, 'd_ttr': 0.6074, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ces'): {'d_size': 31.0802, 'd_overlap_w': 0.0015, 'd_ttr': 0.5882, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.879, 'i_inv': 0.6784, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'tha'): {'d_size': 29.5641, 'd_overlap_w': 0.0005, 'd_ttr': 0.2225, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.805, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'ind'): {'d_size': 26.3509, 'd_overlap_w': 0.0037, 'd_ttr': 0.81, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.805, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('kaz', 'slk'): {'d_size': 18.5318, 'd_overlap_w': 0.0019, 'd_ttr': 0.4922, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'swe'): {'d_size': 17.0778, 'd_overlap_w': 0.0027, 'd_ttr': 0.6415, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.879, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'por'): {'d_size': 15.612, 'd_overlap_w': 0.0031, 'd_ttr': 0.6746, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.879, 'i_inv': 0.6328, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'dan'): {'d_size': 13.5484, 'd_overlap_w': 0.0031, 'd_ttr': 0.6643, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'lit'): {'d_size': 12.6376, 'd_overlap_w': 0.0021, 'd_ttr': 0.3709, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 1.0, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'mkd'): {'d_size': 7.6379, 'd_overlap_w': 0.0069, 'd_ttr': 0.4643, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'fin'): {'d_size': 7.3024, 'd_overlap_w': 0.0021, 'd_ttr': 0.1335, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.805, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'mya'): {'d_size': 6.4809, 'd_overlap_w': 0.0013, 'd_ttr': 0.5469, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.8211, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'hye'): {'d_size': 6.4396, 'd_overlap_w': 0.0027, 'd_ttr': 0.3706, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'slv'): {'d_size': 5.9786, 'd_overlap_w': 0.003, 'd_ttr': 0.3385, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'hin'): {'d_size': 5.6672, 'd_overlap_w': 0.0037, 'd_ttr': 0.7144, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 1.0, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'nob'): {'d_size': 4.7709, 'd_overlap_w': 0.0041, 'd_ttr': 0.5029, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 1.0, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'est'): {'d_size': 3.2373, 'd_overlap_w': 0.0032, 'd_ttr': 0.1441, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'kur'): {'d_size': 3.1266, 'd_overlap_w': 0.0014, 'd_ttr': 0.3118, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('kaz', 'glg'): {'d_size': 3.0199, 'd_overlap_w': 0.0044, 'd_ttr': 0.3683, 'd_overlap_s': '-', 'i_syn': 0.92, 'i_phon': 0.879, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}, ('kaz', 'mar'): {'d_size': 2.9665, 'd_overlap_w': 0.0013, 'd_ttr': 0.4614, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.879, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'tam'): {'d_size': 1.8764, 'd_overlap_w': 0.004, 'd_ttr': 0.6257, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.879, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'urd'): {'d_size': 1.8019, 'd_overlap_w': 0.0044, 'd_ttr': 0.5433, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'azb'): {'d_size': 1.7926, 'd_overlap_w': 0.0035, 'd_ttr': 0.0373, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 1.0, 'i_inv': 0.669, 'i_fam': 0.3333, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'bos'): {'d_size': 1.7076, 'd_overlap_w': 0.0035, 'd_ttr': 0.1065, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'csl'): {'d_size': 1.6684, 'd_overlap_w': 0.0032, 'd_ttr': 1.2437, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'eus'): {'d_size': 1.5623, 'd_overlap_w': 0.0035, 'd_ttr': 0.076, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('kaz', 'ben'): {'d_size': 1.4016, 'd_overlap_w': 0.0011, 'd_ttr': 0.426, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 1.0, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'bel'): {'d_size': 1.3594, 'd_overlap_w': 0.0073, 'd_ttr': 0.0779, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('kaz', 'kat'): {'d_size': 3.9774, 'd_overlap_w': 0.0028, 'd_ttr': 0.1461, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('kaz', 'zsm'): {'d_size': 1.5737, 'd_overlap_w': 0.0059, 'd_ttr': 0.4235, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}}, 'mkd': {('mkd', 'arb'): {'d_size': 8.4512, 'd_overlap_w': 0.0027, 'd_ttr': 0.1373, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'heb'): {'d_size': 8.3607, 'd_overlap_w': 0.0031, 'd_ttr': 0.2158, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.5654, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'rus'): {'d_size': 8.2281, 'd_overlap_w': 0.0199, 'd_ttr': 0.2151, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.4147, 'i_fam': 0.4, 'i_geo': 0.2, 'i_fea': 0.5}, ('mkd', 'kor'): {'d_size': 8.1168, 'd_overlap_w': 0.0019, 'd_ttr': 0.0557, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.641, 'i_inv': 0.5861, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mkd', 'ita'): {'d_size': 8.072, 'd_overlap_w': 0.0078, 'd_ttr': 0.536, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.4473, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'jpn'): {'d_size': 8.0557, 'd_overlap_w': 0.0007, 'd_ttr': 41.8524, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.6162, 'i_inv': 0.5839, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('mkd', 'spa'): {'d_size': 7.7374, 'd_overlap_w': 0.0077, 'd_ttr': 0.5331, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.479, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'fra'): {'d_size': 7.5904, 'd_overlap_w': 0.0085, 'd_ttr': 0.6233, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.5006, 'i_fam': 0.9375, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'nld'): {'d_size': 7.2535, 'd_overlap_w': 0.0074, 'd_ttr': 0.4476, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.5145, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'tur'): {'d_size': 7.2023, 'd_overlap_w': 0.0041, 'd_ttr': 0.0292, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.5922, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('mkd', 'ron'): {'d_size': 7.1239, 'd_overlap_w': 0.0067, 'd_ttr': 0.3835, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.4086, 'i_fam': 0.8889, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'pol'): {'d_size': 6.9536, 'd_overlap_w': 0.0049, 'd_ttr': 0.1189, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.6162, 'i_inv': 0.3697, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'bul'): {'d_size': 6.8855, 'd_overlap_w': 0.0659, 'd_ttr': 0.3252, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.6162, 'i_inv': 0.4367, 'i_fam': 0.1429, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'vie'): {'d_size': 6.7888, 'd_overlap_w': 0.0148, 'd_ttr': 0.8564, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.5611, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mkd', 'deu'): {'d_size': 6.6267, 'd_overlap_w': 0.0064, 'd_ttr': 0.3348, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5456, 'i_inv': 0.5026, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'pes'): {'d_size': 5.9588, 'd_overlap_w': 0.0051, 'd_ttr': 0.5717, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.5922, 'i_inv': 0.4086, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('mkd', 'hun'): {'d_size': 5.8109, 'd_overlap_w': 0.0036, 'd_ttr': 0.0029, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5922, 'i_inv': 0.4278, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('mkd', 'srp'): {'d_size': 5.4035, 'd_overlap_w': 0.0326, 'd_ttr': 0.0754, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.8278, 'i_inv': 0.6737, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.8}, ('mkd', 'ell'): {'d_size': 5.302, 'd_overlap_w': 0.0058, 'd_ttr': 0.2926, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.4852, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.6}, ('mkd', 'hrv'): {'d_size': 4.8191, 'd_overlap_w': 0.0059, 'd_ttr': 0.1327, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.2983, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.4}, ('mkd', 'ukr'): {'d_size': 4.2824, 'd_overlap_w': 0.0181, 'd_ttr': 0.0945, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.3926, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.5}, ('mkd', 'ces'): {'d_size': 4.0692, 'd_overlap_w': 0.0056, 'd_ttr': 0.0721, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.0002, 'i_inv': 0.315, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'tha'): {'d_size': 3.8707, 'd_overlap_w': 0.002, 'd_ttr': 0.4334, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5687, 'i_inv': 0.5531, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mkd', 'ind'): {'d_size': 3.45, 'd_overlap_w': 0.0106, 'd_ttr': 0.4707, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5687, 'i_inv': 0.3444, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('mkd', 'slk'): {'d_size': 2.4263, 'd_overlap_w': 0.0056, 'd_ttr': 0.004, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.7}, ('mkd', 'swe'): {'d_size': 2.2359, 'd_overlap_w': 0.0077, 'd_ttr': 0.1407, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.5797, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'por'): {'d_size': 2.044, 'd_overlap_w': 0.0083, 'd_ttr': 0.193, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.4296, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'dan'): {'d_size': 1.7738, 'd_overlap_w': 0.0088, 'd_ttr': 0.1759, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'lit'): {'d_size': 1.6546, 'd_overlap_w': 0.0053, 'd_ttr': 0.0516, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.641, 'i_inv': 0.392, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.5}, ('mkd', 'fin'): {'d_size': 0.9561, 'd_overlap_w': 0.0048, 'd_ttr': 0.9843, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'mya'): {'d_size': 0.8485, 'd_overlap_w': 0.0034, 'd_ttr': 0.0333, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.5228, 'i_inv': 0.5106, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mkd', 'hye'): {'d_size': 0.8431, 'd_overlap_w': 0.0043, 'd_ttr': 0.0519, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.3926, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'slv'): {'d_size': 0.7828, 'd_overlap_w': 0.0058, 'd_ttr': 0.0977, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.3753, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.4}, ('mkd', 'hin'): {'d_size': 0.742, 'd_overlap_w': 0.0045, 'd_ttr': 0.2643, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.558, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('mkd', 'nob'): {'d_size': 0.6246, 'd_overlap_w': 0.0082, 'd_ttr': 0.0076, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5922, 'i_inv': 0.589, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'est'): {'d_size': 0.4238, 'd_overlap_w': 0.0043, 'd_ttr': 0.8974, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'kur'): {'d_size': 0.4094, 'd_overlap_w': 0.0013, 'd_ttr': 0.149, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('mkd', 'glg'): {'d_size': 0.3954, 'd_overlap_w': 0.0062, 'd_ttr': 0.0547, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.4828, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'mar'): {'d_size': 0.3884, 'd_overlap_w': 0.0012, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.5448, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mkd', 'tam'): {'d_size': 0.2457, 'd_overlap_w': 0.0036, 'd_ttr': 0.1183, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.5366, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mkd', 'urd'): {'d_size': 0.2359, 'd_overlap_w': 0.0033, 'd_ttr': 0.0306, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.5758, 'i_fam': 0.8571, 'i_geo': 0.2, 'i_fea': 0.6}, ('mkd', 'azb'): {'d_size': 0.2347, 'd_overlap_w': 0.004, 'd_ttr': 2.3478, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5922, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('mkd', 'bos'): {'d_size': 0.2236, 'd_overlap_w': 0.0046, 'd_ttr': 1.2419, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.8}, ('mkd', 'csl'): {'d_size': 0.2184, 'd_overlap_w': 0.0027, 'd_ttr': 31.8007, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('mkd', 'eus'): {'d_size': 0.2045, 'd_overlap_w': 0.0035, 'd_ttr': 1.6224, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.641, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'ben'): {'d_size': 0.1835, 'd_overlap_w': 0.0007, 'd_ttr': 0.0081, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.6162, 'i_inv': 0.5978, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.7}, ('mkd', 'bel'): {'d_size': 0.178, 'd_overlap_w': 0.0071, 'd_ttr': 1.5945, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.8}, ('mkd', 'kaz'): {'d_size': 0.1309, 'd_overlap_w': 0.0069, 'd_ttr': 4.5743, 'd_overlap_s': '-', 'i_syn': 0.84, 'i_phon': 0.879, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mkd', 'kat'): {'d_size': 0.5207, 'd_overlap_w': 0.0037, 'd_ttr': 0.8817, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5687, 'i_inv': 0.5037, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mkd', 'zsm'): {'d_size': 0.206, 'd_overlap_w': 0.0055, 'd_ttr': 0.0093, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.3108, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.4}}, 'urd': {('urd', 'arb'): {'d_size': 35.8225, 'd_overlap_w': 0.0059, 'd_ttr': 0.0563, 'd_overlap_s': 0.1666, 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('urd', 'heb'): {'d_size': 35.439, 'd_overlap_w': 0.0015, 'd_ttr': 0.1232, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.5698, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'rus'): {'d_size': 34.8767, 'd_overlap_w': 0.0016, 'd_ttr': 0.1226, 'd_overlap_s': 0.0292, 'i_syn': 0.62, 'i_phon': 0.6162, 'i_inv': 0.5919, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'kor'): {'d_size': 34.4052, 'd_overlap_w': 0.0008, 'd_ttr': 0.2479, 'd_overlap_s': 0.0238, 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5935, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'ita'): {'d_size': 34.215, 'd_overlap_w': 0.0037, 'd_ttr': 0.4561, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.5577, 'i_fam': 0.9, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'jpn'): {'d_size': 34.1459, 'd_overlap_w': 0.0004, 'd_ttr': 64.8429, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.6162, 'i_inv': 0.4859, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'spa'): {'d_size': 32.7967, 'd_overlap_w': 0.0035, 'd_ttr': 0.4529, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5922, 'i_inv': 0.5985, 'i_fam': 0.9231, 'i_geo': 0.3, 'i_fea': 0.7}, ('urd', 'fra'): {'d_size': 32.174, 'd_overlap_w': 0.0043, 'd_ttr': 0.5548, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.5456, 'i_inv': 0.5194, 'i_fam': 0.9375, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'nld'): {'d_size': 30.7457, 'd_overlap_w': 0.0035, 'd_ttr': 0.3587, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.4971, 'i_fam': 0.9, 'i_geo': 0.3, 'i_fea': 0.5}, ('urd', 'tur'): {'d_size': 30.5287, 'd_overlap_w': 0.0017, 'd_ttr': 0.0, 'd_overlap_s': 0.0589, 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.5459, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'ron'): {'d_size': 30.1964, 'd_overlap_w': 0.0029, 'd_ttr': 0.2901, 'd_overlap_s': 0.0614, 'i_syn': 0.67, 'i_phon': 0.641, 'i_inv': 0.5126, 'i_fam': 0.8889, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'pol'): {'d_size': 29.4745, 'd_overlap_w': 0.0022, 'd_ttr': 0.0424, 'd_overlap_s': 0.0578, 'i_syn': 0.53, 'i_phon': 0.6162, 'i_inv': 0.5772, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'bul'): {'d_size': 29.1859, 'd_overlap_w': 0.0021, 'd_ttr': 0.2296, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.5331, 'i_fam': 0.8571, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'vie'): {'d_size': 28.7761, 'd_overlap_w': 0.0098, 'd_ttr': 0.8274, 'd_overlap_s': 0.0713, 'i_syn': 0.75, 'i_phon': 0.5456, 'i_inv': 0.5018, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'deu'): {'d_size': 28.089, 'd_overlap_w': 0.0028, 'd_ttr': 0.2394, 'd_overlap_s': 0.0633, 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.5018, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'pes'): {'d_size': 25.2577, 'd_overlap_w': 0.0308, 'd_ttr': 0.4962, 'd_overlap_s': 0.2626, 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5309, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'hun'): {'d_size': 24.6309, 'd_overlap_w': 0.0015, 'd_ttr': 0.0214, 'd_overlap_s': 0.0554, 'i_syn': 0.56, 'i_phon': 0.5922, 'i_inv': 0.5612, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'srp'): {'d_size': 22.9041, 'd_overlap_w': 0.0018, 'd_ttr': 0.0146, 'd_overlap_s': 0.0507, 'i_syn': 0.72, 'i_phon': 0.8278, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.9}, ('urd', 'ell'): {'d_size': 22.474, 'd_overlap_w': 0.0026, 'd_ttr': 0.1968, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.566, 'i_fam': 0.8333, 'i_geo': 0.2, 'i_fea': 0.7}, ('urd', 'hrv'): {'d_size': 20.4268, 'd_overlap_w': 0.0026, 'd_ttr': 0.0527, 'd_overlap_s': 0.0561, 'i_syn': 0.86, 'i_phon': 0.0002, 'i_inv': 0.562, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'ukr'): {'d_size': 18.1521, 'd_overlap_w': 0.002, 'd_ttr': 0.0258, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.5006, 'i_fam': 0.8333, 'i_geo': 0.2, 'i_fea': 0.5}, ('urd', 'ces'): {'d_size': 17.2483, 'd_overlap_w': 0.0025, 'd_ttr': 0.0129, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.569, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'tha'): {'d_size': 16.4069, 'd_overlap_w': 0.0009, 'd_ttr': 1.0197, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.4862, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'ind'): {'d_size': 14.6237, 'd_overlap_w': 0.007, 'd_ttr': 0.3838, 'd_overlap_s': 0.0623, 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.5088, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'slk'): {'d_size': 10.2844, 'd_overlap_w': 0.0031, 'd_ttr': 0.0183, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.7}, ('urd', 'swe'): {'d_size': 9.4775, 'd_overlap_w': 0.0049, 'd_ttr': 0.0589, 'd_overlap_s': 0.0628, 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.5187, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'por'): {'d_size': 8.664, 'd_overlap_w': 0.0053, 'd_ttr': 0.1027, 'd_overlap_s': 0.0608, 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.4932, 'i_fam': 0.9333, 'i_geo': 0.3, 'i_fea': 0.5}, ('urd', 'dan'): {'d_size': 7.5188, 'd_overlap_w': 0.0053, 'd_ttr': 0.0879, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('urd', 'lit'): {'d_size': 7.0134, 'd_overlap_w': 0.0035, 'd_ttr': 0.2375, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.5551, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'mkd'): {'d_size': 4.2387, 'd_overlap_w': 0.0033, 'd_ttr': 0.0449, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.5758, 'i_fam': 0.8571, 'i_geo': 0.2, 'i_fea': 0.6}, ('urd', 'fin'): {'d_size': 4.0525, 'd_overlap_w': 0.0029, 'd_ttr': 2.0004, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5687, 'i_inv': 0.5551, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'mya'): {'d_size': 3.5966, 'd_overlap_w': 0.0027, 'd_ttr': 0.0001, 'd_overlap_s': 0.0435, 'i_syn': 0.73, 'i_phon': 0.5228, 'i_inv': 0.4889, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'hye'): {'d_size': 3.5737, 'd_overlap_w': 0.0035, 'd_ttr': 0.2383, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5577, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'slv'): {'d_size': 3.3179, 'd_overlap_w': 0.004, 'd_ttr': 0.3491, 'd_overlap_s': 0.0556, 'i_syn': 0.58, 'i_phon': 0.0002, 'i_inv': 0.5088, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('urd', 'hin'): {'d_size': 3.1451, 'd_overlap_w': 0.0056, 'd_ttr': 0.169, 'd_overlap_s': 0.0294, 'i_syn': 0.46, 'i_phon': 0.5922, 'i_inv': 0.3583, 'i_fam': 0.125, 'i_geo': 0.1, 'i_fea': 0.4}, ('urd', 'nob'): {'d_size': 2.6476, 'd_overlap_w': 0.0062, 'd_ttr': 0.0113, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.5197, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'est'): {'d_size': 1.7966, 'd_overlap_w': 0.0039, 'd_ttr': 1.8498, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('urd', 'kur'): {'d_size': 1.7352, 'd_overlap_w': 0.0097, 'd_ttr': 0.462, 'd_overlap_s': 0.1381, 'i_syn': 0.71, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('urd', 'glg'): {'d_size': 1.6759, 'd_overlap_w': 0.0063, 'd_ttr': 0.2454, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.5662, 'i_fam': 0.9231, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'mar'): {'d_size': 1.6463, 'd_overlap_w': 0.0016, 'd_ttr': 0.0484, 'd_overlap_s': 0.0212, 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.4445, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('urd', 'tam'): {'d_size': 1.0413, 'd_overlap_w': 0.0061, 'd_ttr': 0.042, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.4955, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('urd', 'azb'): {'d_size': 0.9948, 'd_overlap_w': 0.0049, 'd_ttr': 4.2807, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5996, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'bos'): {'d_size': 0.9476, 'd_overlap_w': 0.0049, 'd_ttr': 2.4416, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('urd', 'csl'): {'d_size': 0.9259, 'd_overlap_w': 0.0041, 'd_ttr': 49.6521, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('urd', 'eus'): {'d_size': 0.867, 'd_overlap_w': 0.0041, 'd_ttr': 3.0824, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.641, 'i_inv': 0.5996, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('urd', 'ben'): {'d_size': 0.7778, 'd_overlap_w': 0.0015, 'd_ttr': 0.1032, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.6162, 'i_inv': 0.3603, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('urd', 'bel'): {'d_size': 0.7544, 'd_overlap_w': 0.0042, 'd_ttr': 3.0357, 'd_overlap_s': 0.0277, 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('urd', 'kaz'): {'d_size': 0.555, 'd_overlap_w': 0.0044, 'd_ttr': 7.8626, 'd_overlap_s': 0.0258, 'i_syn': 0.71, 'i_phon': 0.879, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('urd', 'kat'): {'d_size': 2.2073, 'd_overlap_w': 0.0033, 'd_ttr': 1.8225, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.5804, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('urd', 'zsm'): {'d_size': 0.8733, 'd_overlap_w': 0.0092, 'd_ttr': 0.1079, 'd_overlap_s': 0.0578, 'i_syn': 0.86, 'i_phon': 0.0002, 'i_inv': 0.5464, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}}, 'mar': {('mar', 'arb'): {'d_size': 21.7592, 'd_overlap_w': 0.0005, 'd_ttr': 0.1404, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mar', 'heb'): {'d_size': 21.5263, 'd_overlap_w': 0.0006, 'd_ttr': 0.2191, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.64, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'rus'): {'d_size': 21.1848, 'd_overlap_w': 0.0006, 'd_ttr': 0.2184, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.6018, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'kor'): {'d_size': 20.8984, 'd_overlap_w': 0.0004, 'd_ttr': 0.0519, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.641, 'i_inv': 0.4829, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'ita'): {'d_size': 20.7828, 'd_overlap_w': 0.0014, 'd_ttr': 0.5386, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.5248, 'i_fam': 0.9, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'jpn'): {'d_size': 20.7409, 'd_overlap_w': 0.0002, 'd_ttr': 41.2163, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.5272, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'spa'): {'d_size': 19.9213, 'd_overlap_w': 0.0013, 'd_ttr': 0.5357, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.5499, 'i_fam': 0.9231, 'i_geo': 0.4, 'i_fea': 0.7}, ('mar', 'fra'): {'d_size': 19.5431, 'd_overlap_w': 0.0016, 'd_ttr': 0.6255, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5456, 'i_inv': 0.5174, 'i_fam': 0.9375, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'nld'): {'d_size': 18.6755, 'd_overlap_w': 0.0013, 'd_ttr': 0.4506, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.5625, 'i_fam': 0.9, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'tur'): {'d_size': 18.5437, 'd_overlap_w': 0.0007, 'd_ttr': 0.0311, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.5638, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mar', 'ron'): {'d_size': 18.3419, 'd_overlap_w': 0.0012, 'd_ttr': 0.3866, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.641, 'i_inv': 0.4744, 'i_fam': 0.8889, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'pol'): {'d_size': 17.9034, 'd_overlap_w': 0.0008, 'd_ttr': 0.1219, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5464, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'bul'): {'d_size': 17.728, 'd_overlap_w': 0.0007, 'd_ttr': 0.3284, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.6162, 'i_inv': 0.5533, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'vie'): {'d_size': 17.4792, 'd_overlap_w': 0.0032, 'd_ttr': 0.8573, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.5456, 'i_inv': 0.5522, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'deu'): {'d_size': 17.0618, 'd_overlap_w': 0.0011, 'd_ttr': 0.338, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5456, 'i_inv': 0.5522, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'pes'): {'d_size': 15.342, 'd_overlap_w': 0.0012, 'd_ttr': 0.5741, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.5342, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.7}, ('mar', 'hun'): {'d_size': 14.9613, 'd_overlap_w': 0.0006, 'd_ttr': 0.0036, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.5922, 'i_inv': 0.4901, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'srp'): {'d_size': 13.9124, 'd_overlap_w': 0.0007, 'd_ttr': 0.078, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.8278, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'ell'): {'d_size': 13.6511, 'd_overlap_w': 0.0009, 'd_ttr': 0.2959, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5752, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.7}, ('mar', 'hrv'): {'d_size': 12.4076, 'd_overlap_w': 0.001, 'd_ttr': 0.1358, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5295, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'ukr'): {'d_size': 11.0259, 'd_overlap_w': 0.0007, 'd_ttr': 0.0973, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.5248, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'ces'): {'d_size': 10.4769, 'd_overlap_w': 0.001, 'd_ttr': 0.0747, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.5957, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'tha'): {'d_size': 9.9659, 'd_overlap_w': 0.0004, 'd_ttr': 0.4191, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.5687, 'i_inv': 0.3767, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('mar', 'ind'): {'d_size': 8.8827, 'd_overlap_w': 0.0024, 'd_ttr': 0.4735, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.5687, 'i_inv': 0.47, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('mar', 'slk'): {'d_size': 6.247, 'd_overlap_w': 0.0012, 'd_ttr': 0.0048, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.7}, ('mar', 'swe'): {'d_size': 5.7568, 'd_overlap_w': 0.0017, 'd_ttr': 0.1439, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.5352, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'por'): {'d_size': 5.2627, 'd_overlap_w': 0.0018, 'd_ttr': 0.1962, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.0002, 'i_inv': 0.4726, 'i_fam': 0.9333, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'dan'): {'d_size': 4.5671, 'd_overlap_w': 0.0019, 'd_ttr': 0.1792, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'lit'): {'d_size': 4.2601, 'd_overlap_w': 0.0012, 'd_ttr': 0.048, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.5408, 'i_fam': 0.75, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'mkd'): {'d_size': 2.5747, 'd_overlap_w': 0.0012, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.5448, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'fin'): {'d_size': 2.4616, 'd_overlap_w': 0.0012, 'd_ttr': 0.9583, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5687, 'i_inv': 0.4824, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('mar', 'mya'): {'d_size': 2.1847, 'd_overlap_w': 0.0013, 'd_ttr': 0.0353, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5228, 'i_inv': 0.4475, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('mar', 'hye'): {'d_size': 2.1707, 'd_overlap_w': 0.0011, 'd_ttr': 0.0483, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5448, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('mar', 'slv'): {'d_size': 2.0153, 'd_overlap_w': 0.0016, 'd_ttr': 0.0924, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.4477, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.5}, ('mar', 'hin'): {'d_size': 1.9104, 'd_overlap_w': 0.0873, 'd_ttr': 0.2676, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.398, 'i_fam': 0.625, 'i_geo': 0.0, 'i_fea': 0.5}, ('mar', 'nob'): {'d_size': 1.6082, 'd_overlap_w': 0.0024, 'd_ttr': 0.0087, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.4265, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('mar', 'est'): {'d_size': 1.0913, 'd_overlap_w': 0.0015, 'd_ttr': 0.8732, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'kur'): {'d_size': 1.054, 'd_overlap_w': 0.0008, 'd_ttr': 0.142, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('mar', 'glg'): {'d_size': 1.018, 'd_overlap_w': 0.0023, 'd_ttr': 0.051, 'd_overlap_s': '-', 'i_syn': 0.85, 'i_phon': 0.0002, 'i_inv': 0.5148, 'i_fam': 0.9231, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'tam'): {'d_size': 0.6325, 'd_overlap_w': 0.0017, 'd_ttr': 0.1213, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.0002, 'i_inv': 0.3687, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('mar', 'urd'): {'d_size': 0.6074, 'd_overlap_w': 0.0016, 'd_ttr': 0.0325, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.4445, 'i_fam': 0.5714, 'i_geo': 0.1, 'i_fea': 0.5}, ('mar', 'azb'): {'d_size': 0.6043, 'd_overlap_w': 0.0018, 'd_ttr': 2.2968, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.6604, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'bos'): {'d_size': 0.5756, 'd_overlap_w': 0.0017, 'd_ttr': 1.211, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'csl'): {'d_size': 0.5624, 'd_overlap_w': 0.0016, 'd_ttr': 31.3079, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'eus'): {'d_size': 0.5266, 'd_overlap_w': 0.0016, 'd_ttr': 1.5844, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.641, 'i_inv': 0.5519, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('mar', 'ben'): {'d_size': 0.4725, 'd_overlap_w': 0.0011, 'd_ttr': 0.0069, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.6162, 'i_inv': 0.3908, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('mar', 'bel'): {'d_size': 0.4582, 'd_overlap_w': 0.0011, 'd_ttr': 1.557, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('mar', 'kaz'): {'d_size': 0.3371, 'd_overlap_w': 0.0013, 'd_ttr': 4.486, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.879, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('mar', 'kat'): {'d_size': 1.3408, 'd_overlap_w': 0.001, 'd_ttr': 0.8578, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.6267, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('mar', 'zsm'): {'d_size': 0.5305, 'd_overlap_w': 0.0026, 'd_ttr': 0.0079, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5324, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}}, 'zsm': {('zsm', 'arb'): {'d_size': 41.0174, 'd_overlap_w': 0.0019, 'd_ttr': 0.1813, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}, ('zsm', 'heb'): {'d_size': 40.5784, 'd_overlap_w': 0.0022, 'd_ttr': 0.2617, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.0002, 'i_inv': 0.5245, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('zsm', 'rus'): {'d_size': 39.9345, 'd_overlap_w': 0.0028, 'd_ttr': 0.261, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.6162, 'i_inv': 0.4783, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('zsm', 'kor'): {'d_size': 39.3946, 'd_overlap_w': 0.0018, 'd_ttr': 0.0162, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.641, 'i_inv': 0.5715, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'ita'): {'d_size': 39.1768, 'd_overlap_w': 0.0116, 'd_ttr': 0.571, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4312, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'jpn'): {'d_size': 39.0977, 'd_overlap_w': 0.0005, 'd_ttr': 33.8011, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.6162, 'i_inv': 0.57, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'spa'): {'d_size': 37.5529, 'd_overlap_w': 0.0115, 'd_ttr': 0.5683, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('zsm', 'fra'): {'d_size': 36.8398, 'd_overlap_w': 0.0128, 'd_ttr': 0.6528, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5456, 'i_inv': 0.4891, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'nld'): {'d_size': 35.2044, 'd_overlap_w': 0.0106, 'd_ttr': 0.4874, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.0002, 'i_inv': 0.4829, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'tur'): {'d_size': 34.9559, 'd_overlap_w': 0.0068, 'd_ttr': 0.0593, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.484, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'ron'): {'d_size': 34.5755, 'd_overlap_w': 0.0103, 'd_ttr': 0.426, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.4445, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'pol'): {'d_size': 33.7489, 'd_overlap_w': 0.0068, 'd_ttr': 0.1618, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.6162, 'i_inv': 0.4094, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'bul'): {'d_size': 33.4184, 'd_overlap_w': 0.0032, 'd_ttr': 0.3696, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.6162, 'i_inv': 0.4697, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'vie'): {'d_size': 32.9492, 'd_overlap_w': 0.0312, 'd_ttr': 0.8685, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5456, 'i_inv': 0.5303, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'deu'): {'d_size': 32.1625, 'd_overlap_w': 0.0081, 'd_ttr': 0.3789, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5456, 'i_inv': 0.4481, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'pes'): {'d_size': 28.9205, 'd_overlap_w': 0.0048, 'd_ttr': 0.6045, 'd_overlap_s': '-', 'i_syn': 0.88, 'i_phon': 0.5922, 'i_inv': 0.4685, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('zsm', 'hun'): {'d_size': 28.2029, 'd_overlap_w': 0.0048, 'd_ttr': 0.0188, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.4617, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'srp'): {'d_size': 26.2257, 'd_overlap_w': 0.0058, 'd_ttr': 0.1144, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.8278, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('zsm', 'ell'): {'d_size': 25.7331, 'd_overlap_w': 0.0049, 'd_ttr': 0.3378, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5922, 'i_inv': 0.5419, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('zsm', 'hrv'): {'d_size': 23.3891, 'd_overlap_w': 0.0092, 'd_ttr': 0.1765, 'd_overlap_s': '-', 'i_syn': 0.0, 'i_phon': 0.0002, 'i_inv': 0.4366, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.4}, ('zsm', 'ukr'): {'d_size': 20.7845, 'd_overlap_w': 0.0034, 'd_ttr': 0.1355, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4312, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'ces'): {'d_size': 19.7496, 'd_overlap_w': 0.0082, 'd_ttr': 0.1107, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.3923, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('zsm', 'tha'): {'d_size': 18.7862, 'd_overlap_w': 0.0019, 'd_ttr': 0.263, 'd_overlap_s': '-', 'i_syn': 0.76, 'i_phon': 0.5687, 'i_inv': 0.5199, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'ind'): {'d_size': 16.7444, 'd_overlap_w': 0.121, 'd_ttr': 0.5093, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5687, 'i_inv': 0.3578, 'i_fam': 0.1111, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'slk'): {'d_size': 11.7759, 'd_overlap_w': 0.0089, 'd_ttr': 0.0211, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('zsm', 'swe'): {'d_size': 10.8519, 'd_overlap_w': 0.0163, 'd_ttr': 0.1849, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.5303, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'por'): {'d_size': 9.9205, 'd_overlap_w': 0.016, 'd_ttr': 0.2386, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4397, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.6}, ('zsm', 'dan'): {'d_size': 8.6092, 'd_overlap_w': 0.018, 'd_ttr': 0.2213, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}, ('zsm', 'lit'): {'d_size': 8.0305, 'd_overlap_w': 0.0084, 'd_ttr': 0.0143, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.641, 'i_inv': 0.4766, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('zsm', 'mkd'): {'d_size': 4.8534, 'd_overlap_w': 0.0055, 'd_ttr': 0.0077, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.3108, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.4}, ('zsm', 'fin'): {'d_size': 4.6402, 'd_overlap_w': 0.0081, 'd_ttr': 0.668, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'mya'): {'d_size': 4.1182, 'd_overlap_w': 0.0074, 'd_ttr': 0.0646, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5228, 'i_inv': 0.4772, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'hye'): {'d_size': 4.092, 'd_overlap_w': 0.0052, 'd_ttr': 0.0144, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('zsm', 'slv'): {'d_size': 3.799, 'd_overlap_w': 0.0135, 'd_ttr': 0.039, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.0002, 'i_inv': 0.4413, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'hin'): {'d_size': 3.6011, 'd_overlap_w': 0.008, 'd_ttr': 0.31, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5922, 'i_inv': 0.5489, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'nob'): {'d_size': 3.0316, 'd_overlap_w': 0.0204, 'd_ttr': 0.028, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.5783, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}, ('zsm', 'est'): {'d_size': 2.0571, 'd_overlap_w': 0.0108, 'd_ttr': 0.6029, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}, ('zsm', 'kur'): {'d_size': 1.9868, 'd_overlap_w': 0.002, 'd_ttr': 0.0699, 'd_overlap_s': '-', 'i_syn': 0.91, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 1.0}, ('zsm', 'glg'): {'d_size': 1.919, 'd_overlap_w': 0.0182, 'd_ttr': 0.0158, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.4914, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.6}, ('zsm', 'mar'): {'d_size': 1.8851, 'd_overlap_w': 0.0026, 'd_ttr': 0.0067, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5324, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('zsm', 'tam'): {'d_size': 1.1923, 'd_overlap_w': 0.0068, 'd_ttr': 0.1612, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.5439, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('zsm', 'urd'): {'d_size': 1.145, 'd_overlap_w': 0.0092, 'd_ttr': 0.0612, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.0002, 'i_inv': 0.5464, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('zsm', 'azb'): {'d_size': 1.1391, 'd_overlap_w': 0.013, 'd_ttr': 1.7162, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.5922, 'i_inv': 0.5354, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('zsm', 'bos'): {'d_size': 1.0851, 'd_overlap_w': 0.0137, 'd_ttr': 0.8628, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}, ('zsm', 'csl'): {'d_size': 1.0602, 'd_overlap_w': 0.0069, 'd_ttr': 25.5691, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('zsm', 'eus'): {'d_size': 0.9927, 'd_overlap_w': 0.0114, 'd_ttr': 1.1539, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.641, 'i_inv': 0.5132, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('zsm', 'ben'): {'d_size': 0.8906, 'd_overlap_w': 0.0018, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.6162, 'i_inv': 0.5867, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('zsm', 'bel'): {'d_size': 0.8638, 'd_overlap_w': 0.006, 'd_ttr': 1.1325, 'd_overlap_s': '-', 'i_syn': 0.83, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('zsm', 'kaz'): {'d_size': 0.6354, 'd_overlap_w': 0.0059, 'd_ttr': 3.4719, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.879, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('zsm', 'kat'): {'d_size': 2.5274, 'd_overlap_w': 0.0045, 'd_ttr': 0.5911, 'd_overlap_s': '-', 'i_syn': 0.8, 'i_phon': 0.5687, 'i_inv': 0.5574, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}}, 'dan': {('dan', 'arb'): {'d_size': 4.7644, 'd_overlap_w': 0.0055, 'd_ttr': 0.0071, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('dan', 'heb'): {'d_size': 4.7134, 'd_overlap_w': 0.0059, 'd_ttr': 0.006, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'rus'): {'d_size': 4.6386, 'd_overlap_w': 0.0096, 'd_ttr': 0.0058, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 0.8, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'kor'): {'d_size': 4.5759, 'd_overlap_w': 0.0064, 'd_ttr': 1.2744, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'ita'): {'d_size': 4.5506, 'd_overlap_w': 0.0448, 'd_ttr': 0.29, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.9, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'jpn'): {'d_size': 4.5414, 'd_overlap_w': 0.0018, 'd_ttr': 140.8013, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'spa'): {'d_size': 4.3619, 'd_overlap_w': 0.0411, 'd_ttr': 0.2864, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'fra'): {'d_size': 4.2791, 'd_overlap_w': 0.0516, 'd_ttr': 0.4062, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 0.9375, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'nld'): {'d_size': 4.0892, 'd_overlap_w': 0.0512, 'd_ttr': 0.1849, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 0.7, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'tur'): {'d_size': 4.0603, 'd_overlap_w': 0.0224, 'd_ttr': 0.1834, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'ron'): {'d_size': 4.0161, 'd_overlap_w': 0.0391, 'd_ttr': 0.1185, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 0.8889, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'pol'): {'d_size': 3.9201, 'd_overlap_w': 0.0263, 'd_ttr': 0.0165, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'bul'): {'d_size': 3.8817, 'd_overlap_w': 0.0091, 'd_ttr': 0.0675, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'vie'): {'d_size': 3.8272, 'd_overlap_w': 0.0726, 'd_ttr': 0.7595, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'deu'): {'d_size': 3.7358, 'd_overlap_w': 0.0378, 'd_ttr': 0.0751, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'pes'): {'d_size': 3.3593, 'd_overlap_w': 0.0123, 'd_ttr': 0.3362, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.9}, ('dan', 'hun'): {'d_size': 3.2759, 'd_overlap_w': 0.0187, 'd_ttr': 0.3961, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'srp'): {'d_size': 3.0462, 'd_overlap_w': 0.017, 'd_ttr': 0.0623, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.6}, ('dan', 'ell'): {'d_size': 2.989, 'd_overlap_w': 0.0154, 'd_ttr': 0.0438, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'hrv'): {'d_size': 2.7168, 'd_overlap_w': 0.0298, 'd_ttr': 0.009, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 0.875, 'i_geo': 0.1, 'i_fea': 0.9}, ('dan', 'ukr'): {'d_size': 2.4142, 'd_overlap_w': 0.0089, 'd_ttr': 0.0373, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'ces'): {'d_size': 2.294, 'd_overlap_w': 0.0277, 'd_ttr': 0.0676, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'tha'): {'d_size': 2.1821, 'd_overlap_w': 0.0069, 'd_ttr': 3.4465, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'ind'): {'d_size': 1.9449, 'd_overlap_w': 0.0597, 'd_ttr': 0.2109, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.6, 'i_fea': 0.8}, ('dan', 'slk'): {'d_size': 1.3678, 'd_overlap_w': 0.0258, 'd_ttr': 0.3765, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.8}, ('dan', 'swe'): {'d_size': 1.2605, 'd_overlap_w': 0.0752, 'd_ttr': 0.0058, 'd_overlap_s': '-', 'i_syn': 0.33, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 0.2857, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'por'): {'d_size': 1.1523, 'd_overlap_w': 0.0357, 'd_ttr': 0.0012, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'lit'): {'d_size': 0.9328, 'd_overlap_w': 0.0196, 'd_ttr': 1.2407, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.75, 'i_geo': 0.0, 'i_fea': 0.8}, ('dan', 'mkd'): {'d_size': 0.5638, 'd_overlap_w': 0.0088, 'd_ttr': 0.522, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'fin'): {'d_size': 0.539, 'd_overlap_w': 0.0174, 'd_ttr': 5.9118, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'mya'): {'d_size': 0.4783, 'd_overlap_w': 0.0153, 'd_ttr': 0.1666, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'hye'): {'d_size': 0.4753, 'd_overlap_w': 0.0068, 'd_ttr': 1.2434, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'slv'): {'d_size': 0.4413, 'd_overlap_w': 0.0236, 'd_ttr': 1.5901, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'hin'): {'d_size': 0.4183, 'd_overlap_w': 0.0066, 'd_ttr': 0.0266, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('dan', 'nob'): {'d_size': 0.3521, 'd_overlap_w': 0.1186, 'd_ttr': 0.3276, 'd_overlap_s': '-', 'i_syn': 0.22, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('dan', 'est'): {'d_size': 0.2389, 'd_overlap_w': 0.0145, 'd_ttr': 5.5426, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('dan', 'kur'): {'d_size': 0.2308, 'd_overlap_w': 0.0016, 'd_ttr': 1.9247, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('dan', 'glg'): {'d_size': 0.2229, 'd_overlap_w': 0.0226, 'd_ttr': 1.2665, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'mar'): {'d_size': 0.219, 'd_overlap_w': 0.0019, 'd_ttr': 0.5387, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('dan', 'tam'): {'d_size': 0.1385, 'd_overlap_w': 0.0042, 'd_ttr': 0.0169, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'urd'): {'d_size': 0.133, 'd_overlap_w': 0.0053, 'd_ttr': 0.1775, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('dan', 'azb'): {'d_size': 0.1323, 'd_overlap_w': 0.012, 'd_ttr': 11.3019, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('dan', 'bos'): {'d_size': 0.126, 'd_overlap_w': 0.014, 'd_ttr': 6.9806, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.875, 'i_geo': 0.1, 'i_fea': 0.5}, ('dan', 'csl'): {'d_size': 0.1231, 'd_overlap_w': 0.0043, 'd_ttr': 108.9115, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'eus'): {'d_size': 0.1153, 'd_overlap_w': 0.0104, 'd_ttr': 8.5061, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('dan', 'ben'): {'d_size': 0.1034, 'd_overlap_w': 0.001, 'd_ttr': 0.7707, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.8}, ('dan', 'bel'): {'d_size': 0.1003, 'd_overlap_w': 0.0039, 'd_ttr': 8.3959, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.8, 'i_geo': 0.1, 'i_fea': 0.7}, ('dan', 'kaz'): {'d_size': 0.0738, 'd_overlap_w': 0.0031, 'd_ttr': 19.4174, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('dan', 'kat'): {'d_size': 0.2936, 'd_overlap_w': 0.005, 'd_ttr': 5.4754, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('dan', 'zsm'): {'d_size': 0.1162, 'd_overlap_w': 0.018, 'd_ttr': 0.7889, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}}, 'fin': {('fin', 'arb'): {'d_size': 8.8395, 'd_overlap_w': 0.0028, 'd_ttr': 0.4679, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('fin', 'heb'): {'d_size': 8.7449, 'd_overlap_w': 0.0032, 'd_ttr': 0.5347, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5687, 'i_inv': 0.552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('fin', 'rus'): {'d_size': 8.6061, 'd_overlap_w': 0.0048, 'd_ttr': 0.5341, 'd_overlap_s': '-', 'i_syn': 0.38, 'i_phon': 0.2804, 'i_inv': 0.5099, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'kor'): {'d_size': 8.4898, 'd_overlap_w': 0.003, 'd_ttr': 0.1441, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.4638, 'i_inv': 0.3638, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('fin', 'ita'): {'d_size': 8.4429, 'd_overlap_w': 0.0166, 'd_ttr': 0.7491, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.4906, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'jpn'): {'d_size': 8.4258, 'd_overlap_w': 0.001, 'd_ttr': 7.5596, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5032, 'i_inv': 0.5146, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'spa'): {'d_size': 8.0929, 'd_overlap_w': 0.0155, 'd_ttr': 0.7474, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4477, 'i_inv': 0.5617, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'fra'): {'d_size': 7.9392, 'd_overlap_w': 0.017, 'd_ttr': 0.7998, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3277, 'i_inv': 0.3996, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'nld'): {'d_size': 7.5868, 'd_overlap_w': 0.015, 'd_ttr': 0.6954, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5687, 'i_inv': 0.4561, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'tur'): {'d_size': 7.5332, 'd_overlap_w': 0.0091, 'd_ttr': 0.3408, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.195, 'i_inv': 0.3885, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'ron'): {'d_size': 7.4512, 'd_overlap_w': 0.014, 'd_ttr': 0.6543, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.3498, 'i_inv': 0.4795, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'pol'): {'d_size': 7.2731, 'd_overlap_w': 0.0102, 'd_ttr': 0.4504, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.2804, 'i_inv': 0.5146, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'bul'): {'d_size': 7.2019, 'd_overlap_w': 0.0049, 'd_ttr': 0.6151, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.2804, 'i_inv': 0.4599, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'vie'): {'d_size': 7.1008, 'd_overlap_w': 0.0261, 'd_ttr': 0.9265, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.427, 'i_inv': 0.5211, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'deu'): {'d_size': 6.9312, 'd_overlap_w': 0.0128, 'd_ttr': 0.6217, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.1864, 'i_inv': 0.4201, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'pes'): {'d_size': 6.2326, 'd_overlap_w': 0.0063, 'd_ttr': 0.7701, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3433, 'i_inv': 0.4572, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'hun'): {'d_size': 6.0779, 'd_overlap_w': 0.0082, 'd_ttr': 0.2758, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.195, 'i_inv': 0.3235, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.4}, ('fin', 'srp'): {'d_size': 5.6518, 'd_overlap_w': 0.007, 'd_ttr': 0.4043, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8632, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('fin', 'ell'): {'d_size': 5.5457, 'd_overlap_w': 0.0069, 'd_ttr': 0.5922, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.3433, 'i_inv': 0.5463, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'hrv'): {'d_size': 5.0405, 'd_overlap_w': 0.012, 'd_ttr': 0.4636, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.448, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'ukr'): {'d_size': 4.4792, 'd_overlap_w': 0.0047, 'd_ttr': 0.4255, 'd_overlap_s': '-', 'i_syn': 0.48, 'i_phon': 0.5687, 'i_inv': 0.4906, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'ces'): {'d_size': 4.2562, 'd_overlap_w': 0.0114, 'd_ttr': 0.4004, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.4818, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'tha'): {'d_size': 4.0486, 'd_overlap_w': 0.0033, 'd_ttr': 0.0281, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.39, 'i_inv': 0.3995, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('fin', 'ind'): {'d_size': 3.6085, 'd_overlap_w': 0.0211, 'd_ttr': 0.7097, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2736, 'i_inv': 0.4034, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.5}, ('fin', 'slk'): {'d_size': 2.5378, 'd_overlap_w': 0.0114, 'd_ttr': 0.2806, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'swe'): {'d_size': 2.3387, 'd_overlap_w': 0.0162, 'd_ttr': 0.4711, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5687, 'i_inv': 0.4418, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('fin', 'por'): {'d_size': 2.1379, 'd_overlap_w': 0.0141, 'd_ttr': 0.5163, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.454, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'dan'): {'d_size': 1.8553, 'd_overlap_w': 0.0174, 'd_ttr': 0.5021, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('fin', 'lit'): {'d_size': 1.7306, 'd_overlap_w': 0.0099, 'd_ttr': 0.1474, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3498, 'i_inv': 0.4657, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'mkd'): {'d_size': 1.0459, 'd_overlap_w': 0.0048, 'd_ttr': 0.248, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.5687, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'mya'): {'d_size': 0.8875, 'd_overlap_w': 0.0069, 'd_ttr': 0.3477, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.3695, 'i_inv': 0.448, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'hye'): {'d_size': 0.8818, 'd_overlap_w': 0.0036, 'd_ttr': 0.1472, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5032, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'slv'): {'d_size': 0.8187, 'd_overlap_w': 0.011, 'd_ttr': 0.1163, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('fin', 'hin'): {'d_size': 0.7761, 'd_overlap_w': 0.0042, 'd_ttr': 0.5717, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3433, 'i_inv': 0.5129, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('fin', 'nob'): {'d_size': 0.6533, 'd_overlap_w': 0.0133, 'd_ttr': 0.2935, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.3433, 'i_inv': 0.4626, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('fin', 'est'): {'d_size': 0.4433, 'd_overlap_w': 0.0113, 'd_ttr': 0.0005, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('fin', 'kur'): {'d_size': 0.4282, 'd_overlap_w': 0.0011, 'd_ttr': 0.0926, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('fin', 'glg'): {'d_size': 0.4135, 'd_overlap_w': 0.01, 'd_ttr': 0.1449, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5687, 'i_inv': 0.5011, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('fin', 'mar'): {'d_size': 0.4062, 'd_overlap_w': 0.0012, 'd_ttr': 0.2447, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5687, 'i_inv': 0.4824, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'tam'): {'d_size': 0.257, 'd_overlap_w': 0.003, 'd_ttr': 0.4498, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5687, 'i_inv': 0.5338, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'urd'): {'d_size': 0.2468, 'd_overlap_w': 0.0029, 'd_ttr': 0.3432, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5687, 'i_inv': 0.5551, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'azb'): {'d_size': 0.2455, 'd_overlap_w': 0.0059, 'd_ttr': 0.0735, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.195, 'i_inv': 0.5207, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('fin', 'bos'): {'d_size': 0.2338, 'd_overlap_w': 0.0062, 'd_ttr': 0.0038, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('fin', 'csl'): {'d_size': 0.2285, 'd_overlap_w': 0.0029, 'd_ttr': 5.4418, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('fin', 'eus'): {'d_size': 0.2139, 'd_overlap_w': 0.0058, 'd_ttr': 0.02, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.3498, 'i_inv': 0.5621, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('fin', 'ben'): {'d_size': 0.1919, 'd_overlap_w': 0.0005, 'd_ttr': 0.205, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4054, 'i_inv': 0.5071, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('fin', 'bel'): {'d_size': 0.1862, 'd_overlap_w': 0.0025, 'd_ttr': 0.0185, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('fin', 'kaz'): {'d_size': 0.1369, 'd_overlap_w': 0.0021, 'd_ttr': 0.3313, 'd_overlap_s': '-', 'i_syn': 0.9, 'i_phon': 0.805, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('fin', 'kat'): {'d_size': 0.5447, 'd_overlap_w': 0.0032, 'd_ttr': 0.0007, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4816, 'i_inv': 0.5823, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('fin', 'zsm'): {'d_size': 0.2155, 'd_overlap_w': 0.0081, 'd_ttr': 0.2023, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 0.5687, 'i_inv': 0.4533, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.6}}, 'bos': {('bos', 'arb'): {'d_size': 37.8021, 'd_overlap_w': 0.0015, 'd_ttr': 0.4933, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('bos', 'heb'): {'d_size': 37.3974, 'd_overlap_w': 0.0017, 'd_ttr': 0.5577, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'rus'): {'d_size': 36.804, 'd_overlap_w': 0.0024, 'd_ttr': 0.5571, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 0.4, 'i_geo': 0.2, 'i_fea': 0.7}, ('bos', 'kor'): {'d_size': 36.3065, 'd_overlap_w': 0.0015, 'd_ttr': 0.1726, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'ita'): {'d_size': 36.1058, 'd_overlap_w': 0.0106, 'd_ttr': 0.7627, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.9, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'jpn'): {'d_size': 36.0328, 'd_overlap_w': 0.0005, 'd_ttr': 6.414, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.8}, ('bos', 'spa'): {'d_size': 34.6091, 'd_overlap_w': 0.0094, 'd_ttr': 0.761, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'fra'): {'d_size': 33.952, 'd_overlap_w': 0.0101, 'd_ttr': 0.8108, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 0.9375, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'nld'): {'d_size': 32.4447, 'd_overlap_w': 0.0085, 'd_ttr': 0.7115, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 0.9, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'tur'): {'d_size': 32.2157, 'd_overlap_w': 0.0053, 'd_ttr': 0.3695, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'ron'): {'d_size': 31.8651, 'd_overlap_w': 0.0099, 'd_ttr': 0.6723, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 0.8889, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'pol'): {'d_size': 31.1033, 'd_overlap_w': 0.0086, 'd_ttr': 0.4763, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'bul'): {'d_size': 30.7987, 'd_overlap_w': 0.0029, 'd_ttr': 0.6348, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'vie'): {'d_size': 30.3663, 'd_overlap_w': 0.0201, 'd_ttr': 0.9307, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'deu'): {'d_size': 29.6412, 'd_overlap_w': 0.0065, 'd_ttr': 0.6411, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 0.8571, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'pes'): {'d_size': 26.6534, 'd_overlap_w': 0.0042, 'd_ttr': 0.7826, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('bos', 'hun'): {'d_size': 25.9921, 'd_overlap_w': 0.0041, 'd_ttr': 0.3054, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'srp'): {'d_size': 24.1698, 'd_overlap_w': 0.0763, 'd_ttr': 0.4315, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 0.125, 'i_geo': 0.0, 'i_fea': 0.6}, ('bos', 'ell'): {'d_size': 23.7159, 'd_overlap_w': 0.0041, 'd_ttr': 0.6129, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 0.8333, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'hrv'): {'d_size': 21.5556, 'd_overlap_w': 0.1028, 'd_ttr': 0.4891, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 0.125, 'i_geo': 0.0, 'i_fea': 0.9}, ('bos', 'ukr'): {'d_size': 19.1552, 'd_overlap_w': 0.0028, 'd_ttr': 0.4522, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'ces'): {'d_size': 18.2014, 'd_overlap_w': 0.012, 'd_ttr': 0.4278, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 0.5714, 'i_geo': 0.0, 'i_fea': 0.7}, ('bos', 'tha'): {'d_size': 17.3136, 'd_overlap_w': 0.0016, 'd_ttr': 0.0465, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'ind'): {'d_size': 15.4319, 'd_overlap_w': 0.0175, 'd_ttr': 0.7251, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.8}, ('bos', 'slk'): {'d_size': 10.8528, 'd_overlap_w': 0.0155, 'd_ttr': 0.3102, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.625, 'i_geo': 0.0, 'i_fea': 0.8}, ('bos', 'swe'): {'d_size': 10.0012, 'd_overlap_w': 0.0119, 'd_ttr': 0.4963, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'por'): {'d_size': 9.1428, 'd_overlap_w': 0.0118, 'd_ttr': 0.5399, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 0.9333, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'dan'): {'d_size': 7.9343, 'd_overlap_w': 0.014, 'd_ttr': 0.5263, 'd_overlap_s': '-', 'i_syn': 0.45, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.875, 'i_geo': 0.1, 'i_fea': 0.5}, ('bos', 'lit'): {'d_size': 7.401, 'd_overlap_w': 0.0088, 'd_ttr': 0.1761, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'mkd'): {'d_size': 4.473, 'd_overlap_w': 0.0046, 'd_ttr': 0.2778, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 0.4286, 'i_geo': 0.0, 'i_fea': 0.8}, ('bos', 'fin'): {'d_size': 4.2765, 'd_overlap_w': 0.0062, 'd_ttr': 0.0033, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'mya'): {'d_size': 3.7954, 'd_overlap_w': 0.004, 'd_ttr': 0.3762, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.9}, ('bos', 'hye'): {'d_size': 3.7712, 'd_overlap_w': 0.0039, 'd_ttr': 0.1758, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'slv'): {'d_size': 3.5012, 'd_overlap_w': 0.0606, 'd_ttr': 0.1438, 'd_overlap_s': '-', 'i_syn': 0.36, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 0.1667, 'i_geo': 0.0, 'i_fea': 0.6}, ('bos', 'hin'): {'d_size': 3.3189, 'd_overlap_w': 0.0052, 'd_ttr': 0.5932, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'nob'): {'d_size': 2.794, 'd_overlap_w': 0.0147, 'd_ttr': 0.3229, 'd_overlap_s': '-', 'i_syn': 0.39, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'est'): {'d_size': 1.8958, 'd_overlap_w': 0.0093, 'd_ttr': 0.0062, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('bos', 'kur'): {'d_size': 1.831, 'd_overlap_w': 0.0016, 'd_ttr': 0.1187, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('bos', 'glg'): {'d_size': 1.7685, 'd_overlap_w': 0.0128, 'd_ttr': 0.1734, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 0.9231, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'mar'): {'d_size': 1.7373, 'd_overlap_w': 0.0017, 'd_ttr': 0.2745, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'tam'): {'d_size': 1.0989, 'd_overlap_w': 0.0041, 'd_ttr': 0.4757, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'urd'): {'d_size': 1.0553, 'd_overlap_w': 0.0049, 'd_ttr': 0.3718, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.8}, ('bos', 'azb'): {'d_size': 1.0498, 'd_overlap_w': 0.0095, 'd_ttr': 0.0391, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'csl'): {'d_size': 0.977, 'd_overlap_w': 0.0037, 'd_ttr': 4.5795, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('bos', 'eus'): {'d_size': 0.9149, 'd_overlap_w': 0.0088, 'd_ttr': 0.0057, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'ben'): {'d_size': 0.8208, 'd_overlap_w': 0.0011, 'd_ttr': 0.2346, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('bos', 'bel'): {'d_size': 0.7961, 'd_overlap_w': 0.0038, 'd_ttr': 0.0049, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 0.4, 'i_geo': 0.1, 'i_fea': 0.6}, ('bos', 'kaz'): {'d_size': 0.5856, 'd_overlap_w': 0.0035, 'd_ttr': 0.2347, 'd_overlap_s': '-', 'i_syn': 0.86, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('bos', 'kat'): {'d_size': 2.3293, 'd_overlap_w': 0.0031, 'd_ttr': 0.0069, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('bos', 'zsm'): {'d_size': 0.9216, 'd_overlap_w': 0.0137, 'd_ttr': 0.2319, 'd_overlap_s': '-', 'i_syn': 0.7, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}}, 'est': {('est', 'arb'): {'d_size': 19.9396, 'd_overlap_w': 0.002, 'd_ttr': 0.458, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('est', 'heb'): {'d_size': 19.7261, 'd_overlap_w': 0.0023, 'd_ttr': 0.5257, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.7028, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('est', 'rus'): {'d_size': 19.4131, 'd_overlap_w': 0.0032, 'd_ttr': 0.5251, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.688, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}, ('est', 'kor'): {'d_size': 19.1507, 'd_overlap_w': 0.002, 'd_ttr': 0.1335, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.641, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.7}, ('est', 'ita'): {'d_size': 19.0448, 'd_overlap_w': 0.0118, 'd_ttr': 0.7438, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'jpn'): {'d_size': 19.0063, 'd_overlap_w': 0.0006, 'd_ttr': 8.0411, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'spa'): {'d_size': 18.2554, 'd_overlap_w': 0.0114, 'd_ttr': 0.742, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5922, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'fra'): {'d_size': 17.9087, 'd_overlap_w': 0.0129, 'd_ttr': 0.7955, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5456, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'nld'): {'d_size': 17.1137, 'd_overlap_w': 0.0113, 'd_ttr': 0.689, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.0002, 'i_inv': 0.6068, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'tur'): {'d_size': 16.9929, 'd_overlap_w': 0.0063, 'd_ttr': 0.3297, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'ron'): {'d_size': 16.808, 'd_overlap_w': 0.0101, 'd_ttr': 0.6472, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.641, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'pol'): {'d_size': 16.4061, 'd_overlap_w': 0.0075, 'd_ttr': 0.4403, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.6162, 'i_inv': 0.6597, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'bul'): {'d_size': 16.2455, 'd_overlap_w': 0.0036, 'd_ttr': 0.6073, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.6162, 'i_inv': 0.6416, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'vie'): {'d_size': 16.0174, 'd_overlap_w': 0.0219, 'd_ttr': 0.9249, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'deu'): {'d_size': 15.6349, 'd_overlap_w': 0.0092, 'd_ttr': 0.614, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5456, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'pes'): {'d_size': 14.0589, 'd_overlap_w': 0.005, 'd_ttr': 0.7652, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5922, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('est', 'hun'): {'d_size': 13.7101, 'd_overlap_w': 0.0056, 'd_ttr': 0.2645, 'd_overlap_s': '-', 'i_syn': 0.48, 'i_phon': 0.5922, 'i_inv': 0.6461, 'i_fam': 0.5, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'srp'): {'d_size': 12.7489, 'd_overlap_w': 0.0062, 'd_ttr': 0.3937, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.8278, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('est', 'ell'): {'d_size': 12.5095, 'd_overlap_w': 0.0052, 'd_ttr': 0.5841, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5922, 'i_inv': 0.6929, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'hrv'): {'d_size': 11.37, 'd_overlap_w': 0.0098, 'd_ttr': 0.4537, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('est', 'ukr'): {'d_size': 10.1038, 'd_overlap_w': 0.0032, 'd_ttr': 0.4152, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'ces'): {'d_size': 9.6008, 'd_overlap_w': 0.0087, 'd_ttr': 0.3898, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.0002, 'i_inv': 0.6784, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'tha'): {'d_size': 9.1324, 'd_overlap_w': 0.002, 'd_ttr': 0.022, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5687, 'i_inv': 0.6372, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'ind'): {'d_size': 8.1399, 'd_overlap_w': 0.0181, 'd_ttr': 0.7036, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.7}, ('est', 'slk'): {'d_size': 5.7245, 'd_overlap_w': 0.0094, 'd_ttr': 0.2693, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'swe'): {'d_size': 5.2754, 'd_overlap_w': 0.0137, 'd_ttr': 0.4612, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6153, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'por'): {'d_size': 4.8226, 'd_overlap_w': 0.0118, 'd_ttr': 0.507, 'd_overlap_s': '-', 'i_syn': 0.51, 'i_phon': 0.0002, 'i_inv': 0.6328, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'dan'): {'d_size': 4.1851, 'd_overlap_w': 0.0145, 'd_ttr': 0.4926, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('est', 'lit'): {'d_size': 3.9038, 'd_overlap_w': 0.0078, 'd_ttr': 0.1368, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.641, 'i_inv': 0.6506, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'mkd'): {'d_size': 2.3594, 'd_overlap_w': 0.0043, 'd_ttr': 0.2367, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.0002, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'fin'): {'d_size': 2.2557, 'd_overlap_w': 0.0113, 'd_ttr': 0.0005, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.5687, 'i_inv': 0.6506, 'i_fam': 0.5, 'i_geo': 0.0, 'i_fea': 0.7}, ('est', 'mya'): {'d_size': 2.002, 'd_overlap_w': 0.0055, 'd_ttr': 0.3366, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5228, 'i_inv': 0.624, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.9}, ('est', 'hye'): {'d_size': 1.9892, 'd_overlap_w': 0.0038, 'd_ttr': 0.1365, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'slv'): {'d_size': 1.8468, 'd_overlap_w': 0.011, 'd_ttr': 0.1062, 'd_overlap_s': '-', 'i_syn': 0.43, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'hin'): {'d_size': 1.7506, 'd_overlap_w': 0.0043, 'd_ttr': 0.5632, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'nob'): {'d_size': 1.4737, 'd_overlap_w': 0.0148, 'd_ttr': 0.2822, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.5922, 'i_inv': 0.5898, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('est', 'kur'): {'d_size': 0.9658, 'd_overlap_w': 0.0013, 'd_ttr': 0.0831, 'd_overlap_s': '-', 'i_syn': 0.82, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('est', 'glg'): {'d_size': 0.9329, 'd_overlap_w': 0.0117, 'd_ttr': 0.1342, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.0002, 'i_inv': 0.6552, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('est', 'mar'): {'d_size': 0.9164, 'd_overlap_w': 0.0015, 'd_ttr': 0.2333, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.6284, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'tam'): {'d_size': 0.5796, 'd_overlap_w': 0.0035, 'd_ttr': 0.4397, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.0002, 'i_inv': 0.6196, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'urd'): {'d_size': 0.5566, 'd_overlap_w': 0.0039, 'd_ttr': 0.3321, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.0002, 'i_inv': 0.6025, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'azb'): {'d_size': 0.5537, 'd_overlap_w': 0.0077, 'd_ttr': 0.0902, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('est', 'bos'): {'d_size': 0.5275, 'd_overlap_w': 0.0093, 'd_ttr': 0.0074, 'd_overlap_s': '-', 'i_syn': 0.42, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.4}, ('est', 'csl'): {'d_size': 0.5154, 'd_overlap_w': 0.0036, 'd_ttr': 5.8052, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.8}, ('est', 'eus'): {'d_size': 0.4826, 'd_overlap_w': 0.0066, 'd_ttr': 0.0281, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.641, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('est', 'ben'): {'d_size': 0.4329, 'd_overlap_w': 0.0008, 'd_ttr': 0.1937, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('est', 'bel'): {'d_size': 0.4199, 'd_overlap_w': 0.0033, 'd_ttr': 0.0262, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.0002, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('est', 'kaz'): {'d_size': 0.3089, 'd_overlap_w': 0.0032, 'd_ttr': 0.3743, 'd_overlap_s': '-', 'i_syn': 0.87, 'i_phon': 0.879, 'i_inv': 0.0, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('est', 'kat'): {'d_size': 1.2286, 'd_overlap_w': 0.0032, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5687, 'i_inv': 0.6832, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('est', 'zsm'): {'d_size': 0.4861, 'd_overlap_w': 0.0108, 'd_ttr': 0.191, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.0002, 'i_inv': 0.669, 'i_fam': 1.0, 'i_geo': 0.5, 'i_fea': 0.9}}, 'hye': {('hye', 'arb'): {'d_size': 10.0239, 'd_overlap_w': 0.0022, 'd_ttr': 0.2376, 'd_overlap_s': 0.0267, 'i_syn': 0.71, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.8}, ('hye', 'heb'): {'d_size': 9.9166, 'd_overlap_w': 0.0025, 'd_ttr': 0.318, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.4911, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'rus'): {'d_size': 9.7593, 'd_overlap_w': 0.0031, 'd_ttr': 0.3173, 'd_overlap_s': 0.0323, 'i_syn': 0.58, 'i_phon': 0.4327, 'i_inv': 0.4693, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'kor'): {'d_size': 9.6273, 'd_overlap_w': 0.0015, 'd_ttr': 0.0, 'd_overlap_s': 0.0262, 'i_syn': 0.52, 'i_phon': 0.5, 'i_inv': 0.5429, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hye', 'ita'): {'d_size': 9.5741, 'd_overlap_w': 0.0056, 'd_ttr': 0.6113, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.6162, 'i_inv': 0.5203, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'jpn'): {'d_size': 9.5548, 'd_overlap_w': 0.0006, 'd_ttr': 25.8367, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5354, 'i_inv': 0.5204, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('hye', 'spa'): {'d_size': 9.1772, 'd_overlap_w': 0.0056, 'd_ttr': 0.6088, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.4717, 'i_inv': 0.5037, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'fra'): {'d_size': 9.003, 'd_overlap_w': 0.0062, 'd_ttr': 0.6865, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.4407, 'i_inv': 0.5006, 'i_fam': 0.9375, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'nld'): {'d_size': 8.6033, 'd_overlap_w': 0.0051, 'd_ttr': 0.5336, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.6162, 'i_inv': 0.5339, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'tur'): {'d_size': 8.5426, 'd_overlap_w': 0.0031, 'd_ttr': 0.1055, 'd_overlap_s': 0.074, 'i_syn': 0.42, 'i_phon': 0.4717, 'i_inv': 0.4963, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'ron'): {'d_size': 8.4496, 'd_overlap_w': 0.0049, 'd_ttr': 0.476, 'd_overlap_s': 0.0804, 'i_syn': 0.52, 'i_phon': 0.3824, 'i_inv': 0.3812, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.4}, ('hye', 'pol'): {'d_size': 8.2476, 'd_overlap_w': 0.0037, 'd_ttr': 0.2175, 'd_overlap_s': 0.0716, 'i_syn': 0.57, 'i_phon': 0.4327, 'i_inv': 0.5204, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'bul'): {'d_size': 8.1669, 'd_overlap_w': 0.0035, 'd_ttr': 0.4225, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.4327, 'i_inv': 0.5052, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'vie'): {'d_size': 8.0522, 'd_overlap_w': 0.0108, 'd_ttr': 0.8822, 'd_overlap_s': 0.089, 'i_syn': 0.77, 'i_phon': 0.5295, 'i_inv': 0.5422, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'deu'): {'d_size': 7.8599, 'd_overlap_w': 0.0046, 'd_ttr': 0.4314, 'd_overlap_s': 0.0885, 'i_syn': 0.57, 'i_phon': 0.4407, 'i_inv': 0.4819, 'i_fam': 0.8571, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'pes'): {'d_size': 7.0676, 'd_overlap_w': 0.0041, 'd_ttr': 0.6422, 'd_overlap_s': 0.0258, 'i_syn': 0.63, 'i_phon': 0.4717, 'i_inv': 0.3203, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'hun'): {'d_size': 6.8923, 'd_overlap_w': 0.0028, 'd_ttr': 0.0527, 'd_overlap_s': 0.0714, 'i_syn': 0.43, 'i_phon': 0.4717, 'i_inv': 0.4756, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'srp'): {'d_size': 6.4091, 'd_overlap_w': 0.003, 'd_ttr': 0.1675, 'd_overlap_s': 0.0615, 'i_syn': 0.67, 'i_phon': 1.0, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.8}, ('hye', 'ell'): {'d_size': 6.2887, 'd_overlap_w': 0.0043, 'd_ttr': 0.392, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4717, 'i_inv': 0.5346, 'i_fam': 0.8333, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'hrv'): {'d_size': 5.7159, 'd_overlap_w': 0.0043, 'd_ttr': 0.2326, 'd_overlap_s': 0.0691, 'i_syn': 0.8, 'i_phon': 0.6162, 'i_inv': 0.4263, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'ukr'): {'d_size': 5.0794, 'd_overlap_w': 0.0037, 'd_ttr': 0.19, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.4726, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'ces'): {'d_size': 4.8265, 'd_overlap_w': 0.0042, 'd_ttr': 0.1634, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.6162, 'i_inv': 0.4882, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'tha'): {'d_size': 4.591, 'd_overlap_w': 0.0017, 'd_ttr': 0.1229, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.5879, 'i_inv': 0.5531, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'ind'): {'d_size': 4.092, 'd_overlap_w': 0.0084, 'd_ttr': 0.554, 'd_overlap_s': 0.081, 'i_syn': 0.71, 'i_phon': 0.5032, 'i_inv': 0.404, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('hye', 'slk'): {'d_size': 2.8778, 'd_overlap_w': 0.0044, 'd_ttr': 0.0562, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'swe'): {'d_size': 2.652, 'd_overlap_w': 0.0061, 'd_ttr': 0.2412, 'd_overlap_s': 0.0788, 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.5797, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'por'): {'d_size': 2.4244, 'd_overlap_w': 0.0058, 'd_ttr': 0.2952, 'd_overlap_s': 0.0742, 'i_syn': 0.6, 'i_phon': 0.6162, 'i_inv': 0.4975, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'dan'): {'d_size': 2.1039, 'd_overlap_w': 0.0068, 'd_ttr': 0.2779, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.7}, ('hye', 'lit'): {'d_size': 1.9625, 'd_overlap_w': 0.0045, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5, 'i_inv': 0.4906, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hye', 'mkd'): {'d_size': 1.1861, 'd_overlap_w': 0.0043, 'd_ttr': 0.0345, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.6162, 'i_inv': 0.3926, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'fin'): {'d_size': 1.134, 'd_overlap_w': 0.0036, 'd_ttr': 0.3873, 'd_overlap_s': '-', 'i_syn': 0.56, 'i_phon': 0.5032, 'i_inv': 0.4676, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hye', 'mya'): {'d_size': 1.0064, 'd_overlap_w': 0.0022, 'd_ttr': 0.1117, 'd_overlap_s': 0.0419, 'i_syn': 0.64, 'i_phon': 0.5519, 'i_inv': 0.5308, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'slv'): {'d_size': 0.9284, 'd_overlap_w': 0.0046, 'd_ttr': 0.0048, 'd_overlap_s': 0.0642, 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'hin'): {'d_size': 0.8801, 'd_overlap_w': 0.0039, 'd_ttr': 0.3652, 'd_overlap_s': 0.0291, 'i_syn': 0.59, 'i_phon': 0.4717, 'i_inv': 0.5081, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.5}, ('hye', 'nob'): {'d_size': 0.7409, 'd_overlap_w': 0.0059, 'd_ttr': 0.0658, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.5641, 'i_inv': 0.589, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'est'): {'d_size': 0.5027, 'd_overlap_w': 0.0038, 'd_ttr': 0.3433, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}, ('hye', 'kur'): {'d_size': 0.4855, 'd_overlap_w': 0.0012, 'd_ttr': 0.0166, 'd_overlap_s': 0.0207, 'i_syn': 0.71, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.8}, ('hye', 'glg'): {'d_size': 0.469, 'd_overlap_w': 0.0049, 'd_ttr': 0.0, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5056, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'mar'): {'d_size': 0.4607, 'd_overlap_w': 0.0011, 'd_ttr': 0.0325, 'd_overlap_s': 0.021, 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5448, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'tam'): {'d_size': 0.2914, 'd_overlap_w': 0.0034, 'd_ttr': 0.2169, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.5747, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'urd'): {'d_size': 0.2798, 'd_overlap_w': 0.0035, 'd_ttr': 0.1076, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5577, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.6}, ('hye', 'azb'): {'d_size': 0.2784, 'd_overlap_w': 0.0036, 'd_ttr': 1.1284, 'd_overlap_s': '-', 'i_syn': 0.62, 'i_phon': 0.4717, 'i_inv': 0.55, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.6}, ('hye', 'bos'): {'d_size': 0.2652, 'd_overlap_w': 0.0039, 'd_ttr': 0.5212, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.7}, ('hye', 'csl'): {'d_size': 0.2591, 'd_overlap_w': 0.0027, 'd_ttr': 19.421, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('hye', 'eus'): {'d_size': 0.2426, 'd_overlap_w': 0.003, 'd_ttr': 0.7254, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5, 'i_inv': 0.4815, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hye', 'ben'): {'d_size': 0.2176, 'd_overlap_w': 0.0006, 'd_ttr': 0.0126, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5354, 'i_inv': 0.5625, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('hye', 'bel'): {'d_size': 0.2111, 'd_overlap_w': 0.003, 'd_ttr': 0.7102, 'd_overlap_s': 0.0277, 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.6737, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.8}, ('hye', 'kaz'): {'d_size': 0.1553, 'd_overlap_w': 0.0027, 'd_ttr': 2.4217, 'd_overlap_s': 0.0267, 'i_syn': 0.8, 'i_phon': 1.0, 'i_inv': 0.6737, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('hye', 'kat'): {'d_size': 0.6176, 'd_overlap_w': 0.0035, 'd_ttr': 0.3354, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.4054, 'i_inv': 0.3673, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.4}, ('hye', 'zsm'): {'d_size': 0.2444, 'd_overlap_w': 0.0052, 'd_ttr': 0.0115, 'd_overlap_s': 0.0579, 'i_syn': 0.8, 'i_phon': 0.6162, 'i_inv': 0.4569, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}}, 'hin': {('hin', 'arb'): {'d_size': 11.3901, 'd_overlap_w': 0.0018, 'd_ttr': 0.0873, 'd_overlap_s': '-', 'i_syn': 0.74, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('hin', 'heb'): {'d_size': 11.2682, 'd_overlap_w': 0.002, 'd_ttr': 0.0104, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.5629, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hin', 'rus'): {'d_size': 11.0894, 'd_overlap_w': 0.0023, 'd_ttr': 0.0107, 'd_overlap_s': '-', 'i_syn': 0.5, 'i_phon': 0.2048, 'i_inv': 0.6015, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'kor'): {'d_size': 10.9395, 'd_overlap_w': 0.0014, 'd_ttr': 2.3832, 'd_overlap_s': '-', 'i_syn': 0.47, 'i_phon': 0.5319, 'i_inv': 0.558, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'ita'): {'d_size': 10.879, 'd_overlap_w': 0.0051, 'd_ttr': 0.2013, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.5922, 'i_inv': 0.5418, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'jpn'): {'d_size': 10.857, 'd_overlap_w': 0.0005, 'd_ttr': 206.5918, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.4717, 'i_inv': 0.498, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'spa'): {'d_size': 10.428, 'd_overlap_w': 0.0051, 'd_ttr': 0.1977, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.4097, 'i_inv': 0.5922, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('hin', 'fra'): {'d_size': 10.23, 'd_overlap_w': 0.0058, 'd_ttr': 0.3211, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2677, 'i_inv': 0.4618, 'i_fam': 0.9375, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'nld'): {'d_size': 9.7759, 'd_overlap_w': 0.005, 'd_ttr': 0.1017, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5922, 'i_inv': 0.4861, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'tur'): {'d_size': 9.7069, 'd_overlap_w': 0.0025, 'd_ttr': 0.4992, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.2871, 'i_inv': 0.5418, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'ron'): {'d_size': 9.6012, 'd_overlap_w': 0.0043, 'd_ttr': 0.0468, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.2952, 'i_inv': 0.489, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'pol'): {'d_size': 9.3717, 'd_overlap_w': 0.003, 'd_ttr': 0.1215, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2048, 'i_inv': 0.5925, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'bul'): {'d_size': 9.2799, 'd_overlap_w': 0.0026, 'd_ttr': 0.0133, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.2048, 'i_inv': 0.5418, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'vie'): {'d_size': 9.1496, 'd_overlap_w': 0.0119, 'd_ttr': 0.7165, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.4788, 'i_inv': 0.5186, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hin', 'deu'): {'d_size': 8.9312, 'd_overlap_w': 0.004, 'd_ttr': 0.0176, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.3862, 'i_inv': 0.5037, 'i_fam': 0.8571, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'pes'): {'d_size': 8.0309, 'd_overlap_w': 0.0041, 'd_ttr': 0.248, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.2871, 'i_inv': 0.489, 'i_fam': 0.75, 'i_geo': 0.1, 'i_fea': 0.5}, ('hin', 'hun'): {'d_size': 7.8316, 'd_overlap_w': 0.0023, 'd_ttr': 0.8966, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.2871, 'i_inv': 0.5039, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'srp'): {'d_size': 7.2826, 'd_overlap_w': 0.0024, 'd_ttr': 0.2431, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.8564, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'ell'): {'d_size': 7.1458, 'd_overlap_w': 0.0035, 'd_ttr': 0.003, 'd_overlap_s': '-', 'i_syn': 0.55, 'i_phon': 0.2871, 'i_inv': 0.6108, 'i_fam': 0.8333, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'hrv'): {'d_size': 6.4949, 'd_overlap_w': 0.0037, 'd_ttr': 0.0951, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5922, 'i_inv': 0.5263, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'ukr'): {'d_size': 5.7716, 'd_overlap_w': 0.0026, 'd_ttr': 0.1811, 'd_overlap_s': '-', 'i_syn': 0.52, 'i_phon': 0.5922, 'i_inv': 0.5252, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'ces'): {'d_size': 5.4843, 'd_overlap_w': 0.0038, 'd_ttr': 0.2556, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5509, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'tha'): {'d_size': 5.2167, 'd_overlap_w': 0.0014, 'd_ttr': 5.8233, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4477, 'i_inv': 0.4526, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('hin', 'ind'): {'d_size': 4.6497, 'd_overlap_w': 0.0089, 'd_ttr': 0.1252, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.3433, 'i_inv': 0.4816, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'slk'): {'d_size': 3.27, 'd_overlap_w': 0.0042, 'd_ttr': 0.8613, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'swe'): {'d_size': 3.0135, 'd_overlap_w': 0.0061, 'd_ttr': 0.0818, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.5922, 'i_inv': 0.5473, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'por'): {'d_size': 2.7548, 'd_overlap_w': 0.0065, 'd_ttr': 0.0237, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.477, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.5}, ('hin', 'dan'): {'d_size': 2.3907, 'd_overlap_w': 0.0066, 'd_ttr': 0.038, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'lit'): {'d_size': 2.23, 'd_overlap_w': 0.004, 'd_ttr': 2.3281, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2952, 'i_inv': 0.5129, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'mkd'): {'d_size': 1.3477, 'd_overlap_w': 0.0045, 'd_ttr': 1.1198, 'd_overlap_s': '-', 'i_syn': 0.71, 'i_phon': 0.5922, 'i_inv': 0.558, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('hin', 'fin'): {'d_size': 1.2885, 'd_overlap_w': 0.0042, 'd_ttr': 9.6108, 'd_overlap_s': '-', 'i_syn': 0.54, 'i_phon': 0.3433, 'i_inv': 0.5129, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'mya'): {'d_size': 1.1436, 'd_overlap_w': 0.0028, 'd_ttr': 0.4659, 'd_overlap_s': '-', 'i_syn': 0.68, 'i_phon': 0.4208, 'i_inv': 0.4421, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hin', 'hye'): {'d_size': 1.1363, 'd_overlap_w': 0.0039, 'd_ttr': 2.3325, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.4717, 'i_inv': 0.5081, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'slv'): {'d_size': 1.055, 'd_overlap_w': 0.0049, 'd_ttr': 2.8954, 'd_overlap_s': '-', 'i_syn': 0.57, 'i_phon': 0.5922, 'i_inv': 0.4816, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'nob'): {'d_size': 0.8418, 'd_overlap_w': 0.0073, 'd_ttr': 0.7723, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.4097, 'i_inv': 0.4505, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.5}, ('hin', 'est'): {'d_size': 0.5712, 'd_overlap_w': 0.0043, 'd_ttr': 9.0478, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'kur'): {'d_size': 0.5517, 'd_overlap_w': 0.0017, 'd_ttr': 3.432, 'd_overlap_s': '-', 'i_syn': 0.75, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('hin', 'glg'): {'d_size': 0.5329, 'd_overlap_w': 0.0065, 'd_ttr': 2.3703, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.5922, 'i_inv': 0.5219, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('hin', 'mar'): {'d_size': 0.5235, 'd_overlap_w': 0.0873, 'd_ttr': 1.1489, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.398, 'i_fam': 0.625, 'i_geo': 0.0, 'i_fea': 0.5}, ('hin', 'tam'): {'d_size': 0.3311, 'd_overlap_w': 0.0054, 'd_ttr': 0.1227, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.5922, 'i_inv': 0.4661, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('hin', 'urd'): {'d_size': 0.318, 'd_overlap_w': 0.0056, 'd_ttr': 0.4875, 'd_overlap_s': '-', 'i_syn': 0.46, 'i_phon': 0.5922, 'i_inv': 0.3583, 'i_fam': 0.125, 'i_geo': 0.1, 'i_fea': 0.4}, ('hin', 'azb'): {'d_size': 0.3163, 'd_overlap_w': 0.0046, 'd_ttr': 17.7398, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.2871, 'i_inv': 0.6107, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('hin', 'bos'): {'d_size': 0.3013, 'd_overlap_w': 0.0052, 'd_ttr': 11.2349, 'd_overlap_s': '-', 'i_syn': 0.63, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'csl'): {'d_size': 0.2944, 'd_overlap_w': 0.0036, 'd_ttr': 160.3955, 'd_overlap_s': '-', 'i_syn': 0.58, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('hin', 'eus'): {'d_size': 0.2757, 'd_overlap_w': 0.0039, 'd_ttr': 13.5408, 'd_overlap_s': '-', 'i_syn': 0.49, 'i_phon': 0.2952, 'i_inv': 0.5327, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('hin', 'ben'): {'d_size': 0.2473, 'd_overlap_w': 0.0014, 'd_ttr': 1.5472, 'd_overlap_s': '-', 'i_syn': 0.53, 'i_phon': 0.3613, 'i_inv': 0.3481, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.4}, ('hin', 'bel'): {'d_size': 0.2399, 'd_overlap_w': 0.0038, 'd_ttr': 13.3747, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.5922, 'i_inv': 0.5363, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('hin', 'kaz'): {'d_size': 0.1765, 'd_overlap_w': 0.0037, 'd_ttr': 29.8132, 'd_overlap_s': '-', 'i_syn': 0.79, 'i_phon': 1.0, 'i_inv': 0.5363, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('hin', 'kat'): {'d_size': 0.7018, 'd_overlap_w': 0.0037, 'd_ttr': 8.9452, 'd_overlap_s': '-', 'i_syn': 0.41, 'i_phon': 0.4477, 'i_inv': 0.5088, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('hin', 'zsm'): {'d_size': 0.2777, 'd_overlap_w': 0.008, 'd_ttr': 1.578, 'd_overlap_s': '-', 'i_syn': 0.89, 'i_phon': 0.5922, 'i_inv': 0.5489, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.7}}, 'ben': {('ben', 'arb'): {'d_size': 46.0553, 'd_overlap_w': 0.0003, 'd_ttr': 0.1787, 'd_overlap_s': 0.0143, 'i_syn': 0.84, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.9}, ('ben', 'heb'): {'d_size': 45.5623, 'd_overlap_w': 0.0003, 'd_ttr': 0.2589, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5936, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.7}, ('ben', 'rus'): {'d_size': 44.8393, 'd_overlap_w': 0.0003, 'd_ttr': 0.2582, 'd_overlap_s': 0.0157, 'i_syn': 0.65, 'i_phon': 0.303, 'i_inv': 0.5962, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'kor'): {'d_size': 44.2332, 'd_overlap_w': 0.0002, 'd_ttr': 0.0179, 'd_overlap_s': 0.0127, 'i_syn': 0.55, 'i_phon': 0.5, 'i_inv': 0.5256, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('ben', 'ita'): {'d_size': 43.9886, 'd_overlap_w': 0.0007, 'd_ttr': 0.5689, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.6162, 'i_inv': 0.5625, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.7}, ('ben', 'jpn'): {'d_size': 43.8998, 'd_overlap_w': 0.0001, 'd_ttr': 34.2373, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.4327, 'i_inv': 0.4724, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.5}, ('ben', 'spa'): {'d_size': 42.1652, 'd_overlap_w': 0.0006, 'd_ttr': 0.5662, 'd_overlap_s': '-', 'i_syn': 0.72, 'i_phon': 0.4717, 'i_inv': 0.5849, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'fra'): {'d_size': 41.3646, 'd_overlap_w': 0.0008, 'd_ttr': 0.6511, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4407, 'i_inv': 0.4753, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.5}, ('ben', 'nld'): {'d_size': 39.5283, 'd_overlap_w': 0.0006, 'd_ttr': 0.485, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5193, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'tur'): {'d_size': 39.2493, 'd_overlap_w': 0.0003, 'd_ttr': 0.0573, 'd_overlap_s': 0.0204, 'i_syn': 0.68, 'i_phon': 0.3613, 'i_inv': 0.5667, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'ron'): {'d_size': 38.8221, 'd_overlap_w': 0.0006, 'd_ttr': 0.4235, 'd_overlap_s': 0.0196, 'i_syn': 0.69, 'i_phon': 0.3824, 'i_inv': 0.4997, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'pol'): {'d_size': 37.894, 'd_overlap_w': 0.0004, 'd_ttr': 0.1592, 'd_overlap_s': 0.02, 'i_syn': 0.67, 'i_phon': 0.303, 'i_inv': 0.5472, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'bul'): {'d_size': 37.5229, 'd_overlap_w': 0.0004, 'd_ttr': 0.367, 'd_overlap_s': '-', 'i_syn': 0.69, 'i_phon': 0.303, 'i_inv': 0.5383, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'vie'): {'d_size': 36.9961, 'd_overlap_w': 0.0018, 'd_ttr': 0.8678, 'd_overlap_s': 0.0207, 'i_syn': 0.82, 'i_phon': 0.4407, 'i_inv': 0.5242, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.6}, ('ben', 'deu'): {'d_size': 36.1127, 'd_overlap_w': 0.0005, 'd_ttr': 0.3763, 'd_overlap_s': 0.0203, 'i_syn': 0.61, 'i_phon': 0.4407, 'i_inv': 0.5404, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'pes'): {'d_size': 32.4726, 'd_overlap_w': 0.0006, 'd_ttr': 0.6026, 'd_overlap_s': 0.0159, 'i_syn': 0.75, 'i_phon': 0.3613, 'i_inv': 0.5182, 'i_fam': 0.75, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'hun'): {'d_size': 31.6668, 'd_overlap_w': 0.0003, 'd_ttr': 0.0175, 'd_overlap_s': 0.0196, 'i_syn': 0.62, 'i_phon': 0.3613, 'i_inv': 0.5319, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'srp'): {'d_size': 29.4468, 'd_overlap_w': 0.0004, 'd_ttr': 0.112, 'd_overlap_s': 0.0193, 'i_syn': 0.61, 'i_phon': 0.8485, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('ben', 'ell'): {'d_size': 28.8937, 'd_overlap_w': 0.0005, 'd_ttr': 0.3351, 'd_overlap_s': '-', 'i_syn': 0.73, 'i_phon': 0.3613, 'i_inv': 0.5893, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'hrv'): {'d_size': 26.2618, 'd_overlap_w': 0.0006, 'd_ttr': 0.1738, 'd_overlap_s': 0.0202, 'i_syn': 0.87, 'i_phon': 0.6162, 'i_inv': 0.5667, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.7}, ('ben', 'ukr'): {'d_size': 23.3373, 'd_overlap_w': 0.0004, 'd_ttr': 0.133, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5256, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'ces'): {'d_size': 22.1753, 'd_overlap_w': 0.0005, 'd_ttr': 0.1083, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.6162, 'i_inv': 0.5737, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'tha'): {'d_size': 21.0936, 'd_overlap_w': 0.0002, 'd_ttr': 0.2716, 'd_overlap_s': '-', 'i_syn': 0.81, 'i_phon': 0.2804, 'i_inv': 0.4147, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'ind'): {'d_size': 18.801, 'd_overlap_w': 0.0013, 'd_ttr': 0.507, 'd_overlap_s': 0.0205, 'i_syn': 0.76, 'i_phon': 0.4054, 'i_inv': 0.5144, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'slk'): {'d_size': 13.2222, 'd_overlap_w': 0.0006, 'd_ttr': 0.0198, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'swe'): {'d_size': 12.1848, 'd_overlap_w': 0.0008, 'd_ttr': 0.1822, 'd_overlap_s': 0.0194, 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.4906, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'por'): {'d_size': 11.139, 'd_overlap_w': 0.0008, 'd_ttr': 0.2359, 'd_overlap_s': 0.0195, 'i_syn': 0.74, 'i_phon': 0.6162, 'i_inv': 0.4991, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'dan'): {'d_size': 9.6666, 'd_overlap_w': 0.001, 'd_ttr': 0.2185, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.8}, ('ben', 'lit'): {'d_size': 9.0168, 'd_overlap_w': 0.0006, 'd_ttr': 0.0158, 'd_overlap_s': '-', 'i_syn': 0.66, 'i_phon': 0.3824, 'i_inv': 0.4886, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.5}, ('ben', 'mkd'): {'d_size': 5.4496, 'd_overlap_w': 0.0007, 'd_ttr': 0.0068, 'd_overlap_s': '-', 'i_syn': 0.77, 'i_phon': 0.6162, 'i_inv': 0.5978, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.7}, ('ben', 'fin'): {'d_size': 5.2102, 'd_overlap_w': 0.0005, 'd_ttr': 0.6844, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.4054, 'i_inv': 0.5071, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'mya'): {'d_size': 4.624, 'd_overlap_w': 0.0008, 'd_ttr': 0.0626, 'd_overlap_s': 0.0185, 'i_syn': 0.67, 'i_phon': 0.3743, 'i_inv': 0.4589, 'i_fam': 1.0, 'i_geo': 0.0, 'i_fea': 0.5}, ('ben', 'hye'): {'d_size': 4.5945, 'd_overlap_w': 0.0006, 'd_ttr': 0.016, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.5354, 'i_inv': 0.5625, 'i_fam': 0.875, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'slv'): {'d_size': 4.2656, 'd_overlap_w': 0.0007, 'd_ttr': 0.0416, 'd_overlap_s': 0.0189, 'i_syn': 0.64, 'i_phon': 0.6162, 'i_inv': 0.5332, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.6}, ('ben', 'hin'): {'d_size': 4.0435, 'd_overlap_w': 0.0014, 'd_ttr': 0.3073, 'd_overlap_s': 0.0165, 'i_syn': 0.53, 'i_phon': 0.3613, 'i_inv': 0.3481, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.4}, ('ben', 'nob'): {'d_size': 3.404, 'd_overlap_w': 0.0011, 'd_ttr': 0.0265, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.2048, 'i_inv': 0.4769, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.5}, ('ben', 'est'): {'d_size': 2.3097, 'd_overlap_w': 0.0008, 'd_ttr': 0.6181, 'd_overlap_s': '-', 'i_syn': 0.65, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.3, 'i_fea': 0.8}, ('ben', 'kur'): {'d_size': 2.2308, 'd_overlap_w': 0.0005, 'd_ttr': 0.0736, 'd_overlap_s': 0.0125, 'i_syn': 0.69, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.8}, ('ben', 'glg'): {'d_size': 2.1547, 'd_overlap_w': 0.0011, 'd_ttr': 0.0174, 'd_overlap_s': '-', 'i_syn': 0.78, 'i_phon': 0.6162, 'i_inv': 0.5182, 'i_fam': 0.875, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'mar'): {'d_size': 2.1166, 'd_overlap_w': 0.0011, 'd_ttr': 0.0059, 'd_overlap_s': 0.0146, 'i_syn': 0.5, 'i_phon': 0.6162, 'i_inv': 0.3908, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'tam'): {'d_size': 1.3388, 'd_overlap_w': 0.0013, 'd_ttr': 0.1586, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.6162, 'i_inv': 0.4661, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'urd'): {'d_size': 1.2857, 'd_overlap_w': 0.0015, 'd_ttr': 0.0591, 'd_overlap_s': '-', 'i_syn': 0.61, 'i_phon': 0.6162, 'i_inv': 0.3603, 'i_fam': 0.625, 'i_geo': 0.1, 'i_fea': 0.5}, ('ben', 'azb'): {'d_size': 1.279, 'd_overlap_w': 0.0012, 'd_ttr': 1.7496, 'd_overlap_s': '-', 'i_syn': 0.64, 'i_phon': 0.3613, 'i_inv': 0.6532, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'bos'): {'d_size': 1.2183, 'd_overlap_w': 0.0011, 'd_ttr': 0.8826, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.8}, ('ben', 'csl'): {'d_size': 1.1904, 'd_overlap_w': 0.001, 'd_ttr': 25.9063, 'd_overlap_s': '-', 'i_syn': 0.67, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'eus'): {'d_size': 1.1146, 'd_overlap_w': 0.001, 'd_ttr': 1.1785, 'd_overlap_s': '-', 'i_syn': 0.6, 'i_phon': 0.2163, 'i_inv': 0.5514, 'i_fam': 1.0, 'i_geo': 0.4, 'i_fea': 0.6}, ('ben', 'bel'): {'d_size': 0.9699, 'd_overlap_w': 0.001, 'd_ttr': 1.1567, 'd_overlap_s': 0.0137, 'i_syn': 0.76, 'i_phon': 0.6162, 'i_inv': 0.5983, 'i_fam': 0.875, 'i_geo': 0.3, 'i_fea': 0.9}, ('ben', 'kaz'): {'d_size': 0.7135, 'd_overlap_w': 0.0011, 'd_ttr': 3.5307, 'd_overlap_s': 0.0132, 'i_syn': 0.73, 'i_phon': 1.0, 'i_inv': 0.5983, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.9}, ('ben', 'kat'): {'d_size': 2.8378, 'd_overlap_w': 0.0007, 'd_ttr': 0.6062, 'd_overlap_s': '-', 'i_syn': 0.59, 'i_phon': 0.5032, 'i_inv': 0.5849, 'i_fam': 1.0, 'i_geo': 0.2, 'i_fea': 0.6}, ('ben', 'zsm'): {'d_size': 1.1228, 'd_overlap_w': 0.0018, 'd_ttr': 0.0, 'd_overlap_s': 0.0192, 'i_syn': 0.87, 'i_phon': 0.6162, 'i_inv': 0.5867, 'i_fam': 1.0, 'i_geo': 0.1, 'i_fea': 0.7}}}

##Lithuanian

In [ ]:
rank_dict = {}
for pair,values in filtered_lrl_distances['lit'].items():
  if pair[1] not in all_helpers_3code:
    continue
  for feat,value in values.items():
    if feat not in rank_dict.keys():
      rank_dict[feat] = []
    rank_dict[feat].append((value, pair[1]))

for feat, rank in rank_dict.items():
  print(feat, rank)
  if feat in ['d_size', 'd_overlap_w', 'd_overlap_s']:
    rank.sort(reverse=True)
  else:
    rank.sort()
  for tup in rank:
    #print(tup[1])
    print(tup)
  print('\n')

##Armenian

In [ ]:
rank_dict = {}
for pair,values in filtered_lrl_distances['hye'].items():
  if pair[1] not in all_helpers_3code:
    continue
  for feat,value in values.items():
    if feat not in rank_dict.keys():
      rank_dict[feat] = []
    rank_dict[feat].append((value, pair[1]))

for feat, rank in rank_dict.items():
  print(feat, rank)
  if feat in ['d_size', 'd_overlap_w', 'd_overlap_s']:
    rank.sort(reverse=True)
  else:
    rank.sort()
  for tup in rank:
    print(tup[1], tup[0])
  print('\n')

##Urdu

In [ ]:
rank_dict = {}
for pair,values in filtered_lrl_distances['urd'].items():
  if pair[1] not in all_helpers_3code:
    continue
  for feat,value in values.items():
    if feat not in rank_dict.keys():
      rank_dict[feat] = []
    rank_dict[feat].append((value, pair[1]))

for feat, rank in rank_dict.items():
  print(feat, rank)
  if feat in ['d_size', 'd_overlap_w', 'd_overlap_s']:
    rank.sort(reverse=True)
  else:
    rank.sort()
  for tup in rank:
    print(tup[1], tup[0])
  print('\n')

##Bengali

In [ ]:
rank_dict = {}
for pair,values in filtered_lrl_distances['ben'].items():
  if pair[1] not in all_helpers_3code:
    continue
  for feat,value in values.items():
    if feat not in rank_dict.keys():
      rank_dict[feat] = []
    rank_dict[feat].append((value, pair[1]))

for feat, rank in rank_dict.items():
  print(feat, rank)
  if feat in ['d_size', 'd_overlap_w', 'd_overlap_s']:
    rank.sort(reverse=True)
  else:
    rank.sort()
  for tup in rank:
    print(tup[1], tup[0])
  print('\n')

#LTR

##Filter only relevant dict items

In [ ]:
final_c_h_dict = {'lit':{}, 'hye':{}, 'urd':{}, 'ben':{}}
for child, pairs in filtered_lrl_distances.items():
  if child not in ['lit', 'hye', 'urd', 'ben']:
    continue

  for pair, dists in pairs.items():
    if pair[1] not in all_helpers_3code:
      continue
    final_c_h_dict[child][pair] = {}
    final_c_h_dict[child][pair] = dists

final_c_h_dict_sort = {'lit':{}, 'hye':{}, 'urd':{}, 'ben':{}}
for child,values in final_c_h_dict.items():
  sorted_child = {}
  for key in sorted(values):
    sorted_child[key] = final_c_h_dict[child][key]
  final_c_h_dict_sort[child]= sorted_child

##Obtain langrank train lists

###X_train

####All

In [ ]:
x_train_dct = {}
for child,values in final_c_h_dict_sort.items():
  x_train_dct[child] = {}
  for pair,feats in values.items():
    x_train_dct[child][pair] = []
    for feat, value in feats.items():
      x_train_dct[child][pair].append(value)

print(x_train_dct)

x_train_LOO_lit = []
for child,values in x_train_dct.items():
  if child == 'lit':
    continue
  for pair,feats in values.items():
    x_train_LOO_lit.append(feats)

x_train_LOO_hye = []
for child,values in x_train_dct.items():
  if child == 'hye':
    continue
  for pair,feats in values.items():
    x_train_LOO_hye.append(feats)

x_train_LOO_urd = []
for child,values in x_train_dct.items():
  if child == 'urd':
    continue
  for pair,feats in values.items():
    x_train_LOO_urd.append(feats)

x_train_LOO_ben = []
for child,values in x_train_dct.items():
  if child == 'ben':
    continue
  for pair,feats in values.items():
    x_train_LOO_ben.append(feats)

####Dependent

In [ ]:
x_train_dct = {}
for child,values in final_c_h_dict_sort.items():
  x_train_dct[child] = {}
  for pair,feats in values.items():
    x_train_dct[child][pair] = []
    for feat, value in feats.items():
      x_train_dct[child][pair].append(value)

print(x_train_dct)

x_train_LOO_lit = []
for child,values in x_train_dct.items():
  if child == 'lit':
    continue
  for pair,feats in values.items():
    x_train_LOO_lit.append(feats[:4])

x_train_LOO_hye = []
for child,values in x_train_dct.items():
  if child == 'hye':
    continue
  for pair,feats in values.items():
    x_train_LOO_hye.append(feats[:4])

x_train_LOO_urd = []
for child,values in x_train_dct.items():
  if child == 'urd':
    continue
  for pair,feats in values.items():
    x_train_LOO_urd.append(feats[:4])

x_train_LOO_ben = []
for child,values in x_train_dct.items():
  if child == 'ben':
    continue
  for pair,feats in values.items():
    x_train_LOO_ben.append(feats[:4])

####Independent

In [ ]:
x_train_dct = {}
for child,values in final_c_h_dict_sort.items():
  x_train_dct[child] = {}
  for pair,feats in values.items():
    x_train_dct[child][pair] = []
    for feat, value in feats.items():
      x_train_dct[child][pair].append(value)

x_train_LOO_lit = []
for child,values in x_train_dct.items():
  if child == 'lit':
    continue
  for pair,feats in values.items():
    x_train_LOO_lit.append(feats[4:])

x_train_LOO_hye = []
for child,values in x_train_dct.items():
  if child == 'hye':
    continue
  for pair,feats in values.items():
    x_train_LOO_hye.append(feats[4:])

x_train_LOO_urd = []
for child,values in x_train_dct.items():
  if child == 'urd':
    continue
  for pair,feats in values.items():
    x_train_LOO_urd.append(feats[4:])

x_train_LOO_ben = []
for child,values in x_train_dct.items():
  if child == 'ben':
    continue
  for pair,feats in values.items():
    x_train_LOO_ben.append(feats[4:])

x_train_all_childs = []
for child,values in x_train_dct.items():
  for pair,feats in values.items():
    x_train_all_childs.append(feats[4:])

###True rank

In [ ]:
lit_true = [('arb',9), ('bel',0), ('deu',2), ('hin',0), ('hrv',0), ('hun',0), ('ind',0), ('kaz',1), ('kor',5), ('kur',0), ('mar',0), ('mya',0), ('pes',0), ('pol',6), ('por',0), ('ron',3), ('rus',10), ('slv',0), ('srp',0), ('swe',4), ('tur',8), ('vie',7), ('zsm',0)]

hye_true = [('arb',0), ('bel',9), ('deu',0), ('hin',6), ('hrv',0), ('hun',0), ('ind',0), ('kaz',5), ('kor',4), ('kur',7), ('mar',0), ('mya',2), ('pes',1), ('pol',0), ('por',0), ('ron',0), ('rus',10), ('slv',2), ('srp',0), ('swe',0), ('tur',8), ('vie',0), ('zsm',3)]

urd_true = [('arb',0), ('bel',6), ('deu',3), ('hin',10), ('hrv',0), ('hun',0), ('ind',0), ('kaz',0), ('kor',7), ('kur',0), ('mar',8), ('mya',1), ('pes',0), ('pol',5), ('por',0), ('ron',0), ('rus',6), ('slv',4), ('srp',2), ('swe',0), ('tur',9), ('vie',0), ('zsm',0)]

ben_true = [('arb',0), ('bel',10), ('deu',0), ('hin',9), ('hrv',4), ('hun',0), ('ind',0), ('kaz',0), ('kor',2), ('kur',0), ('mar',0), ('mya',7), ('pes',3), ('pol',0), ('por',0), ('ron',0), ('rus',0), ('slv',8), ('srp',0), ('swe',5), ('tur',1), ('vie',0), ('zsm',6)]

###Y_train

In [ ]:
y_true = []
for child_scores in lit_true, hye_true, urd_true, ben_true:
  child_truth = []
  for tup in child_scores:
    child_truth.append(tup[1])
  y_true.append(child_truth)

y_train_LOO_lit = []
for child_scores in hye_true, urd_true, ben_true:
  for tup in child_scores:
    y_train_LOO_lit.append(tup[1])

y_train_LOO_hye = []
for child_scores in lit_true, urd_true, ben_true:
  for tup in child_scores:
    y_train_LOO_hye.append(tup[1])

y_train_LOO_urd = []
for child_scores in lit_true, hye_true, ben_true:
  for tup in child_scores:
    y_train_LOO_urd.append(tup[1])

y_train_LOO_ben = []
for child_scores in lit_true, hye_true, urd_true:
  for tup in child_scores:
    y_train_LOO_ben.append(tup[1])

y_train_all_childs = []
for child_scores in lit_true, hye_true, urd_true, ben_true:
  for tup in child_scores:
    y_train_all_childs.append(tup[1])

##LGBMRanker (train)

###Lithuanian

In [ ]:
model = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=8,
						   max_depth=-1, learning_rate=0.1, n_estimators=100,
						   min_child_samples=5, min_split_gain = 1, min_data_in_leaf = 1, feature_fraction = 0.8)

model.fit(x_train_LOO_lit, y_train_LOO_lit, group=[23,23,23])

model.booster_.save_model('LOO_lit.pt')

###Armenian

In [ ]:
model = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=8,
						   max_depth=-1, learning_rate=0.1, n_estimators=100,
						   min_child_samples=5, min_split_gain = 1, min_data_in_leaf = 1, feature_fraction = 0.8)

model.fit(x_train_LOO_hye, y_train_LOO_hye, group=[23,23,23])

model.booster_.save_model('LOO_hye.pt')

###Urdu

In [ ]:
model = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=8,
						   max_depth=-1, learning_rate=0.1, n_estimators=100,
						   min_child_samples=5, min_split_gain = 1, min_data_in_leaf = 1, feature_fraction = 0.8)

model.fit(x_train_LOO_urd, y_train_LOO_urd, group=[23,23,23])

model.booster_.save_model('LOO_urd.pt')

###Bengali

In [ ]:
model = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=8,
						   max_depth=-1, learning_rate=0.1, n_estimators=100,
						   min_child_samples=5, min_split_gain = 1, min_data_in_leaf = 1, feature_fraction = 0.8)

model.fit(x_train_LOO_ben, y_train_LOO_ben, group=[23,23,23])

model.booster_.save_model('LOO_ben.pt')

###All childs

In [ ]:
model = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=8,
						   max_depth=-1, learning_rate=0.1, n_estimators=100,
						   min_child_samples=5, min_split_gain = 1, min_data_in_leaf = 1, feature_fraction = 0.8)

model.fit(x_train_all_childs, y_train_all_childs, group=[23,23,23,23])

model.booster_.save_model('LOO_all_childs_indep.pt')

##LGBMRanker (predict)

###Test on all

In [ ]:
x_test_lit = []
x_test_hye = []
x_test_urd = []
x_test_ben = []

for child,values in x_train_dct.items():
  if child == 'lit':
    for pair,feats in values.items():
      x_test_lit.append(feats)
  if child == 'hye':
    for pair,feats in values.items():
      x_test_hye.append(feats)
  if child == 'urd':
    for pair,feats in values.items():
      x_test_urd.append(feats)
  if child == 'ben':
    for pair,feats in values.items():
      x_test_ben.append(feats)

###Test on dependent

In [ ]:
x_test_lit = []
x_test_hye = []
x_test_urd = []
x_test_ben = []

for child,values in x_train_dct.items():
  if child == 'lit':
    for pair,feats in values.items():
      x_test_lit.append(feats[:4])
  if child == 'hye':
    for pair,feats in values.items():
      x_test_hye.append(feats[:4])
  if child == 'urd':
    for pair,feats in values.items():
      x_test_urd.append(feats[:4])
  if child == 'ben':
    for pair,feats in values.items():
      x_test_ben.append(feats[:4])

###Test on independent

In [ ]:
x_test_lit = []
x_test_hye = []
x_test_urd = []
x_test_ben = []

for child,values in x_train_dct.items():
  if child == 'lit':
    for pair,feats in values.items():
      x_test_lit.append(feats[4:])
  if child == 'hye':
    for pair,feats in values.items():
      x_test_hye.append(feats[4:])
  if child == 'urd':
    for pair,feats in values.items():
      x_test_urd.append(feats[4:])
  if child == 'ben':
    for pair,feats in values.items():
      x_test_ben.append(feats[4:])

###Lithuanian

In [ ]:
bst = lgb.Booster(model_file='LOO_lit.pt')
predict_contribs = bst.predict(x_test_lit, pred_contrib=True)
predict_scores = predict_contribs.sum(-1)
print(predict_scores)
print(predict_scores.max())
print(predict_scores[16])
print(predict_contribs[16])
ind = list(np.argsort(-predict_scores))

lr_all_y_true_lit = []
lr_all_score_lit = []
for j,i in enumerate(ind):
  if 10-j <= 0:
    rank = 0
  else:
    rank = 10-j
  print("%d. %s : score=%.2f, true rank: %s" % (rank, lit_true[i][0], predict_scores[i], lit_true[i][1]))

  lr_all_y_true_lit.append(lit_true[i][1])
  lr_all_score_lit.append(rank)

###Armenian

In [ ]:
bst = lgb.Booster(model_file='LOO_hye.pt')
predict_contribs = bst.predict(x_test_hye, pred_contrib=True)
predict_scores = predict_contribs.sum(-1)
ind = list(np.argsort(-predict_scores))
 
lr_all_y_true_hye = []
lr_all_score_hye = []
for j,i in enumerate(ind):
  if 10-j <= 0:
    rank = 0
  else:
    rank = 10-j
  print("%d. %s : score=%.2f, true rank: %s" % (rank, hye_true[i][0], predict_scores[i], hye_true[i][1]))
  
  lr_all_y_true_hye.append(hye_true[i][1])
  lr_all_score_hye.append(rank)

###Urdu

In [ ]:
bst = lgb.Booster(model_file='LOO_urd.pt')
predict_contribs = bst.predict(x_test_urd, pred_contrib=True)
predict_scores = predict_contribs.sum(-1)
print(predict_scores)
print(predict_scores[3])
print(predict_contribs[3])
print(predict_scores[10])
print(predict_contribs[10])
ind = list(np.argsort(-predict_scores))

lr_all_y_true_urd = []
lr_all_score_urd = []
for j,i in enumerate(ind):
  if 10-j <= 0:
    rank = 0
  else:
    rank = 10-j
  print("%d. %s : score=%.2f, true rank: %s" % (rank, urd_true[i][0], predict_scores[i], urd_true[i][1]))

  lr_all_y_true_urd.append(urd_true[i][1])
  lr_all_score_urd.append(rank)

###Bengali

In [ ]:
bst = lgb.Booster(model_file='LOO_ben.pt')
predict_contribs = bst.predict(x_test_ben, pred_contrib=True)
predict_scores = predict_contribs.sum(-1)
ind = list(np.argsort(-predict_scores))

lr_all_y_true_ben = []
lr_all_score_ben = []
for j,i in enumerate(ind):
  if 10-j <= 0:
    rank = 0
  else:
    rank = 10-j
  print("%d. %s : score=%.2f, true rank: %s" % (rank, ben_true[i][0], predict_scores[i], ben_true[i][1]))

  lr_all_y_true_ben.append(ben_true[i][1])
  lr_all_score_ben.append(rank)

##NDCG

In [ ]:
print('*** Rank using LangRank***')

lr_all_y_true_total = [lr_all_y_true_lit, lr_all_y_true_hye, lr_all_y_true_urd, lr_all_y_true_ben]
lr_all_score_total = [lr_all_score_lit, lr_all_score_hye, lr_all_score_urd, lr_all_score_ben]

print('ranking algorithm: ', round(metrics.ndcg_score(lr_all_y_true_total, lr_all_score_total, k=3, sample_weight=None, ignore_ties=False), 4))

*** Rank using LangRank***
ranking algorithm:  0.4983
